In [4]:
import requests
from bs4 import BeautifulSoup

def get_recipe_links(base_url):
    recipe_links = []
    start = 0  # Starting index for pagination
    max_iterations = 1000  # Prevent infinite loops by limiting iterations
    iterations = 0

    while True:
        if iterations >= max_iterations:
            print("Reached maximum iterations. Exiting to prevent infinite loop.")
            break

        current_url = f"{base_url}?start={start}"
        try:
            response = requests.get(current_url)
            response.raise_for_status()  
            soup = BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {current_url}: {e}")
            break  # Exit if there was an error

        found_links = False
        for link in soup.find_all('a', href=True):
            if '/recipes/' in link['href']:
                absolute_url = f"https://www.sainsburysmagazine.co.uk{link['href']}"
                if absolute_url not in recipe_links:  #Avoid duplicates
                    recipe_links.append(absolute_url)
                    found_links = True

        print(f"Fetched {len(recipe_links)} links from {current_url}")

        #Increment start for the next load
        start += 9  
        iterations += 1  

        if not found_links:
            print("No new links found. Exiting.")
            break

    return recipe_links  


if __name__ == "__main__":
    base_url = 'https://www.sainsburysmagazine.co.uk/recipes/mains'
    recipe_links = get_recipe_links(base_url)

    for url in recipe_links:
        print(url)

    with open('recipe_links.txt', 'w', encoding='utf-8') as f:
        for url in recipe_links:
            f.write(url + '\n')

    print(f"Total recipe links collected: {len(recipe_links)}")


Fetched 133 links from https://www.sainsburysmagazine.co.uk/recipes/mains?start=0
Fetched 133 links from https://www.sainsburysmagazine.co.uk/recipes/mains?start=9
No new links found. Exiting.
https://www.sainsburysmagazine.co.uk/recipes/category/course
https://www.sainsburysmagazine.co.uk/recipes/breakfasts
https://www.sainsburysmagazine.co.uk/recipes/canapes
https://www.sainsburysmagazine.co.uk/recipes/desserts
https://www.sainsburysmagazine.co.uk/recipes/drinks
https://www.sainsburysmagazine.co.uk/recipes/mains
https://www.sainsburysmagazine.co.uk/recipes/sides
https://www.sainsburysmagazine.co.uk/recipes/starters
https://www.sainsburysmagazine.co.uk/recipes/category/baking
https://www.sainsburysmagazine.co.uk/recipes/birthday-cakes
https://www.sainsburysmagazine.co.uk/recipes/bread
https://www.sainsburysmagazine.co.uk/recipes/brownies
https://www.sainsburysmagazine.co.uk/recipes/cakes
https://www.sainsburysmagazine.co.uk/recipes/cheesecakes
https://www.sainsburysmagazine.co.uk/reci

In [8]:
# import requests
# from bs4 import BeautifulSoup
# headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
#         }
# base_url = 'https://www.sainsburysmagazine.co.uk/api/tags/12/recipes/2.json'
# page = 1

# links =[]
# while True:
#     current_url = f"{base_url}{page}.json"
#     response = requests.get(current_url, headers=headers)
#     print(response.status_code)
#     break;



404


In [12]:
import requests
from bs4 import BeautifulSoup

# Base URL for Sainsbury's Magazine recipes
base_url = 'https://www.sainsburysmagazine.co.uk/recipes/mains'

# Initialize a list to store all recipe links
all_links = []

# Send a request to the main page
response = requests.get(base_url)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all <a> tags with the desired structure
    recipe_links = soup.find_all('a', href=True)  # Find all <a> tags with href attributes
    
    for link in recipe_links:
        # Check if the href contains the desired pattern for recipes
        href = link['href']
        if '/recipes/mains/' in href:
            full_link = f"https://www.sainsburysmagazine.co.uk{href}"  # Construct the full URL
            all_links.append(full_link)

    # Print all collected recipe links
    print("Collected Recipe Links:")
    for link in all_links:
        print(link)
else:
    print(f"Failed to retrieve the main page: {response.status_code}")


Collected Recipe Links:
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot


In [16]:
import requests

# API URL format
api_url_format = 'https://www.sainsburysmagazine.co.uk/api/tags/12/recipes/{}.json'

# Initialize a list to store all recipe links
all_links = []

# Loop through the page numbers until 239 (since 238 is the last page for a total of 2139 recipes)
for page_number in range(1, 239):
    # Construct the API URL for the current page
    api_url = api_url_format.format(page_number)
    
    # Make a request to the API endpoint
    response = requests.get(api_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Check if the response contains recipes
        recipes = data.get('recipes', [])
        for recipe in recipes:
            # Extract the recipe link (assuming the key is 'url' or adjust as per the actual response)
            link = recipe.get('url')  # Assuming there's a URL key for each recipe
            if link:
                # Make sure to construct the full URL if it's a relative link
                full_link = f"https://www.sainsburysmagazine.co.uk{link}"
                all_links.append(full_link)
    else:
        print(f"Failed to retrieve data for page {page_number}: {response.status_code}")

# Print all collected recipe links
print(f"Total collected recipe links: {len(all_links)}")
for link in all_links:
    print(link)


Total collected recipe links: 0


In [21]:
import requests

# API URL format for pagination
api_url_format = 'https://www.sainsburysmagazine.co.uk/api/tags/12/recipes/{}.json'

# Loop through the first few pages to check the JSON response
for page_number in range(1, 4):  # Change to range(1, 239) later
    # Construct the API URL for the current page
    api_url = api_url_format.format(page_number)

    # Make a request to the API endpoint
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()
        
        # Print the JSON response for inspection
        print(f"Page {page_number} Response:")
        print(data)  # Print the full response for the first few pages
    else:
        print(f"Failed to retrieve data for page {page_number}: {response.status_code}")


Page 1 Response:
{'limit': 9, 'total': 2139, 'totalPages': 268, 'isLastPage': False, 'html': '<div class="teaser">\n    <div class="inner">\n        <a href="/recipes/mains/sausage-pale-ale-and-bean-soup">\n            <div class="picture-column"><div class="picture picture-115">\n        <picture><img data-src="/uploads/media/350x400/06/16966-sausage-pale-ale-and-bean-soup.jpg?v=1-0" alt="Sausage, pale-ale and bean soup">\n        </picture>\n    </div></div><div class="details"><h5>Sausage, pale-ale and bean soup</h5><div class="intro">This nourishing bowl of soup can be made without the booze – just replace the ale with additional stock and make sure to season well</div><p>Recipe by Nadine Brown</p><div class="recipeDetails"><div>\n                                <img data-src="/assets/main/images/icons/time.svg" alt="time">\n                                <time datetime="PT0H25M">25 mins</time>\n                            </div><div>Serves: 2</div><div><div class="rating">Rating:

Page 2 Response:
{'limit': 9, 'total': 2139, 'totalPages': 268, 'isLastPage': False, 'html': '<div class="teaser">\n    <div class="inner">\n        <a href="/recipes/mains/dhal-with-crispy-kale">\n            <div class="picture-column"><div class="picture picture-115">\n        <picture><img data-src="/uploads/media/350x400/08/16858-dhal-with-crispy-kale.jpg?v=1-0" alt="Dhal with crispy kale">\n        </picture>\n    </div></div><div class="details"><h5>Dhal with crispy kale</h5><div class="intro">Dhals are popular in Sri Lanka and are usually pretty quick and easy to make. The crispy kale adds a lovely crunch, while the creamy coconut milk helps balance the heat from the fresh and dried chillies.</div><p>Recipe by Kolamba</p><div class="recipeDetails"><div>\n                                <img data-src="/assets/main/images/icons/time.svg" alt="time">\n                                <time datetime="PT0H55M">55 mins</time>\n                            </div><div>Serves: 4-6 as a sh

Page 3 Response:
{'limit': 9, 'total': 2139, 'totalPages': 238, 'isLastPage': False, 'html': '<div class="teaser">\n    <div class="inner">\n        <a href="/recipes/mains/frying-pan-meatball-moussaka">\n            <div class="picture-column"><div class="picture picture-115">\n        <picture><img data-src="/uploads/media/350x400/07/10137-Frying-pan-moussaka.jpg?v=1-0" alt="Frying pan ‘meatball moussaka’">\n        </picture>\n    </div></div><div class="details"><h5>Frying pan ‘meatball moussaka’</h5><div class="intro">This tasty midweek meal uses frozen vegetables and a pack of meatballs to save time and have dinner on the table in 30 minutes  </div><p>Recipe by Abigail Spooner</p><div class="recipeDetails"><div>\n                                <img data-src="/assets/main/images/icons/time.svg" alt="time">\n                                <time datetime="PT0H30M">30 mins</time>\n                            </div><div>Serves: 4</div><div><div class="rating">Rating:<span class="rat

In [25]:
import requests
from bs4 import BeautifulSoup

# API URL format for pagination
api_url_format = 'https://www.sainsburysmagazine.co.uk/api/tags/12/recipes/{}.json'

# List to store all collected recipe links
all_links = []

# Loop through all pages to collect recipe links
total_pages = 238  # As per the JSON response
for page_number in range(1, total_pages + 1):
    # Construct the API URL for the current page
    api_url = api_url_format.format(page_number)

    # Make a request to the API endpoint
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()
        
        # Extract the HTML content from the response
        html_content = data['html']

        # Use BeautifulSoup to parse the HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all <a> tags that contain the recipe links
        links = soup.find_all('a', href=True)

        # Collect the href attributes
        for link in links:
            href = link['href']
            if href.startswith('/recipes/mains/'):  # Ensure it's a recipe link
                full_link = f'https://www.sainsburysmagazine.co.uk{href}'  # Construct full URL
                all_links.append(full_link)

# Print the total collected recipe links
print(f'Total collected recipe links: {len(all_links)}')
print(all_links)  # Optional: print the collected links


Total collected recipe links: 1713
['https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dhal-with-crispy-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pork-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains

In [26]:
# Define your list
my_list = ['https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dhal-with-crispy-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pork-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-salmon-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quick-lamb-biryani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-toad-in-the-hole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/boiled-gammon-with-pease-pudding', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-frying-pan-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-green-prawn-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-meatball-moussaka', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-tagine-and-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-salty-lentil-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seared-horseradish-steak-with-red-wine-and-rosemary-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-cheesy-puy-lentil-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-stroganoff-jacket-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/30-minute-spicy-prawn-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lazy-caramelised-onion-sausage-hotpot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/flatbreads-with-spiced-salmon-and-zesty-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-smoky-corn-and-peppers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-chickpea-and-kale-frittata-with-tomato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/garlic-butter-chicken-with-cheesy-cauli-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-buttermilk-marinated-spatchcock-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/super-quick-cook-steak-skewers-with-chimichurri', 'https://www.sainsburysmagazine.co.uk/recipes/mains/all-american-ribs-with-sticky-bourbon-glaze', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beetroot-greens-and-brie-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-with-charred-sweetcorn-salsa-and-fresh-mint-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/yours-and-mine-seafood-boils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/everything-seasoning-chicken-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-lemonade-chicken-with-rainbow-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/triple-citrus-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/next-level-hot-dogs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hot-honey-barbecue-platter', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sicilian-style-mackerel-with-white-beans-and-caponata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dolcelatte-prosciutto-and-pickled-peach-pizza-with-hot-honey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-courgette-and-butter-bean-braise', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broad-bean-dill-and-hazelnut-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-courgette-involtini', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-summer-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-chicken-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-tuna-kebab', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-loin-tikka-paratha-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-mango-and-cashew-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-gentlemans-relish-galette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-beef-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-carrot-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-chicken-souvlaki', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-cheeseburger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/glazed-pork-with-mango-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/houmous-flatbread-pizzas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-saltimbocca-burger-with-crispy-prosciutto-and-parmesan-mayo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smash-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-chicken-milanese-with-shaved-asparagus-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tortelloni-with-minty-pea-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawns-with-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-sea-bass-larb', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-and-chip-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gazpacho', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spice-crusted-tuna-with-polyphenol-rich-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-rice-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-and-roasted-cauliflower-with-herby-bulgur-wheat-and-tahini', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-poke-recipe-tofu-avocado', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-and-goats-cheese-open-sandwich', 'https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-style-egg-mayo-sandos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-herby-yogurt-and-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-in-a-basket', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hasselback-halloumi-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-pineapple-skewers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pigs-in-blankets-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-steak-tacos-with-cucumber-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-gyoza-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vibrant-watercress-and-pea-soup-with-bacon-crumbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-bream-with-pickled-radish-and-pea-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-saffron-chicken-white-bean-and-chorizo-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-warm-salmon-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-lentil-and-roast-tomato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-and-bean-mash-with-cheesy-fennel', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dill-and-feta-rice-with-crispy-fried-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-puff-plait', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-tenderloin-with-apples-and-walnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-lemon-olive-butter-and-new-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-stuffed-aubergines', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rack-of-lamb-with-anchovy-salsa-and-spring-vegetables', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-tomato-chicken-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-moussaka', 'https://www.sainsburysmagazine.co.uk/recipes/mains/impossible-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sweet-chilli-halloumi-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sesame-beef-and-spring-onion-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kale-and-chickpea-linguine-with-a-nutty-crunch', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-trout-and-pea-cacio-e-pepe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-greens-and-white-bean-rigatoni', 'https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-chicken-with-pepper-caprese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-beef-stifado', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-brisket-and-black-bean-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/garlicky-asparagus-carbonara', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-minced-beef-and-mushroom-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-tofu-with-zingy-lime-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/egyptian-filo-pie-with-chopped-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-sweetcorn-brunch-fritters', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-oven-baked-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jersey-royal-and-crab-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pub-style-steak-sarnie-with-beer-onions', 'https://www.sainsburysmagazine.co.uk/recipes/mains/silk-handkerchiefs-with-smoked-beurre-blanc', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-satay-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-greens-shakshuka', 'https://www.sainsburysmagazine.co.uk/recipes/mains/triple-mustard-roast-beef-and-yorkshires', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-with-spring-veg-and-dill-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-al-limone', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chilli-con-carne-from-knorr-stock-pots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-knorr-spaghetti-bolognese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-lamb-with-mustardy-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butter-basted-sage-pork-chops-with-beetroot-puree-and-garlic-chard', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-chicken-livers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/parma-chicken-schnitz-with-cheesy-kale-and-celeriac-fries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/noodle-omelette-with-sweet-chilli-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-easy-pea-and-seafood', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-and-potatoes-with-preserved-lemon-aioli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hasselback-roast-squash-with-whipped-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-roast-lamb-with-provencal-panzanella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-flatbread-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-kleftiko', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-pillows-in-silky-green-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-lamb-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chilli-with-squash-and-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/welsh-rarebit-galette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-bacon-and-white-bean-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/honey-mustard-chicken-and-apples', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-mushrooms-with-chimichurri-parsnip-and-butter-bean-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-citrus-pork', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tartiflette-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-sweet-potato-succotash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-gnocchi-and-sausage-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-sauerkraut-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-lamb-meatballs-in-a-creamy-porcini-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-short-rubs-with-sweet-potato-mash-and-pickled-walnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/surf-and-turf-skillet-with-garlic-dippers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roast-cacio-e-pepe-cauliflower', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-tofu-goulash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chunky-chorizo-and-butter-bean-soup-1', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-coconut-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-spinach-and-mushroom-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-mushroom-grains-with-date-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-lamb-chops-with-sweet-potato-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-and-sausages-with-pickled-peppers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/colcannon-smoked-fish-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-aubergine-steak-with-creamed-corn-and-a-herby-green-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herby-meatballs-with-broccoli-pesto-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pie-and-liquor', 'https://www.sainsburysmagazine.co.uk/recipes/mains/three-mushroom-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-with-sausage-and-chicory', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-and-apple-soup-with-horseradish-dill-dumplings', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-socca-pancake-with-smoked-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chunky-chorizo-and-butter-bean-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/stove-top-mac-and-cheese-with-pancetta-crumbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beigels-or-bagels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-bhuna', 'https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-risotto-with-celery-and-tarragon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tahini-chicken-greens-and-beans', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-cod-patatas-bravas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/milk-braised-pork-steaks-with-cabbage', 'https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-sausage-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-brunch-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-salmon-and-vegetable-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/emerald-dhal-with-golden-tofu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tofu-and-greens-noodle-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gingered-mushroom-super-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-cajun-cauli-tacos-and-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-cauli-and-sweet-potato-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seared-salmon-with-tarragon-red-wine-sauce-and-braised-chicory', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-wine-vegan-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/double-dip-buttermilk-fried-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-and-paneer-torkari', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-pot-pie-with-puff-pastry-croutons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-chicken-with-citrus-and-lavender', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-mushrooms-madeira-sauce-and-celeriac', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-and-sprout-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-cheesy-shallot-tarte-tatin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/double-chestnut-mushroom-wellington', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-and-confit-duck-with-bubble-and-squeak-cakes-cranberry-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bigos-hunters-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-enchilada-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/root-veggie-nut-roast-with-chestnut-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-and-ginger-jam-glazed-ham-with-nutmeg-and-sweet-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-veggie-tarte-tatin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/winter-chicken-caesar', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-smoked-salmon-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-parsnip-and-cashew-strudels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-ale-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brussels-sprout-chestnut-and-bacon-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-pork-and-potato-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-pasta-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zesty-salmon-and-roasted-broccoli-bulgur', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hariyali-guinea-fowl-with-roasted-onions', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cider-poached-cod-with-spiced-mussels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/campfire-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cider-pulled-pork-and-campfire-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pumpkin-soup-with-nutty-bacon-crumble', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cobweb-pancakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-and-pear-soup-with-blue-cheese-toasts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-cabbage-rolls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/root-vegetable-and-coconut-fish-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-mozzarella-sausage-and-sweet-pepper-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-pithivier', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-harissa-lasagne-with-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-braised-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tagliatelle-with-walnut-and-kale-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/maple-mustard-ham-and-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/traditional-welsh-rarebit', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-rice-and-mussels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-tarragon-roast-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-potatoes-and-plums-with-marsala-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/negroni-chicken-skewers-with-fennel-and-orange-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-chicken-and-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-cod-parcels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summery-spaghetti-and-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/easy-honey-garlic-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-and-chickpeas-en-papillote', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asian-style-prawn-and-grapefruit-salad-with-crispy-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ribs-red-beans-and-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crackling-good-pork-and-apple-traybake-with-crunchy-pickle-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-steak-and-blue-cheese-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-style-lamb-with-warm-houmous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-pork-belly-with-pickled-plums', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-chicken-summer-tomato-and-basil-panzanella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-crusted-lamb-with-green-quinoa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/golden-coconut-turkey-and-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-porcini-burger-with-truffle-mayo-and-crispy-onion-rings', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-tenderstem-broccoli-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-chickpea-and-chorizo-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/parmesan-chicken-butties-with-red-cabbage-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/taco-pizza-flatbreads-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/marinated-halloumi-with-honey-and-thyme', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-shoulder', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-feta-pizza-with-hot-honey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-salmon-in-chard-with-caramelised-lemons-garlic-and-almond-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-squash-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-prawn-burger-with-sriracha-butter', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lime-and-coconut-pork-belly-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/feta-stuffed-peppers-with-butter-bean-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bedfordshire-clangers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/the-high-steaks-sharing-board', 'https://www.sainsburysmagazine.co.uk/recipes/mains/afternoon-sea-sharing-board', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peperonata-bucatini-with-sardines', 'https://www.sainsburysmagazine.co.uk/recipes/mains/green-couscous-with-torn-tahini-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/wensleydale-and-apple-tart-with-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-salmon-kedgeree', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-lamb-burger-with-pickled-red-cabbage-chilli-and-garlic-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-prawn-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-bianca-with-ham-and-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-and-crispy-bean-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-tofu-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/runner-bean-linguine-with-ricotta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/arabic-style-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-and-butter-bean-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fennel-olive-and-tomato-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-sea-bass-with-pesto-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/minty-lamb-with-rainbow-veg-and-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-shawarma-picnic-loaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mango-chicken-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ham-and-roasted-pepper-tortilla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-basa-with-smoky-tomato-beans', 'https://www.sainsburysmagazine.co.uk/recipes/mains/green-goddess-salmon-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/honey-mustard-bacon-steaks-with-sweetcorn-crush', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-kipper-and-potato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/feelgood-fish-and-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-hawaiian-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-chicken-caprese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/flavour-bomb-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sherry-braised-chicken-with-chorizo-and-chickpeas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-asparagus-and-bean-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-tarte-tatin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lancashire-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/massaman-beef-and-aubergine-skewers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chargrilled-chicken-with-zingy-coriander-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-paprikash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-beans-on-toast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-roast-lamb', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-beef-chow-mein', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-cutlets-with-pearl-barley', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-nova-scotia-lobster-roll', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bobotie-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-satay-sitr-fry-with-a-sweet-sour-radish-pickle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-chicken-roast-dinner', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-roasted-garlic-and-burnt-spring-onion-butter', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-spatchcock-poussins-with-barbecued-peach-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-portobello-veggie-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-ham-and-leek-filo-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-broccoli-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-en-papillote-with-caponata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-squash-and-salsa-di-noci', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-broccoli-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pea-ham-and-mint-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-bacon-and-corn-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-cauliflower-biryani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-with-figs-and-shallots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tex-mex-5-a-day-veggie-feast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/corned-beef-hash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/popeye-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-carrot-and-lentil-soup-with-bhajia-nuggets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-lamb-stir-fry-with-half-the-carbs-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-and-cheese-meat-pies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lancashire-cheese-leek-and-onion-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-smoked-fish-and-spinach-tagliatelle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/meaty-mushroom-pho', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-meatball-hotpot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rice-stuffed-onions-with-feta-and-dill', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-patties-with-nutty-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-fillet-with-garlicky-mushroom-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-gnocchi-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veg-and-tofu-spicy-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-sausage-and-cabbage-saute', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-and-ginger-beef-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/super-greens-thai-green-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-tomatoes-with-broccoli-smash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/marmalade-sausages-with-parsnip-colcannon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lama-keema', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-swedish-style-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/clementine-cumberland-glazed-ham', 'https://www.sainsburysmagazine.co.uk/recipes/mains/patatas-bravas-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-fish-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-satay-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-beef-bourguignon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cullen-skink', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-stir-fry-with-quorn-and-plums', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broken-lasagne-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/low-waste-butternut-squash-and-sage-rigatoni', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooker-golden-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-mushroom-and-leek-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-coconut-and-lentil-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-teriyaki-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-spaghetti-with-crispy-capers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/curried-turkey-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-leftover-turkey-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leftovers-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/poussins-with-potato-celeriac-and-porcini-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-turkey-minestrone', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tinned-fish-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-sprout-colcannon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-cod-in-prosciutto-di-parma-with-green-pea-puree', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sage-and-apricot-stuffed-pork-tenderloin-with-roasted-pear-parsnip-and-potato', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cider-chicken-casserole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cinnamon-and-saffron-lamb-shank-with-roasted-roots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-cod-with-sake-shiitake-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ipa-casserole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-chestnut-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-thighs-with-mustardy-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-potato-cakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-katsu-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chestnut-and-mushroom-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/truffled-mushroom-soup-with-cheesy-croutes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-meatball-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-gammon-stew-with-buttermilk-dumplings', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-and-garlic-butter-turkey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-and-pesto-traybake-from-sacla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/savoy-cabbage-gratin-with-herby-cheddar-crumbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chickpea-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-steak-with-cheesy-creamed-corn', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-halloumi-and-spinach-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/liver-bacon-and-onions-with-white-bean-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-restaurant-style-butter-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-paneer-and-pea-biryani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-laksa-with-spicy-sambal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/philly-cheeseteak-stuffed-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-chicken-tortellini-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-kale-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-green-peas-and-coconut-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilean-cottage-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-cheese-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-melt-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/murgh-makhani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-barley-one-pot-with-zesty-gremolata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chilli-prawn-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-arrabiata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-pesto-pasta-alla-norma-with-sacla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweetcorn-and-bacon-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jaffna-lamb-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-smoky-bacon-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-basa-with-sweetcorn-pilau', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-harissa-and-marmalade-baked-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-beetroot-pesto-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-dill-cucumber-yogurt-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-chorizo-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-creamed-corn', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-halloumi-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-and-stilton-crumble', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-meatball-red-pepper-and-feta-rigatoni-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-clonakilty-blackpudding-stuffed-peppers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-crispy-filo-feta-and-clonakilty-blackpudding-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-burgers-with-apple-and-fennel', 'https://www.sainsburysmagazine.co.uk/recipes/mains/oven-roasted-jerk-pork', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-lamb-filo-pie-with-chermoula-drizzle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-bacon-and-tomato-toastie-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ham-hock-and-pea-fritters', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-corn-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caprese-grain-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-cheeseboard-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smashed-broad-bean-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-egg-and-brown-rice-bowls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-butter-salmon-and-red-pepper-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-cashew-massaman-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-saag-makhani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-chicken-korma', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-orange-beet-salad-with-crispy-chickpeas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-broccolini-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanakopita-stuffed-peppers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-hot-dogs-with-corn-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/texan-style-barbecue-beef-brisket-and-succotash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-sage-and-blackberry-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ribeye-with-burnt-spring-onion-aioli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/venison-burgers-with-mustard-fries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-tomato-zaatar-houmous-and-harissa-tarts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-lamb-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-nicoise-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-feta-greek-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/blt-panzanella-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tortang-talong', 'https://www.sainsburysmagazine.co.uk/recipes/mains/red-lentil-koftes-with-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-kedgeree', 'https://www.sainsburysmagazine.co.uk/recipes/mains/minted-pea-and-lemon-carbonara', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-roasted-radishes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-steak-fajita-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/poutine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-glazed-tuna-with-ginger-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sea-bass-with-buttered-brown-shrimp-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/trout-with-a-gremolata-crumb', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-spring-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-traybake-lemon-butter-asparagus', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-meatball-traybake',
 'https://www.sainsburysmagazine.co.uk/recipes/mains/quick-roast-chicken-with-tarragon-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/charred-spring-onion-miso-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-carrot-and-halloumi-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-porcini-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-shoulder-of-lamb-with-chickpeas-onions-garlic-and-oregano', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-and-pistachio-pesto-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/minty-lamb-chops-with-watercress-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-beef-short-ribs-with-haricot-beans-and-spinach', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-pork-shoulder', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sour-squash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-harvest-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crab-and-spring-onion-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-mushroom-cheesecake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-greens-and-mung-beans-with-turmeric-oil', 'https://www.sainsburysmagazine.co.uk/recipes/mains/honey-miso-chicken-squash-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-shrimp-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scrambled-eggs-with-asparagus-salami-and-oregano', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-and-caper-chicken-thighs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/korean-style-beef-noodle-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herby-spring-chicken-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/savoy-cabbage-with-celeriac-puree-and-brown-butter-hazelnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-spiced-meatloaf-roast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-lentil-and-quinoa-pilaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-cottage-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mac-and-3-cheese-stuffed-squash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crab-stuffed-plaice-with-cucumber-cream-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-meatball-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pork-bao-buns-and-homemade-pickles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-butternut-with-chickpeas-and-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-beef-and-noodle-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-trout-with-parsnip-rosti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jackfruit-faitas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/saffron-salmon-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-balti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-balmoral-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-sweet-potato-tomato-and-coconut-curry',
 'https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-creamy-black-bean-jackets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-beetroot-dhal-with-homemade-coriander-chutney', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-vegan-mushroom-and-leek-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chard-and-chestnut-gnocchi-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-sweet-potato-and-bean-chilli-from-knorr-stock-pots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-katsu-curry-from-the-vegetarian-butcher', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-pasta-primavera-from-knorr-stock-pots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-potato-and-pea-curry-from-knorr-stock-pots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-cajun-spiced-poke-bowl-with-winter-coleslaw-from-the-vegetarian-butcher', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-ultimate-vegan-tacos-from-the-vegetarian-butcher', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-5-spice-tofu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-caraway-beef-and-beetroot-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pavo-relleno-mexican-stuffed-roast-turkey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pomegranate-and-clementine-slow-roast-lamb-leg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-seven-spice-pork-belly-with-cranberry-and-orange-ketchup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-orange-glazed-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-parsnip-and-apple-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-roast-pork-loin-with-cranberry-glazed-pigs-in-blankets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/easy-carve-roast-turkey-with-chestnut-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-stuffed-spuds', 'https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-turkey-on-the-barbecue', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-wensleydale-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sugar-plum-ham', 'https://www.sainsburysmagazine.co.uk/recipes/mains/firecracker-chicken-with-cavolo-nero-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chestnut-pesto-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-cauliflower-pasta-with-creamy-tahini-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-chicken-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-roots-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-mushroom-lentil-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-beetroot-and-squash-wellington', 'https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roast-celeriac-with-sage-and-walnut-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-tray-roast-with-balsamic-glaze-from-taste-the-difference', 'https://www.sainsburysmagazine.co.uk/recipes/mains/keralan-cauliflower-and-lentil-curry', 
'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-pea-and-watercress-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-barley-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-fennel-and-olive-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-veggie-lasagne-rolls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-crumbed-cod', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-courgette-pesto-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/duck-marsala-and-chestnut-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-soy-and-sesame-tofu-with-broccoli-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pumpkin-and-sage-souffle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-cashews-and-lentils-with-date-chutney', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-katsu-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leek-cacio-e-pepe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-apple-and-stilton-strudel', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-toad-in-the-hole-with-sherry-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-root-and-barley-broth', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tarka-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-aubergine-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-duck-legs-with-blackberries-and-celeriac-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-madras', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-veggie-curry-with-radish-raita', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-turnip-and-feta-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/one-pot-pasta-with-bacon-and-peas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/squash-and-chickpea-one-pot-with-herby-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/biang-biang-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-stuffed-herb-and-garlic-veggiea', 'https://www.sainsburysmagazine.co.uk/recipes/mains/eggs-florentine-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-gnocchi-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-chops-with-smashed-green-bean-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-harissa-seafood-stew-from-belazu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gingered-beef-ribeye-with-noodle-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-garlic-roast-chicken-with-crispy-ham-and-veggies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-plum-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spatchcock-bbq-chicken-with-burnt-onion-aioli-and-caramelised-lemons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-cherry-tomato-and-olive-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-anchovy-butter-and-mustard-roast-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken-pilaf-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-tofu-and-cashew-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-salad-with-pork-scratching-croutons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potatoes-with-chorizo-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-finger-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veg-patch-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-scallop-and-pea-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-mussel-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-paneer-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gravlax-baked-salmon-with-sweet-dill-mustard-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pot-roast-chicken-with-cider-and-tarragon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brunch-burger-with-bacon-ketchup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-broad-bean-and-seaweed-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tear-and-share-spiced-veg-roll', 'https://www.sainsburysmagazine.co.uk/recipes/mains/minty-pea-and-bacon-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/carrot-dogs-with-dill-pickles-and-remoulade', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-and-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-griddled-vegetables-grain-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-salmon-fishcakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-feta-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-lamb-kebabs-with-summer-pea-salad',
 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-stew-cottage-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-ham-tart',
 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-feta-tart-with-poppy-seed-pastry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-veggie-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-mole-burritos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/potato-and-ricotta-gnocchi-with-purple-sprouting-broccoli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-orzotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-kofta-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-yassa-with-hasselback-sweet-potato', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-salmon-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butter-bean-pepper-and-chorizo-saute-with-seared-cod-and-asparagus', 
'https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-pea-pesto-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-okra-soup-and-semo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fatty-meatloaf-with-bacon-cheese-and-tomato-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/charred-chorizo-peach-pitta-panzanella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/groundnut-beef-stew-with-ground-rice-balls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-lamb-and-yogurt-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-pork-and-fennel-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-with-jewelled-rice-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-miso-steak-with-sesame-sweet-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-with-lentil-tabbouleh', 'https://www.sainsburysmagazine.co.uk/recipes/mains/white-wine-coq-au-vin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pork-meatloaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potatoes-with-almond-tahini', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-crumbed-cod-with-minty-crushed-potatoes-and-peas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-pork-rice-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-egg-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-with-a-spring-broth-of-asparagus-jersey-royals-and-mussels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/stuffed-roast-pork-fillet-with-cannellini-bean-and-tomato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-skewers-with-satay-sauce-and-smashed-cucumber', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pork-skewers-with-cucumber', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-lamb-burgers-with-pickled-vegetables', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-chamoy-chicken-with-grains', 'https://www.sainsburysmagazine.co.uk/recipes/mains/swedish-sandwich-cake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-salmon-with-warm-saag-aloo-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-casserole-with-walnut-gremolata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-prune-and-sherry-casserole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herb-crusted-salmon-with-garlic-fries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zesty-salmon-parcels-with-dill-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jalapeno-crusted-salmon-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-spaghetti-with-squash-salami-and-pine-nuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-style-chicken-with-braised-spring-veg-and-horseradish-cream', 'https://www.sainsburysmagazine.co.uk/recipes/mains/provencal-pine-nut-crusted-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/springtime-frittata-with-roasted-radishes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/maple-mustard-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/stuffed-picnic-loaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-vegetable-pot-pies-with-oxo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-caper-and-olive-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/masala-mackerel-with-coconut-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sour-pork-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-lamb-with-a-watercress-and-caper-crust', 'https://www.sainsburysmagazine.co.uk/recipes/mains/curried-halloumi-fritters-raita-and-fried-egg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-lamb-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-beef-burrito-rolls-with-broccoli-stalk-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/colmans-mint-glazed-roast-leg-of-lamb', 'https://www.sainsburysmagazine.co.uk/recipes/mains/colmans-herbed-mustard-crust-roast-beef', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-veggie-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/meatball-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-beef-ragout', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leek-ham-and-apple-barley-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seared-steak-taco-board', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sage-onion-toad-in-the-hole-with-cider-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-steak-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-chicken-and-mushroom-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/batch-cook-honey-balsamic-beef-brisket', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jackfruit-and-mushroom-stew-with-herb-dumplings', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roasted-lamb-with-spring-onion-mash-and-pistou', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-wellington-with-porcini-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-citrus-roast-duck-with-crispy-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/masala-cauliflower-and-aduki-bean-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-spiced-veggie-sausage-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-baked-beans', 'https://www.sainsburysmagazine.co.uk/recipes/mains/filo-fish-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-and-haggis-escalopes-with-whisky-and-chicken-butter-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roots-barley-and-lamb-cawl-with-cheddar-puffs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-mackerel-and-leek-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-island-lamb', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moussaka-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-e-fagioli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/horseradish-coated-cod-fillets-with-beetroot-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-masala-bean-pot-pies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-cobbler-with-cheese-scones', 'https://www.sainsburysmagazine.co.uk/recipes/mains/venison-steak-diane-with-truffle-fries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-roast-pork-and-winter-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-wine-pot-roast-duck', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-red-wine-stew-with-porcini-and-chestnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/triple-cheese-stuffed-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-sage-and-pine-nut-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-rib-of-beef-with-chimichurri', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-sloe-gin-and-plum-ham', 'https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-spaghetti-bolognese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/port-clementine-and-5-spice-ham', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-maple-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-four-cheese-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vineyard-chicken-with-crispy-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-pork-steaks', 'https://www.sainsburysmagazine.co.uk/recipes/mains/porchetta-style-turkey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/5-a-day-turmeric-traybake-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/curried-lamb-shanks-with-mauritian-mango-kutcha', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jewelled-stuffed-butternut-squash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-tacos-with-feta-and-watermelon-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pepper-and-chorizo-pie-with-a-cornbread-crust', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-pork-and-crispy-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/wine-braised-sausages-with-olive-oil-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brown-butter-mushroom-and-sage-pappardelle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheeseboard-nachos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-soup-toastie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-sausage-gnocchi-hotpot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-mozzarella-turkey-patties-and-griddled-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/plum-and-stilton-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/provencale-daube-style-pot-roast-beef', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-spanish-style-shepherds-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-and-egg-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-pork-with-rosemary-potatoes-and-sprouting-broccoli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beef-fillet-with-celeriac-puree', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-cabbage-and-leek-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca-baked-cod', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fajita-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/green-curry-roast-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-eggs-with-garlic-toasts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salt-and-pepper-salmon-with-spring-onion-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-tapenade-tart-with-broccoli-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/curried-fish-pie-with-spiced-potato-topping', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-sage-and-ricotta-open-crust-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-topped-chicken-with-broccoli-smash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-lamb-patties-and-sweet-potato-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-hake-with-almond-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-chicory-apple-and-walnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baja-fish-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-caramel-pork-noodle-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-aubergine-pesto-feta-gyros', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-gammon-and-pineapple-with-colcannon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-nectarine-and-prosciutto-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-saffron-and-paprika-spiced-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-prawn-and-chorizo-paella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-with-mussels-bay-and-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-with-leeks-lime-and-curry-butter', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sardines-with-cucumber-salsa-verde-salad-and-garlic-mayonnaise', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-rags-with-speedy-sausage-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-cauliflower-steaks-with-pineapple-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-chickpea-and-carrot-veggie-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-inspired-sausage-plait', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coriander-feta-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-watercress-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-with-zesty-peas-and-mint', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-prosciutto-cannelloni', 'https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-sweet-potato-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-chicken-with-green-grains', 'https://www.sainsburysmagazine.co.uk/recipes/mains/woolton-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-and-rainbow-veg-grain-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-lemon-chicken-casserole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/campers-caponata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-salad-with-halloumi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/store-cupboard-jambalaya', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-mac-and-cheese-with-caramelised-tomatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-sausages-with-beer-braised-onions', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-risotto-with-basil-oil', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-red-curry-noodle-soup-with-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/charred-spring-onion-and-ricotta-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/open-lasagne-with-roasted-squash-crispy-prosciutto-and-amaretti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-romesco-sauce-and-butterbean-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-split-pea-dhal-with-roasted-sweet-potato', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-leg-of-lamb-with-garlic-and-rosemary', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-caponata-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-and-lime-fish-and-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-minestrone', 'https://www.sainsburysmagazine.co.uk/recipes/mains/minty-salsa-verde-lamb-with-lemony-potatoes-and-leeks', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-chicken-with-roast-veg-and-olives', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-chicken-fennel-and-garlic-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-sardine-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-bourguignon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fattoush-salad-with-herbed-labneh-balls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kale-and-ricotta-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-smoked-haddock-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-lamb-tabbouleh-and-beetroot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-turkey-larb', 'https://www.sainsburysmagazine.co.uk/recipes/mains/curried-mushroom-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-beef-stew-with-gruyere-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lobster-mac-n-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-miso-mushroom-casarecce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-aubergine-rice-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-steak-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-spring-veg-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-hake-lime-and-coconut-pilaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-tarragon-crushed-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-avocado-corn-and-feta-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-wrapped-cod-and-mediterranean-vegetables', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-tomato-chicken-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hake-celeriac-puree-and-balsamic-chickpeas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-with-caper-lemon-sauce-and-green-bean-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-burgers-with-tahini-coleslaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/golden-glow-chicken-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/shepherds-pie-with-root-veg-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-chicken-ramen', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-italian-savoury-bread-pudding', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-stuffed-pork-with-roasted-roots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-winter-veg-traybake-with-speedy-rhubarb-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-tomato-fusilli-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scouse', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-soy-beef-and-green-bean-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/green-goddess-glow-bowl-with-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-veggie-rice-with-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-turkey-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cider-lamb-chops-with-celeriac-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pork-on-a-bed-of-cider-soaked-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-chinese-beef-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/haggis-neeps-and-tatties-tart-with-whisky-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/restorative-chicken-and-leek-noodle-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tofu-katsu-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-peanut-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-lasagne-with-butternut-squash-and-hazelnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crayfish-and-champagne-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/curried-smoked-haddock-chowder', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-blue-cheese-ciabatta-with-beetroot-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-stuffing-roulades-with-sprout-and-chestnut-potato-cakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/citrus-salmon-with-green-quinoa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-madras-with-mango-and-cucumber-raita', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-with-stir-fried-sprouts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-lemon-butter-roasted-turkey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cashew-and-pistachio-stuffing-nut-roast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/porcini-bacon-and-sage-roast-rib-of-beef-with-port-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-beef-fillet-with-truffle-mayo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-chicken-and-leek-sourdough-tartiflette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/witches-brew-smoky-beef-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pork-with-balsamic-mushrooms-crispy-potatoes-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-chicken-and-courgettes-with-pistachio-yogurt-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/luxe-smoked-salmon-gnocchi-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-florentine-mushroom-sauce-and-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/easy-cheesy-pasta-with-greens', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-loin-with-beetroot-and-lentil-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/duck-and-red-wine-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-beef-stew-and-cheese-biscuits', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kipper-and-salmon-fish-cakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-ham-and-greens-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/marmite-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ham-and-leek-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-hotdogs-with-crispy-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-jalapeno-quesadillas-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-spinach-tortelloni-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butter-bean-makhani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chinese-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tray-baked-squash-beetroot-and-giant-couscous-with-goats-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-harvest-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-leek-pie-with-crispy-potato', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fusilli-with-spinach-mushrooms-and-goats-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-taco-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-falafel-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/alpine-chicken-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beer-marmite-braised-beef-with-horseradish-dumplings', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-salmon-nicoise-with-soft-boiled-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jerk-pork-belly-with-mango-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-tofu-and-broccoli-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-chicken-and-quinoa-with-corn', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-and-chickpea-kofte', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-steaks-with-squash-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ale-stew-with-cheese-and-mustard-croutes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/truffled-mushroom-mac-n-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pea-ham-and-goats-cheese-tortilla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-veg-gnocchi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-pea-and-cauliflower-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-schnitzel-salad-with-cheats-garlic-aioli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-coconut-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-butter-bean-summer-casserole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-ratatouille-with-lentils-and-lime-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-fritters-with-pasta-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fire-grilled-steak-sandwich-with-chimichurri', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-chickpea-and-chorizo-stew-with-fried-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-burgers-with-cucumber-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-roast-mackerel-with-spiced-chickpeas-and-raita', 'https://www.sainsburysmagazine.co.uk/recipes/mains/buttery-courgette-and-rocket-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ale-pie-with-pickled-walnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-mackerel-with-orange-and-beetroot-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-with-orange-cumin-and-apricot-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/honey-and-lemon-pork-with-grains', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parmo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-aubergine-with-tabbouleh-and-walnut-turmeric-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-traybake-with-spinach-and-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-chicken-and-peach-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/maple-smoked-salmon-and-potato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cashew-carrot-pomegranate-and-halloumi-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sloppy-joe-bowls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-cheese-dogs-with-mustard-onions', 'https://www.sainsburysmagazine.co.uk/recipes/mains/satay-steak-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mustard-honey-salmon-with-veggie-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/all-day-breakfast-gnocchi-fry-up', 'https://www.sainsburysmagazine.co.uk/recipes/mains/springtime-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pea-and-ricotta-gnocchi-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-pork-with-sweet-potato-wedges-and-sweetcorn-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-cashew-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-burger-and-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/oregano-roast-beef-with-crispy-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-crusted-lamb-rack', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-pea-fishcakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/california-salmon-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tahini-and-olive-spaghetti-with-tomato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-and-goats-cheese-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cornish-pasties', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-apricot-chickpea-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cantonese-aubergine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-fajitas-with-jalapeno-salsa-and-avocado', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-bean-meatballs-in-spiced-tomato-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sticky-pork-char-siu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-pasta-with-ham-crisps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-chicken-curry-with-thai-style-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-jackets-with-beetroot-ratatouille', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gin-and-tonic-salmon-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-salmon-fishcakes-with-loaded-potato-skins', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butterflied-lamb-with-kentucky-mop-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-black-bean-and-sweet-potato-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/royal-almond-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-asparagus-tagliatelle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-paneer-skewers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-halloumi-burger-with-sweetcorn-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brinner-burrito-with-marmite-hollandaise', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chickpea-and-sweet-potato-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-mustard-pork-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-baked-fish-with-orzo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/individual-minted-lamb-wellingtons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/maple-soy-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pomegranate-beef-with-spiced-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-chicken-and-grains', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-roast-lamb-with-leeks-and-caerphilly', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-leg-of-lamb-with-fennel', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-roast-lamb-recipe-gizzi-erskine-reicpe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-spiced-whole-celeriac-with-raita', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-tortellini-minestrone', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-goats-cheese-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-fajita-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-asparagus-crumble', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweetcorn-courgette-and-feta-oven-baked-fritters', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sizzling-garlic-prawn-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-cauliflower-and-quinoa-goodness-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-salmon-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-jalapeno-buddha-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/choucroute-garnie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-dill-cullen-skink', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-meatballs-with-courgetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-aubergine-squash-feta-grain-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-chicken-wraps-with-quick-pickled-cucumber', 'https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/friday-night-fish-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-baked-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-lamb-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-omelette-with-tomato-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-kiev-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-roast-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-stir-fry-with-chilli-and-basil-2', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pineapple-pork-mince-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-chow-mein', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quinoa-grape-and-feta-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-sage-n-squash-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/soy-butter-steak-with-bearnaise-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sardine-croquettes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-beans-on-sourdough', 'https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-pork-with-water-chestnuts-and-greens', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-stilton-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sea-bass-with-parsley-and-anchovy-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-beef-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-satay-with-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-patties-with-honey-roast-butternut-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-cornbread-cobbler', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bolognese-pancakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pappardelle-with-rosemary-and-sausage-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-creamy-leeks-and-butter-beans', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-leek-carbonara', 'https://www.sainsburysmagazine.co.uk/recipes/mains/best-ever-lamb-casserole', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-orange-and-almond-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fondue-mac-and-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-and-halloumi-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/red-pepper-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-vegetable-and-pancetta-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sausage-and-parsnip-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ham-sweetcorn-and-cheese-bread-pudding', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rich-lamb-and-pork-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/meatball-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-garlic-thyme-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-sausage-gnocchi-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-lamb-and-sweet-potato-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-chicken-pie-with-chorizo-and-olives', 'https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-black-bean-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-apple-winter-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-fishcakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-pie-with-cheesy-rosti-topping', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-turkey-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fridge-forage-tex-mex-rice-with-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-mango-chicken-with-fragrant-pilau', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-lentil-and-halloumi-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-with-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-korma', 'https://www.sainsburysmagazine.co.uk/recipes/mains/super-green-halloumi-omelette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/walnut-and-red-pepper-stuffed-cabbage-leaves', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-fennel-and-orange-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-rogan-josh', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-shepherds-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-brisket-with-mexican-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pineapple-pulled-pork', 'https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rolled-turkey-breast-with-rosemary-and-cranberry-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-baked-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beetroot-and-apple-stuffing-loaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cauliflower-tart-with-thyme-and-pistachios', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herbed-mackerel-with-beetroot-puree-and-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/duck-stir-fry-with-plum-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-squash-with-pumpkin-seed-and-cranberry-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-turkey-with-turkey-butter-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brussels-sprouts-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-crispy-duck-with-cranberry-glaze', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-gravlax-pasta-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-cauliflower-sage-and-walnut-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/irish-lamb-ragout', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-chook-with-pea-and-parsley-dumplings', 'https://www.sainsburysmagazine.co.uk/recipes/mains/simple-sausage-and-rosemary-cassoulet', 'https://www.sainsburysmagazine.co.uk/recipes/mains/philadelphia-style-spiced-steak-naanwiches', 'https://www.sainsburysmagazine.co.uk/recipes/mains/stilton-mac-and-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/somerset-pork-with-apples-and-cider', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-sweet-potato-rice-bowls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-cod-with-sweet-and-smoky-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-tomato-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-pie-potato-skins', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pearl-barley-risotto-with-blue-cheese-and-squash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-beef-brisket-with-cranberries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-citrus-and-spice-turkey', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-shawarma', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-and-onion-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/full-english-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coq-au-vin-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-spinach-dhansak', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-boursin-chicken-with-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-beef-brisket', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-stuffed-potato-skins', 'https://www.sainsburysmagazine.co.uk/recipes/mains/malaysian-sweet-potato-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-beet-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hake-with-samphire-and-malai-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sardine-and-tomato-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-beer-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-melt-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chickpea-and-spinach-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-paprika-goulash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-and-broccoli-pizzette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-duck-breast-with-couscous-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-broad-beans-mint-and-ricotta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-sweet-potato-lentils-and-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sri-lankan-chicken-and-chickpea-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-cider-and-parsnip-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-shells-with-sardines-fennel-and-saffron', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-masala-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-stroganoff', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-and-ham-hash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-meatball-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-honey-baked-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-soup-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/minced-beef-tatties', 'https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-sweet-potato-and-lime-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-walnut-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-beef-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-mustardy-lentils-beetroot-and-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-shepherds-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/wild-mushroom-speltotto-with-pine-nuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/healthier-bangers-and-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-quesadillas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-lemon-and-honey-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-sausage-and-crispy-potato-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/indian-spiced-lamb-chops-with-mango-and-cucumber-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-fish-and-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-couscous-with-roast-squash-and-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/battered-haddock-and-mushy-pea-fishcakes-with-homemade-tartare-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-madras-sandwich', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-herb-and-golden-ale-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ham-egg-and-chip-tortilla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-tagliatelle-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-salmon-and-summer-veg-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-courgette-and-basil-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-edamame-and-spring-onion-omelette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-stuffed-aubergine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hariyali-sea-bream', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-mozzarella-melts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-mezze-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fennel-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/store-cupboard-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-indian-lamb-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-pea-and-mushroom-tikka-masala', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-feta-and-pesto-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-chicken-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-kale-and-butter-bean-one-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-greens-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-mac-and-greens', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gammon-with-mustard-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-lentil-nut-roast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/green-veg-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-style-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-spiced-cauliflower', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-tomato-tagliatelle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coffee-and-rum-marinated-steaks', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-bianca-with-broccoli-and-garlic', 'https://www.sainsburysmagazine.co.uk/recipes/mains/margherita-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-baked-eggs-with-peperonata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-roasted-spinach-polpette-and-tomatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-salmon-with-watercress-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-cajun-grilled-corn-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/super-moist-roast-chicken-with-lemon-and-thyme', 'https://www.sainsburysmagazine.co.uk/recipes/mains/watercress-pesto-and-lemon-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/easy-chicken-shawarma', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aloo-gosht', 'https://www.sainsburysmagazine.co.uk/recipes/mains/traybaked-teriyaki-mackerel-with-greens', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parma-ham-wellingtons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-crayfish-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fish-parcels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/oven-roasted-chicken-and-avocado-with-asparagus', 'https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-pasta-with-lemon-avocado-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/stacked-sweet-potato-burgers-with-avocado-lime-cream', 'https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-stuffed-baked-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-carbonara', 'https://www.sainsburysmagazine.co.uk/recipes/mains/blt-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mango-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-prawns-with-cinnamon-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/antipasti-tartlets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-steak-and-roasted-new-potato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-steak-with-tomato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-smoked-salmon-and-hollandaise-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-onion-goats-cheese-and-rocket-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-finger-sarnies-with-fennel-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/radish-spring-onion-and-watercress-salad-with-baked-trout', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-roast-chicken-with-root-vegetable-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/giant-sharing-bacon-jam-rosti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-filo-parcel', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tenderstem-and-goats-cheese-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/persian-herb-rice-with-broad-beans', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chorizo-and-mixed-grain-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-kedgeree', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fish-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smashed-salmon-hash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-peanut-pork-with-tenderstem', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chickpea-cauliflower-and-squash-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/satay-beef-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-and-aubergine-with-minted-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-spanish-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle-meatball-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-pastilla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-pea-and-watercress-tagliatelle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-chicken-with-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/riojan-pork-chorizo-and-potato-pot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-badger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-pork-with-sweet-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-diane', 'https://www.sainsburysmagazine.co.uk/recipes/mains/artichoke-spaghetti-with-romanesco', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dill-mustard-trout-with-smashed-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-stuffed-with-lemony-goats-cheese-and-tarragon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-with-cauliflower-couscous-and-chermoula-tomatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-smoked-salmon-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-steak-with-cheesy-polenta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-goats-cheese-and-prosciutto-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-madeira-and-chestnut-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/squash-stilton-and-kale-pies-with-walnut-pastry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sicilian-roast-chicken-with-fennel-and-blood-oranges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-meatballs-with-red-pepper-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-roast-chicken-with-tomato-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-lamb-with-quick-pickled-cucumber', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kale-carbonara', 'https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-and-spinach-tortelloni-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/whisky-venison-steaks-with-neeps-and-tatties', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mango-chipotle-rice-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/campfire-roast-fish-with-lemon-and-fennel', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-red-pork', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-style-crispy-duck-salad-with-blood-oranges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-prawn-noodle-jar', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leaner-chicken-cacciatore', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pernod-baked-mackerel-with-fennel-and-watercress', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-chicory-and-blue-cheese-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/venison-and-red-wine-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-salmon-with-sesame-noodles-and-cucumber-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/venison-sausage-and-squash-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-gorgonzola-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-ham-and-pea-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-chicken-nachos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-cheese-and-pepper-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sizzling-squash-with-herby-cucumber-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/noodles-with-chicken-and-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-giant-cous-cous-broth', 'https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-squash-and-walnut-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-and-black-rice-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-sumac-and-pomegranate-soup-with-roasted-peanuts-coriander-and-lime', 'https://www.sainsburysmagazine.co.uk/recipes/mains/golden-coconut-and-cauliflower-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cumin-carrot-and-lamb-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-potato-and-ham-hash-with-sage-fried-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-turkey-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-beef-wellington', 'https://www.sainsburysmagazine.co.uk/recipes/mains/red-leicester-pickled-onion-and-watercress-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-sage-and-chestnut-wellington', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-tofu-with-miso-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-smoked-salmon-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-gewurztraminer-and-grapes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-prawn-and-coconut-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/umami-chicken-ramen', 'https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-cod-and-brussels-sprout-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-orange-chicken-with-broccoli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-gnocchi-and-turkey-hash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-corn-and-bean-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-lamb-pie-with-almonds-and-apricots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-balsamic-onions-and-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-dhal-with-roasted-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tartiflette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-mac-n-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/colombian-style-chicken-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-style-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potatoes-stuffed-with-chickpeas-and-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/shepherds-sag-aloo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-biryani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/braised-fennel-with-figs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asian-chicken-and-ginger-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moussaka', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-style-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gammon-and-egg-with-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-lamb-skewers-with-herby-bulgur', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-schnitzel-with-rose-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-yogurt-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/massaman-curry-buddha-bowls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/macaroni-cheese-with-crispy-bacon-and-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chick-n-sours-korean-chicken-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-pesto-tortelloni-broth', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-spiced-potato-and-zhoug-pitta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/squid-and-mackerel-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-aubergine-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pizzadilla', 'https://www.sainsburysmagazine.co.uk/recipes/mains/feta-aubergine-pomegranate-and-harissa-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/watermelon-jelly-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheeseburger-nacho-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle-giant-fishcake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zaatar-potato-salad-with-olives-and-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-spelt-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-spinach-egg-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-ginger-chicken-pho', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-sushi-salad-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-korma', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-and-saffron-chicken-pitta-with-blackened-chilli-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-salmon-with-blueberry-lime-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-tagliatelle-with-roasted-red-pepper-cream-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/golden-vegetable-dhal-with-soft-boiled-eggs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rice-spinach-and-eggs-with-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pad-kra-pao', 'https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-ricotta-lemon-and-herbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salt-and-pepper-rib-of-beef-with-bloody-mary-sauce-and-roast-potato-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/shoyu-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-tuna-burgers-with-chilli-mango-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/charred-broccoli-borlotti-bean-and-hazelnut-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-cod-with-sesame-cucumbers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-chicken-udon-noodles-2', 'https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-pine-nut-pesto-penne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-tea-brined-chicken-with-warm-rice-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-gnocchi-with-savoury-granola', 'https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roasted-cauliflower', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-goats-cheese-risotto-mint-gremolata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-chickpea-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-chicken-and-shiitake-broth', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-pineapple-and-herb-roasted-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/venison-steaks-with-miso-parsnip-fries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-coconut-rice-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-marmalade-chicken-drumsticks-with-blue-cheese-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-manchego-potato-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/caribbean-lamb-hotpot-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chorizo-pie-saffron-pastry-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chermoula-cod-bulgar-salad-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-wonton-soup-one-pound-meals-miguel-barclay', 'https://www.sainsburysmagazine.co.uk/recipes/mains/garam-masala-chicken-spicy-potatoes-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-roast-chicken-with-cornbread-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tenderstem-and-ham-quiches', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-pork-patties-with-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-adobo-with-roast-squash-rice-and-smoky-tomato-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbacoa-beef-brisket-tacos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-goats-cheese-tart-recipe', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-butternut-with-honey-and-walnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-lamb-pilaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/yellow-duck-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/manchego-fritters-with-seville-orange-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/farc-on', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kedgeree-with-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tom-daley-s-harissa-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-shallot-and-mushroom-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/houmous-pepper-goats-cheese-and-sweet-chilli-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-and-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/higgledy-piggledy-greek-beef-pasta-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-tacos-with-smoky-coleslaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-schnitzel-with-lemon-and-caper-butter', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-cashew-stir-fried-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-sea-bass-with-ginger-and-spring-onion', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-bites-with-sweet-chilli-dip', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-mac-n-cheese-cake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-gumbo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-salmon-and-kale-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quick-smoked-haddock-and-cauliflower-chowder', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-black-bean-and-squash-burrito-bowl', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-salad-of-pan-fried-duck-with-clementines', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-beans-on-toast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/porky-pancetta-ragu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beef-sirloin-with-juniper-and-black-pepper-2', 'https://www.sainsburysmagazine.co.uk/recipes/mains/savoury-parsnip-and-pecan-slice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-pearl-barley-risotto-with-feta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/honey-roast-salmon-and-broccoli-crustless-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/korean-barbecue-beef-stir-fry-with-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-style-chicken-and-fennel-braise', 'https://www.sainsburysmagazine.co.uk/recipes/mains/charred-cauliflower-and-coconut-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-crown-with-herb-butter', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cranberry-and-orange-glazed-ham', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-coulibiac', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-roasted-cod-with-chorizo-and-persimon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jose-hot-dogs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-streetcorn-hot-dogs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-chop-root-veg-and-apple-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bengali-coconut-fish-pie-with-mustard-seed-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/satay-chicken-with-soba-and-cucumber-noodle-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/singapore-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-beef-pho', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beer-battered-tofu-tacos-with-a-chipotle-cashew-soured-cream', 'https://www.sainsburysmagazine.co.uk/recipes/mains/korean-style-stew-with-marinated-tofu-pieces', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-n-sour-bahn-mi-baguettes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quick-pickled-rice-noodle-salad-with-soy-glazed-griddled-tofu', 'https://www.sainsburysmagazine.co.uk/recipes/mains/naan-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-watercress-chilli-pesto-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/firecracker-prawn-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-sausage-and-squash-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-wrapped-chicken-with-creamy-lentils', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-chicken-and-mushroom-hot-pot-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-ham', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-liver-pancetta-tagliatelle-with-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-cheese-toastie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-easy-black-bean-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-chorizo-and-melting-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-houmous-and-harissa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/white-bean-soup-with-candied-vanilla-bacon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pappardelle-with-prawns-and-vanilla-beurre-blanc', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-spiced-chicken-legs-roasted-with-lemons-tomatoes-and-celery', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cauliflower-pea-and-cashew-pilaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-tagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tom-yum-prawn-noodle-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-fish-stew-with-cheese-toasts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-noodles-with-spinach-and-ricotta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hot-dogs-with-new-york-streetcart-onions', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sesame-pork-noodle-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ghastly-ghoul-ash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-pickled-veg-with-seared-tuna-and-herbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-duck-with-soft-herb-and-grapefruit-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herb-omelette-with-feta-and-broad-beans', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fresh-tomato-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/trout-with-broad-beans-and-courgettes-in-a-wine-and-cream-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-mustard-and-goats-cheese-galette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baharat-spiced-baked-aubergines-with-feta-fennel-and-spelt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-beef-nectarine-and-rocket-salad-with-thai-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-and-mozzarella-pockets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-spicy-charred-corn-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-glazed-sausages-and-onion-rings-2', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hot-dogs-with-celeriac-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tapanade-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-onion-topping', 'https://www.sainsburysmagazine.co.uk/recipes/mains/satay-noodle-lettuce-wraps-with-prawns-and-mango', 'https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-chicken-fatteh', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-smoked-mackerel-with-preserved-lemon-quinoa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hot-smoked-trout-and-beans-with-a-creamy-dill-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-garden-salad-with-peaches-and-mozzarella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-burritos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cobb-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-soup-bowl-with-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-meze-pitta-pockets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/loaded-pulled-pork-subs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salad-of-roasted-beetroot-goat-s-cheese-and-hazelnuts-on-wild-rocket', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mac-n-cheese-pancetta-cups', 'https://www.sainsburysmagazine.co.uk/recipes/mains/zaatar-lamb-chops-with-salted-yogurt-sweet-herbs-and-pomegranate-molasses', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-rogan-josh-with-mint-and-coriander-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quick-cook-ancho-con-carne-with-cucumber-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-new-potatoes-cherry-tomatoes-and-basil', 'https://www.sainsburysmagazine.co.uk/recipes/mains/https-www-youtube-com-watch-v-gjbibr6dv1s', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-loin-steaks-in-cider-and-tarragon-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-salmon-and-puy-lentil-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-chicken-with-stir-fried-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-cheddar-and-mushroom-pasta-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/masala-meatball-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-mackerel-with-toasted-fennel-and-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/miso-marinated-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/burger-with-smoky-stilton-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-roast-chicken-with-watercress-and-spiced-peaches', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-pistachio-koftas-with-fennel-radish-and-apricot-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-tuna-nicoise-ratatouille-stack', 'https://www.sainsburysmagazine.co.uk/recipes/mains/porchetta-style-roast-pork-with-a-courgette-chickpea-and-spinach-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-crusted-fish-with-ginger-pak-choi', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-squash-and-goats-cheese-salad-with-griddled-spring-onions-and-pumpkin-seed-crunch', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-pesto-and-pepper-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/charred-aubergine-and-tomato-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-chicken-and-olive-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/camembert-baked-with-maple-syrup-pecans-and-blueberries', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brie-and-beef-baguette-with-beetroot-jam', 'https://www.sainsburysmagazine.co.uk/recipes/mains/twice-cooked-goats-cheese-souffle-s-with-apple-and-walnut-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-cutlets-with-harissa-yogurt-and-midget-trees', 'https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-creamy-cavolo-nero-and-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bashed-chicken-with-walnut-and-watercress-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/summer-gnocchi-with-avocado-pesto-and-tomatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-fillet-with-asian-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-stuffed-picnic-rolls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sri-lankan-crab-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-maple-glazed-pork-chops', 'https://www.sainsburysmagazine.co.uk/recipes/mains/somerset-mussels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-poussins-with-roasties', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cote-de-boeuf-with-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-lemon-chicken-noodle-broth', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mattar-paneer-a-medium-curry-of-peas-tomatoes-and-indian-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chickpea-burgers-in-roasted-portobello-buns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cod-with-roasted-cauliflower-and-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-lettuce-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-lemon-garlic-and-thyme-chicken-kebabs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chermoula-lamb-chops-with-herb-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chicken-drumsticks-with-ginger-barbecue-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tarte-flambee-with-comte-and-watercress-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/anchovy-and-chilli-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mustardy-mushroom-stroganoff', 'https://www.sainsburysmagazine.co.uk/recipes/mains/citrus-crusted-salmon-with-buttered-leeks-and-roasted-new-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-and-leek-chicken-topped-with-green-pea-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-ricotta-meatballs-in-tomato-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gorgonzola-and-pear-salad-with-caramelised-walnuts-and-radicchio', 'https://www.sainsburysmagazine.co.uk/recipes/mains/florentine-fish-pie-with-smashed-potato-and-chives', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lightly-curried-haddock-pie-with-crunchy-mustard-seed-and-cardamom-crumbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-scallops-and-prawns-with-black-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/poussins-stuffed-with-spelt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-savoy-cabbage-with-emmental-and-breadcrumbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quinoa-salad-with-broccoli-preserved-lemon-and-avocado-oil', 'https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-leek-and-egg-salad-with-mustard-and-caper-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/persian-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mozzarella-in-carrozza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parmesan-with-rocket-and-new-potato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-mince-with-aromatic-rice-and-wilted-greens', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-mince', 'https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-kathi-rolls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-bacon-and-kale-pies-with-celeriac-and-horseradish-rosti-topping', 'https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-basil-and-feta-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cumin-chicken-with-giant-couscous-courgette-peas-and-mint', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-style-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-burgers-with-sweet-potato-wedges', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-smoked-salmon-and-cream-cheese-topping', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-avocado', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-goats-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/duck-green-beans-and-walnuts', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-bacon-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/piri-piri-rice-and-peas-with-garlic-prawns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pot-roasted-curried-chicken-with-garam-masala-mango-chutney-and-lemon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/orange-sticky-chicken-with-three-green-veg', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-garlic-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-paillards-with-fennel-honey-and-lemon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herb-crusted-sea-bass-with-toasted-coconut-and-tomato-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/jerk-fish-with-seaweed-salted-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-salmon-with-cauliflower-couscous-and-garlic-spinach', 'https://www.sainsburysmagazine.co.uk/recipes/mains/reviving-chicken-broth', 'https://www.sainsburysmagazine.co.uk/recipes/mains/deep-filled-crab-saffron-and-parmesan-tartlets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-marmalade-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/light-and-fragrant-fish-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/big-bowl-veggie-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-steak-with-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/haggis-stuffed-chicken-breasts-with-chilli-and-garlic-stir-fried-swede-and-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sichuan-roasted-red-pepper-soup-with-ve-spice-edamame-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-soup-with-horseradish-lemon-and-parsley', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kale-pea-and-spring-onion-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-orange-and-quinoa-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-prawn-rice-noodle-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/melting-brie-and-ratatouille-pasta-gratin', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-oriental-chicken-parcels', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-breast-with-apricot-gammon-and-pistachio-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-venison-haunch-with-beetroot-shallots-and-horseradish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-guinea-fowl-with-prune-and-bacon-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-with-currants-and-ginger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/oven-baked-fish-and-chips', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-spiced-cranberry-pecan-stuffing-and-maple-glaze', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sage-turkey-burgers-with-cranberry-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-pumpkin-and-chickpea-korma-with-crispy-shallots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-kale-and-goats-cheese-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-leek-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-thyme-and-garlic-roasted-chicken-with-chestnut-stuffing-filled-red-onions', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-red-pepper-sundried-tomato-and-ricotta-cannelloni', 'https://www.sainsburysmagazine.co.uk/recipes/mains/top-each-toasted-crumpet-with-2-crispy-bacon-rashers-and-a-poached-egg-serve-with-the-roasted-vine-tomatoes-on-the-side-and-season-with-freshly-ground', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-crumpet', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-garlic-raita', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-spiced-duck-legs-with-plums', 'https://www.sainsburysmagazine.co.uk/recipes/mains/caribbean-vegetable-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-chicken-with-broccoli-and-spelt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-hoisin-turkey-meatball-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-rostis-with-fried-eggs-and-avocado-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-vegetable-minestrone', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-and-lentil-pies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/duck-breasts-with-spiced-blackberry-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-roast-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-mozzarella-and-chive-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/best-ever-chicken-caesar-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-bacon-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-spiced-salmon-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hot-smoked-trout-new-potato-and-pea-shoot-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/30-minute-seafood-paella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-aubergine-freekeh', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-pak-choi-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-parma-ham-agrodolce-peppers-and-pangrattato', 'https://www.sainsburysmagazine.co.uk/recipes/mains/singapore-style-prawn-laksa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fast-roast-beef-with-thyme-beets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/the-ivy-shepherd-s-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-biryani', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-fennel-and-pepper-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/the-ultimate-veggie-burger-with-lemon-chipotle-mayo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ham-hock-broad-bean-and-new-potato-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-mackerel-fennel-and-orange-quinoa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-onion-and-bacon-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/kale-mushroom-and-gorgonzola-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-stew-with-garlic-sourdough', 'https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-lamb-mint-and-tomato-couscous-wraps', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-socca-with-cucumber-pepper-and-tahini-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-pepper-and-halloumi-kebabs-with-lemony-houmous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/persian-style-spatchcock-barbecued-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fig-gorgonzola-and-thyme-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-style-picnic-loaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-vegetable-pot-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/potato-mozzarella-and-spring-onion-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-chicken-drumsticks-with-salsa-and-griddled-apricots', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-lime-and-ginger-sea-bass', 'https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-with-avocado-roasted-vegetables-and-tahini-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-beef-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-halloumi-kebabs-with-freekeh-orange-and-walnut-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-with-coconut-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-quesadillas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-pepper-arrabbiata-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-nicoise-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-fish-with-herby-potatoes-and-courgettes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/paprika-chicken-with-griddled-corn-and-lemon-mayo', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-pancetta-salad-with-raspberries-and-mustard-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/muhammara-lamb-cutlets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-panzanella-salad-with-deep-fried-mozzarella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-tagliatelle-with-kale-caramelised-onions-and-goats-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-avocado-couscous-with-harissa-yogurt', 'https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-with-aubergine-tomato-feta-and-basil-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brioche-burger-buns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cod-baked-in-paper-with-basil-and-asparagus', 'https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-lemon-and-mozzarella-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-chicken-coleslaw-with-ginger-almond-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/seared-beef-salad-with-carrot-noodles-and-tahini-dressing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crab-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chickpea-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-red-sweet-potato-red-pepper-and-coconut-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-red-onion-flatbreads', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-nicoise-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-pork-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-ed-kebabs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pepper-and-onion-frittata-with-manchego', 'https://www.sainsburysmagazine.co.uk/recipes/mains/raw-thai-citrus-crunch-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/monkfish-chorizo-and-vegetable-paella-with-saffron-aioli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-chops-with-crushed-new-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spring-spanakopita', 'https://www.sainsburysmagazine.co.uk/recipes/mains/orange-roasted-chicken-spring-vegetables', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-tagine-with-preserved-lemon-and-olives', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-gnocchi-with-watercress-mint-pesto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-cod', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-duck-breast-with-plum-star-anise-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-piri-piri-sardines-with-tomato-and-onion-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/the-ultimate-crab-cakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-balls-with-courgetti-and-tomato-and-basil-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fish-and-spinach-curry-with-cauliflower-rice-and-pistachios', 'https://www.sainsburysmagazine.co.uk/recipes/mains/basil-tomato-and-aubergine-soba-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/soy-and-ginger-tuna-with-soba-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-fattoush-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pad-thai-noodles-with-mango-and-crab', 'https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-prawns-with-mango-noodles-and-herbs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chicken-stir-fry-with-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crab-tomato-and-parmesan-lattice-pies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-shiitake-mushroom-pho-with-carrot-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-fishcakes-with-tomato-chermoula-sauce-and-nutty-couscous', 'https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-coconut-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dukkah-sweet-potato-and-spinach-dhal', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crumbed-katsu-style-chicken-with-cucumber-and-peanut-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-veggie-shepherd-pies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cider-chicken-and-bacon-pie-with-crushed-new-potato-topping', 'https://www.sainsburysmagazine.co.uk/recipes/mains/saag-paneer-with-cauliflower-rice', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-tomato-prawns-with-courgette-spaghetti', 'https://www.sainsburysmagazine.co.uk/recipes/mains/shredded-duck-and-watercress-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-spanish-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sumac-chicken-with-watercress-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/purple-sprouting-broccoli-blue-cheese-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-broccoli-and-anchovy-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-cheese-pancakes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-duck-stir-fry-with-lime', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-macaroni-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-mushroom-pot-pies', 'https://www.sainsburysmagazine.co.uk/recipes/mains/persian-chicken-with-prunes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-and-creme-fraiche-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-spinach-quinoa-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-moroccan-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-meatballs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/winter-chicken-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/gorgonzola-and-broccoli-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-salmon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-aubergine-moussaka', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fast-chicken-butter-bean-and-tomato-stew', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-romesco-goats-cheese-pasta', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-cranberry-stuffing-and-pigs-in-blankets', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-sage-chestnut-and-apple-stuffing-balls-honey-mustard-chipolata-rolls-and-marsala-gravy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beef-en-croute', 'https://www.sainsburysmagazine.co.uk/recipes/mains/porcini-risotto-with-roast-garlic', 'https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-ribs-of-beef-with-roasted-red-onions-and-watercress-and-stilton-relish', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-breasts-with-date-and-pine-nut-quinoa-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/parmesan-crumbed-pork-with-roasted-tomatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-gnocchi-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pepper-feta-and-smoked-paprika-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mushroom-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/brazil-nut-and-chestnut-roast', 'https://www.sainsburysmagazine.co.uk/recipes/mains/three-cheese-roulade', 'https://www.sainsburysmagazine.co.uk/recipes/mains/three-pepper-steaks', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-cacciatore', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-with-artichokes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crab-tortillas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-burgers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-schnitzelc', 'https://www.sainsburysmagazine.co.uk/recipes/mains/macaro-no-cheese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-onepot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-beef-massaman', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-pecorino-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/two-cheese-spinach-and-potato-bake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-aubergine-quinoa-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-chicken-with-warm-courgette-and-fennel-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fresh-tomato-chickpea-and-spinach-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/the-ritz-game-pie', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-con-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-vegetable-tortilla-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/apple-pear-and-stilton-soup-with-chorizo-and-croutons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lambtagine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/easy-roast-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/red-onion-bangers-with-kale-mash', 'https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-chorizo-frittata', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-paneer', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sunshine-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/quick-bolognese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-onepot', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-prawn-and-broad-bean-pilaf', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crumbed-kaffir-lime-chicken-with-lime-and-ginger-mayonnaise', 'https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-paella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-stuffed-aubergines', 'https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-tomato-bolognese', 'https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chicken-in-brioche-buns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bean-ratatouille', 'https://www.sainsburysmagazine.co.uk/recipes/mains/dairy-free-fish-pie-with-crispy-kale', 'https://www.sainsburysmagazine.co.uk/recipes/mains/warm-butter-bean-chicken-and-chorizo-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/herbed-chicken-drumsticks', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-jerk-chicken-skewers-with-brown-rice-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/anjum-anand-s-indian-style-chicken-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-pepper-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/oven-baked-prawn-pea-and-rocket-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-kebabs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-goats-cheese-and-hazelnut-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-and-sausage-baguette', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-quesadillas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-courgette-and-pea-quiche', 'https://www.sainsburysmagazine.co.uk/recipes/mains/fast-roast-chicken-with-asparagus', 'https://www.sainsburysmagazine.co.uk/recipes/mains/classic-cheese-and-bacon-burger', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-thai-pork-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-sausages-and-goats-cheese-hotdogs', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-watercress-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/baked-chicken-with-olives-and-jalapenos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/poached-trout-with-watercress-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/duck-breasts-with-a-piquant-lime-and-ginger-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/padthai', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cannelloni-with-balsamic-glazed-tomatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/purple-sprouting-broccoli-and-parma-ham-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/croque-monsieur', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pork-fillet-with-smoked-paprika', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-chicken-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chow-mein', 'https://www.sainsburysmagazine.co.uk/recipes/mains/individual-venison-wellingtons', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-soup', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pheasant-with-grapes-and-sweetheart-cabbage', 'https://www.sainsburysmagazine.co.uk/recipes/mains/classic-roast-turkey-with-chestnut-stuffing-balls', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-wensleydale-and-toasted-hazelnut-wellington', 'https://www.sainsburysmagazine.co.uk/recipes/mains/onion-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/french-goat-s-cheese-ham-pepper-and-red-onion-pizza', 'https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-lamb-with-feta-peppers-and-dill', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-and-garlic-tuna-stir-fry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tagliatelle-with-prawns-parma-ham-and-parsley', 'https://www.sainsburysmagazine.co.uk/recipes/mains/john-west-tuna-greek-style-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-rosemary-pearl-barley-risotto', 'https://www.sainsburysmagazine.co.uk/recipes/mains/john-west-salmon-and-cheddar-grills-2', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-goats-cheese-tart', 'https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-turkey-meatballs-and-roasted-peppers', 'https://www.sainsburysmagazine.co.uk/recipes/mains/langoustine-and-chicken-paella', 'https://www.sainsburysmagazine.co.uk/recipes/mains/italian-stuffed-roast-pork', 'https://www.sainsburysmagazine.co.uk/recipes/mains/green-chicken-curry-with-noodles', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tom-kerridge-slow-roast-lamb-with-boulangere-potatoes', 'https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-cod-burritos', 'https://www.sainsburysmagazine.co.uk/recipes/mains/hairy-bikers-massaman-duck-curry', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-tomato-and-chilli-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-prawn-pockets-with-tomato-salsa', 'https://www.sainsburysmagazine.co.uk/recipes/mains/oxtail-stew-with-pumpkin-and-cinnamon', 'https://www.sainsburysmagazine.co.uk/recipes/mains/cumberland-glazed-roast-lamb-shoulder', 'https://www.sainsburysmagazine.co.uk/recipes/mains/one-pot-moroccan-chicken', 'https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-herb-and-pistachio-stuffing', 'https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-style-crispy-duck-with-plum-sauce', 'https://www.sainsburysmagazine.co.uk/recipes/mains/skinny-roasted-tomato-aubergine-and-chickpea-lasagne', 'https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-pork-buns', 'https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-squash-and-prawns-with-coconut-and-chilli', 'https://www.sainsburysmagazine.co.uk/recipes/mains/easy-teriyaki-chicken-traybake', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-festive-roast-pork', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-slow-cooker-pork-carnitas', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-with-bacon-and-mushrooms', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-spicy-garlic-prawn-and-rose-linguine', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chinese-style-pork-rolls-with-quick-pickled-slaw', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-orange-fennel-and-burrata-salad', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-herby-roast-chicken-with-crispy', 'https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-rose-harissa-cauli-flatbreads-with-butter-bean-humous']


# Join the items with a newline character and print
result = "\n".join(my_list)
print(result)


https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/dhal-with-crispy-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pork-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-salmon-curry
https://www.sainsburysmagazine.co.uk/recipes/

In [27]:
# Multi-line string of links
links_string = """
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/dhal-with-crispy-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pork-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-salmon-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/quick-lamb-biryani
https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-toad-in-the-hole
https://www.sainsburysmagazine.co.uk/recipes/mains/boiled-gammon-with-pease-pudding
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-frying-pan-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-green-prawn-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-meatball-moussaka
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-tagine-and-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-salty-lentil-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/seared-horseradish-steak-with-red-wine-and-rosemary-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-cheesy-puy-lentil-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-stroganoff-jacket-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/30-minute-spicy-prawn-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/lazy-caramelised-onion-sausage-hotpot
https://www.sainsburysmagazine.co.uk/recipes/mains/flatbreads-with-spiced-salmon-and-zesty-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-smoky-corn-and-peppers
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-chickpea-and-kale-frittata-with-tomato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/garlic-butter-chicken-with-cheesy-cauli-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-buttermilk-marinated-spatchcock-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/super-quick-cook-steak-skewers-with-chimichurri
https://www.sainsburysmagazine.co.uk/recipes/mains/all-american-ribs-with-sticky-bourbon-glaze
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beetroot-greens-and-brie-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-with-charred-sweetcorn-salsa-and-fresh-mint-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/yours-and-mine-seafood-boils
https://www.sainsburysmagazine.co.uk/recipes/mains/everything-seasoning-chicken-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-lemonade-chicken-with-rainbow-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/triple-citrus-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/next-level-hot-dogs
https://www.sainsburysmagazine.co.uk/recipes/mains/hot-honey-barbecue-platter
https://www.sainsburysmagazine.co.uk/recipes/mains/sicilian-style-mackerel-with-white-beans-and-caponata
https://www.sainsburysmagazine.co.uk/recipes/mains/dolcelatte-prosciutto-and-pickled-peach-pizza-with-hot-honey
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-courgette-and-butter-bean-braise
https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/broad-bean-dill-and-hazelnut-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-courgette-involtini
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-summer-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-chicken-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-tuna-kebab
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-loin-tikka-paratha-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-mango-and-cashew-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-gentlemans-relish-galette
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-beef-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/french-carrot-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-chicken-souvlaki
https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-cheeseburger
https://www.sainsburysmagazine.co.uk/recipes/mains/glazed-pork-with-mango-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/houmous-flatbread-pizzas
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-saltimbocca-burger-with-crispy-prosciutto-and-parmesan-mayo
https://www.sainsburysmagazine.co.uk/recipes/mains/smash-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-chicken-milanese-with-shaved-asparagus-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/tortelloni-with-minty-pea-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/prawns-with-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-sea-bass-larb
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-and-chip-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/gazpacho
https://www.sainsburysmagazine.co.uk/recipes/mains/spice-crusted-tuna-with-polyphenol-rich-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-rice-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-and-roasted-cauliflower-with-herby-bulgur-wheat-and-tahini
https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-poke-recipe-tofu-avocado
https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-and-goats-cheese-open-sandwich
https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-style-egg-mayo-sandos
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-herby-yogurt-and-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-in-a-basket
https://www.sainsburysmagazine.co.uk/recipes/mains/hasselback-halloumi-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-pineapple-skewers
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pigs-in-blankets-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-steak-tacos-with-cucumber-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-gyoza-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/vibrant-watercress-and-pea-soup-with-bacon-crumbs
https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-bream-with-pickled-radish-and-pea-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-saffron-chicken-white-bean-and-chorizo-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-warm-salmon-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-lentil-and-roast-tomato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-and-bean-mash-with-cheesy-fennel
https://www.sainsburysmagazine.co.uk/recipes/mains/dill-and-feta-rice-with-crispy-fried-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-puff-plait
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-tenderloin-with-apples-and-walnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-lemon-olive-butter-and-new-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-stuffed-aubergines
https://www.sainsburysmagazine.co.uk/recipes/mains/rack-of-lamb-with-anchovy-salsa-and-spring-vegetables
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-tomato-chicken-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-moussaka
https://www.sainsburysmagazine.co.uk/recipes/mains/impossible-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sweet-chilli-halloumi-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sesame-beef-and-spring-onion-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/kale-and-chickpea-linguine-with-a-nutty-crunch
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-trout-and-pea-cacio-e-pepe
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-greens-and-white-bean-rigatoni
https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-chicken-with-pepper-caprese
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-beef-stifado
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-brisket-and-black-bean-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/garlicky-asparagus-carbonara
https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-minced-beef-and-mushroom-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-tofu-with-zingy-lime-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/egyptian-filo-pie-with-chopped-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-sweetcorn-brunch-fritters
https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-oven-baked-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/jersey-royal-and-crab-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/pub-style-steak-sarnie-with-beer-onions
https://www.sainsburysmagazine.co.uk/recipes/mains/silk-handkerchiefs-with-smoked-beurre-blanc
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-satay-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-greens-shakshuka
https://www.sainsburysmagazine.co.uk/recipes/mains/triple-mustard-roast-beef-and-yorkshires
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-with-spring-veg-and-dill-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-al-limone
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chilli-con-carne-from-knorr-stock-pots
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-knorr-spaghetti-bolognese
https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-lamb-with-mustardy-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/butter-basted-sage-pork-chops-with-beetroot-puree-and-garlic-chard
https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-chicken-livers
https://www.sainsburysmagazine.co.uk/recipes/mains/parma-chicken-schnitz-with-cheesy-kale-and-celeriac-fries
https://www.sainsburysmagazine.co.uk/recipes/mains/noodle-omelette-with-sweet-chilli-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-easy-pea-and-seafood
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-and-potatoes-with-preserved-lemon-aioli
https://www.sainsburysmagazine.co.uk/recipes/mains/hasselback-roast-squash-with-whipped-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-roast-lamb-with-provencal-panzanella
https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-flatbread-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-kleftiko
https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-pillows-in-silky-green-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-lamb-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chilli-with-squash-and-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/welsh-rarebit-galette
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-bacon-and-white-bean-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/honey-mustard-chicken-and-apples
https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-mushrooms-with-chimichurri-parsnip-and-butter-bean-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-citrus-pork
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tartiflette-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-sweet-potato-succotash
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-gnocchi-and-sausage-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-sauerkraut-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-lamb-meatballs-in-a-creamy-porcini-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-short-rubs-with-sweet-potato-mash-and-pickled-walnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/surf-and-turf-skillet-with-garlic-dippers
https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roast-cacio-e-pepe-cauliflower
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-tofu-goulash
https://www.sainsburysmagazine.co.uk/recipes/mains/chunky-chorizo-and-butter-bean-soup-1
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-coconut-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-spinach-and-mushroom-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-mushroom-grains-with-date-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-lamb-chops-with-sweet-potato-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-and-sausages-with-pickled-peppers
https://www.sainsburysmagazine.co.uk/recipes/mains/colcannon-smoked-fish-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-aubergine-steak-with-creamed-corn-and-a-herby-green-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/herby-meatballs-with-broccoli-pesto-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/pie-and-liquor
https://www.sainsburysmagazine.co.uk/recipes/mains/three-mushroom-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-with-sausage-and-chicory
https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-and-apple-soup-with-horseradish-dill-dumplings
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-socca-pancake-with-smoked-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/chunky-chorizo-and-butter-bean-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/stove-top-mac-and-cheese-with-pancetta-crumbs
https://www.sainsburysmagazine.co.uk/recipes/mains/beigels-or-bagels
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-bhuna
https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-risotto-with-celery-and-tarragon
https://www.sainsburysmagazine.co.uk/recipes/mains/tahini-chicken-greens-and-beans
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-cod-patatas-bravas
https://www.sainsburysmagazine.co.uk/recipes/mains/milk-braised-pork-steaks-with-cabbage
https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-sausage-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-brunch-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-salmon-and-vegetable-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/emerald-dhal-with-golden-tofu
https://www.sainsburysmagazine.co.uk/recipes/mains/tofu-and-greens-noodle-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/gingered-mushroom-super-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-cajun-cauli-tacos-and-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-cauli-and-sweet-potato-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/seared-salmon-with-tarragon-red-wine-sauce-and-braised-chicory
https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-wine-vegan-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/double-dip-buttermilk-fried-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-and-paneer-torkari
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-pot-pie-with-puff-pastry-croutons
https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-chicken-with-citrus-and-lavender
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-mushrooms-madeira-sauce-and-celeriac
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-and-sprout-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-cheesy-shallot-tarte-tatin
https://www.sainsburysmagazine.co.uk/recipes/mains/double-chestnut-mushroom-wellington
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-and-confit-duck-with-bubble-and-squeak-cakes-cranberry-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/bigos-hunters-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-enchilada-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/root-veggie-nut-roast-with-chestnut-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-and-ginger-jam-glazed-ham-with-nutmeg-and-sweet-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-veggie-tarte-tatin
https://www.sainsburysmagazine.co.uk/recipes/mains/winter-chicken-caesar
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-smoked-salmon-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-parsnip-and-cashew-strudels
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-ale-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/brussels-sprout-chestnut-and-bacon-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-pork-and-potato-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-pasta-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/zesty-salmon-and-roasted-broccoli-bulgur
https://www.sainsburysmagazine.co.uk/recipes/mains/hariyali-guinea-fowl-with-roasted-onions
https://www.sainsburysmagazine.co.uk/recipes/mains/cider-poached-cod-with-spiced-mussels
https://www.sainsburysmagazine.co.uk/recipes/mains/campfire-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/cider-pulled-pork-and-campfire-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pumpkin-soup-with-nutty-bacon-crumble
https://www.sainsburysmagazine.co.uk/recipes/mains/cobweb-pancakes
https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-and-pear-soup-with-blue-cheese-toasts
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-cabbage-rolls
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/root-vegetable-and-coconut-fish-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-mozzarella-sausage-and-sweet-pepper-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-pithivier
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-harissa-lasagne-with-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-braised-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/tagliatelle-with-walnut-and-kale-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/maple-mustard-ham-and-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/traditional-welsh-rarebit
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-rice-and-mussels
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-tarragon-roast-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-potatoes-and-plums-with-marsala-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/negroni-chicken-skewers-with-fennel-and-orange-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-chicken-and-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-cod-parcels
https://www.sainsburysmagazine.co.uk/recipes/mains/summery-spaghetti-and-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/easy-honey-garlic-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-and-chickpeas-en-papillote
https://www.sainsburysmagazine.co.uk/recipes/mains/asian-style-prawn-and-grapefruit-salad-with-crispy-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/ribs-red-beans-and-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/crackling-good-pork-and-apple-traybake-with-crunchy-pickle-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-steak-and-blue-cheese-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-style-lamb-with-warm-houmous
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-pork-belly-with-pickled-plums
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-chicken-summer-tomato-and-basil-panzanella
https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-crusted-lamb-with-green-quinoa
https://www.sainsburysmagazine.co.uk/recipes/mains/golden-coconut-turkey-and-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-porcini-burger-with-truffle-mayo-and-crispy-onion-rings
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-tenderstem-broccoli-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-chickpea-and-chorizo-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/parmesan-chicken-butties-with-red-cabbage-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/taco-pizza-flatbreads-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/marinated-halloumi-with-honey-and-thyme
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-shoulder
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-feta-pizza-with-hot-honey
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-salmon-in-chard-with-caramelised-lemons-garlic-and-almond-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-squash-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-prawn-burger-with-sriracha-butter
https://www.sainsburysmagazine.co.uk/recipes/mains/lime-and-coconut-pork-belly-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/feta-stuffed-peppers-with-butter-bean-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/bedfordshire-clangers
https://www.sainsburysmagazine.co.uk/recipes/mains/the-high-steaks-sharing-board
https://www.sainsburysmagazine.co.uk/recipes/mains/afternoon-sea-sharing-board
https://www.sainsburysmagazine.co.uk/recipes/mains/peperonata-bucatini-with-sardines
https://www.sainsburysmagazine.co.uk/recipes/mains/green-couscous-with-torn-tahini-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/wensleydale-and-apple-tart-with-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-salmon-kedgeree
https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-lamb-burger-with-pickled-red-cabbage-chilli-and-garlic-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-prawn-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-bianca-with-ham-and-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-and-crispy-bean-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-tofu-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/runner-bean-linguine-with-ricotta
https://www.sainsburysmagazine.co.uk/recipes/mains/arabic-style-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-and-butter-bean-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fennel-olive-and-tomato-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-sea-bass-with-pesto-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/minty-lamb-with-rainbow-veg-and-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-shawarma-picnic-loaf
https://www.sainsburysmagazine.co.uk/recipes/mains/mango-chicken-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/ham-and-roasted-pepper-tortilla
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-basa-with-smoky-tomato-beans
https://www.sainsburysmagazine.co.uk/recipes/mains/green-goddess-salmon-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/honey-mustard-bacon-steaks-with-sweetcorn-crush
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-kipper-and-potato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/feelgood-fish-and-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-hawaiian-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-chicken-caprese
https://www.sainsburysmagazine.co.uk/recipes/mains/flavour-bomb-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/sherry-braised-chicken-with-chorizo-and-chickpeas
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-asparagus-and-bean-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-tarte-tatin
https://www.sainsburysmagazine.co.uk/recipes/mains/lancashire-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/massaman-beef-and-aubergine-skewers
https://www.sainsburysmagazine.co.uk/recipes/mains/chargrilled-chicken-with-zingy-coriander-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-paprikash
https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-beans-on-toast
https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-roast-lamb
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-beef-chow-mein
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-cutlets-with-pearl-barley
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-nova-scotia-lobster-roll
https://www.sainsburysmagazine.co.uk/recipes/mains/bobotie-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-satay-sitr-fry-with-a-sweet-sour-radish-pickle
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-chicken-roast-dinner
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-roasted-garlic-and-burnt-spring-onion-butter
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-spatchcock-poussins-with-barbecued-peach-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-portobello-veggie-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-ham-and-leek-filo-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-broccoli-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-en-papillote-with-caponata
https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-squash-and-salsa-di-noci
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-broccoli-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/pea-ham-and-mint-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-bacon-and-corn-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-cauliflower-biryani
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-with-figs-and-shallots
https://www.sainsburysmagazine.co.uk/recipes/mains/tex-mex-5-a-day-veggie-feast
https://www.sainsburysmagazine.co.uk/recipes/mains/corned-beef-hash
https://www.sainsburysmagazine.co.uk/recipes/mains/popeye-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-carrot-and-lentil-soup-with-bhajia-nuggets
https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-lamb-stir-fry-with-half-the-carbs-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-and-cheese-meat-pies
https://www.sainsburysmagazine.co.uk/recipes/mains/lancashire-cheese-leek-and-onion-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-smoked-fish-and-spinach-tagliatelle
https://www.sainsburysmagazine.co.uk/recipes/mains/meaty-mushroom-pho
https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-meatball-hotpot
https://www.sainsburysmagazine.co.uk/recipes/mains/rice-stuffed-onions-with-feta-and-dill
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-patties-with-nutty-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-fillet-with-garlicky-mushroom-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-gnocchi-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/veg-and-tofu-spicy-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-sausage-and-cabbage-saute
https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-and-ginger-beef-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/super-greens-thai-green-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-tomatoes-with-broccoli-smash
https://www.sainsburysmagazine.co.uk/recipes/mains/marmalade-sausages-with-parsnip-colcannon
https://www.sainsburysmagazine.co.uk/recipes/mains/lama-keema
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-swedish-style-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/clementine-cumberland-glazed-ham
https://www.sainsburysmagazine.co.uk/recipes/mains/patatas-bravas-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-fish-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-satay-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-beef-bourguignon
https://www.sainsburysmagazine.co.uk/recipes/mains/cullen-skink
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-stir-fry-with-quorn-and-plums
https://www.sainsburysmagazine.co.uk/recipes/mains/broken-lasagne-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/low-waste-butternut-squash-and-sage-rigatoni
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooker-golden-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-mushroom-and-leek-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-coconut-and-lentil-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-teriyaki-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-spaghetti-with-crispy-capers
https://www.sainsburysmagazine.co.uk/recipes/mains/curried-turkey-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-leftover-turkey-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/leftovers-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/poussins-with-potato-celeriac-and-porcini-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-turkey-minestrone
https://www.sainsburysmagazine.co.uk/recipes/mains/tinned-fish-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-sprout-colcannon
https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-cod-in-prosciutto-di-parma-with-green-pea-puree
https://www.sainsburysmagazine.co.uk/recipes/mains/sage-and-apricot-stuffed-pork-tenderloin-with-roasted-pear-parsnip-and-potato
https://www.sainsburysmagazine.co.uk/recipes/mains/cider-chicken-casserole
https://www.sainsburysmagazine.co.uk/recipes/mains/cinnamon-and-saffron-lamb-shank-with-roasted-roots
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-cod-with-sake-shiitake-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ipa-casserole
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-chestnut-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-thighs-with-mustardy-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-potato-cakes
https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-katsu-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/chestnut-and-mushroom-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/truffled-mushroom-soup-with-cheesy-croutes
https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-meatball-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-gammon-stew-with-buttermilk-dumplings
https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-and-garlic-butter-turkey
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-and-pesto-traybake-from-sacla
https://www.sainsburysmagazine.co.uk/recipes/mains/savoy-cabbage-gratin-with-herby-cheddar-crumbs
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chickpea-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-steak-with-cheesy-creamed-corn
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-halloumi-and-spinach-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/liver-bacon-and-onions-with-white-bean-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-restaurant-style-butter-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-paneer-and-pea-biryani
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-laksa-with-spicy-sambal
https://www.sainsburysmagazine.co.uk/recipes/mains/philly-cheeseteak-stuffed-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-chicken-tortellini-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-kale-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-green-peas-and-coconut-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/chilean-cottage-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-cheese-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-melt-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/murgh-makhani
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-barley-one-pot-with-zesty-gremolata
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chilli-prawn-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-arrabiata
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-pesto-pasta-alla-norma-with-sacla
https://www.sainsburysmagazine.co.uk/recipes/mains/sweetcorn-and-bacon-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/jaffna-lamb-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-smoky-bacon-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-basa-with-sweetcorn-pilau
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-harissa-and-marmalade-baked-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-beetroot-pesto-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-dill-cucumber-yogurt-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-chorizo-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-creamed-corn
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-halloumi-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-and-stilton-crumble
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-meatball-red-pepper-and-feta-rigatoni-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-clonakilty-blackpudding-stuffed-peppers
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-crispy-filo-feta-and-clonakilty-blackpudding-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-burgers-with-apple-and-fennel
https://www.sainsburysmagazine.co.uk/recipes/mains/oven-roasted-jerk-pork
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-lamb-filo-pie-with-chermoula-drizzle
https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-bacon-and-tomato-toastie-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/ham-hock-and-pea-fritters
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-corn-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caprese-grain-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-cheeseboard-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/smashed-broad-bean-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-egg-and-brown-rice-bowls
https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-butter-salmon-and-red-pepper-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-cashew-massaman-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-saag-makhani
https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-chicken-korma
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-orange-beet-salad-with-crispy-chickpeas
https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-broccolini-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/spanakopita-stuffed-peppers
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-hot-dogs-with-corn-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/texan-style-barbecue-beef-brisket-and-succotash
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-sage-and-blackberry-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/ribeye-with-burnt-spring-onion-aioli
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/venison-burgers-with-mustard-fries
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-tomato-zaatar-houmous-and-harissa-tarts
https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-lamb-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-nicoise-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-feta-greek-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/blt-panzanella-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/tortang-talong
https://www.sainsburysmagazine.co.uk/recipes/mains/red-lentil-koftes-with-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-kedgeree
https://www.sainsburysmagazine.co.uk/recipes/mains/minted-pea-and-lemon-carbonara
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-roasted-radishes
https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-steak-fajita-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/poutine
https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-glazed-tuna-with-ginger-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/sea-bass-with-buttered-brown-shrimp-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/trout-with-a-gremolata-crumb
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-spring-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-traybake-lemon-butter-asparagus
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-meatball-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/quick-roast-chicken-with-tarragon-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/charred-spring-onion-miso-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-carrot-and-halloumi-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-porcini-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-shoulder-of-lamb-with-chickpeas-onions-garlic-and-oregano
https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-and-pistachio-pesto-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/minty-lamb-chops-with-watercress-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-beef-short-ribs-with-haricot-beans-and-spinach
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-pork-shoulder
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sour-squash
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-harvest-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/crab-and-spring-onion-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-mushroom-cheesecake
https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-greens-and-mung-beans-with-turmeric-oil
https://www.sainsburysmagazine.co.uk/recipes/mains/honey-miso-chicken-squash-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-shrimp-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/scrambled-eggs-with-asparagus-salami-and-oregano
https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-and-caper-chicken-thighs
https://www.sainsburysmagazine.co.uk/recipes/mains/korean-style-beef-noodle-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/herby-spring-chicken-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/savoy-cabbage-with-celeriac-puree-and-brown-butter-hazelnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-spiced-meatloaf-roast
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-lentil-and-quinoa-pilaf
https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-cottage-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/mac-and-3-cheese-stuffed-squash
https://www.sainsburysmagazine.co.uk/recipes/mains/crab-stuffed-plaice-with-cucumber-cream-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-meatball-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pork-bao-buns-and-homemade-pickles
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-butternut-with-chickpeas-and-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-beef-and-noodle-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-trout-with-parsnip-rosti
https://www.sainsburysmagazine.co.uk/recipes/mains/jackfruit-faitas
https://www.sainsburysmagazine.co.uk/recipes/mains/saffron-salmon-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-balti
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-balmoral-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-sweet-potato-tomato-and-coconut-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-creamy-black-bean-jackets
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-beetroot-dhal-with-homemade-coriander-chutney
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-vegan-mushroom-and-leek-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/chard-and-chestnut-gnocchi-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-sweet-potato-and-bean-chilli-from-knorr-stock-pots
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-katsu-curry-from-the-vegetarian-butcher
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-pasta-primavera-from-knorr-stock-pots
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-potato-and-pea-curry-from-knorr-stock-pots
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-cajun-spiced-poke-bowl-with-winter-coleslaw-from-the-vegetarian-butcher
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-ultimate-vegan-tacos-from-the-vegetarian-butcher
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-5-spice-tofu
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-caraway-beef-and-beetroot-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/pavo-relleno-mexican-stuffed-roast-turkey
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pomegranate-and-clementine-slow-roast-lamb-leg
https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-seven-spice-pork-belly-with-cranberry-and-orange-ketchup
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-orange-glazed-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-parsnip-and-apple-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-roast-pork-loin-with-cranberry-glazed-pigs-in-blankets
https://www.sainsburysmagazine.co.uk/recipes/mains/easy-carve-roast-turkey-with-chestnut-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-stuffed-spuds
https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-turkey-on-the-barbecue
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-wensleydale-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/sugar-plum-ham
https://www.sainsburysmagazine.co.uk/recipes/mains/firecracker-chicken-with-cavolo-nero-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/chestnut-pesto-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-cauliflower-pasta-with-creamy-tahini-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-chicken-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-roots-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-mushroom-lentil-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-beetroot-and-squash-wellington
https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roast-celeriac-with-sage-and-walnut-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-tray-roast-with-balsamic-glaze-from-taste-the-difference
https://www.sainsburysmagazine.co.uk/recipes/mains/keralan-cauliflower-and-lentil-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-pea-and-watercress-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-barley-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-fennel-and-olive-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-veggie-lasagne-rolls
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-crumbed-cod
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-courgette-pesto-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/duck-marsala-and-chestnut-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-soy-and-sesame-tofu-with-broccoli-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pumpkin-and-sage-souffle
https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-cashews-and-lentils-with-date-chutney
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-katsu-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/leek-cacio-e-pepe
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-apple-and-stilton-strudel
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-toad-in-the-hole-with-sherry-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-root-and-barley-broth
https://www.sainsburysmagazine.co.uk/recipes/mains/tarka-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-aubergine-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-duck-legs-with-blackberries-and-celeriac-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-madras
https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-veggie-curry-with-radish-raita
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-turnip-and-feta-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/one-pot-pasta-with-bacon-and-peas
https://www.sainsburysmagazine.co.uk/recipes/mains/squash-and-chickpea-one-pot-with-herby-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/biang-biang-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-stuffed-herb-and-garlic-veggiea
https://www.sainsburysmagazine.co.uk/recipes/mains/eggs-florentine-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-gnocchi-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-chops-with-smashed-green-bean-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-harissa-seafood-stew-from-belazu
https://www.sainsburysmagazine.co.uk/recipes/mains/gingered-beef-ribeye-with-noodle-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-garlic-roast-chicken-with-crispy-ham-and-veggies
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-plum-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/spatchcock-bbq-chicken-with-burnt-onion-aioli-and-caramelised-lemons
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-cherry-tomato-and-olive-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-anchovy-butter-and-mustard-roast-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken-pilaf-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-tofu-and-cashew-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-salad-with-pork-scratching-croutons
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potatoes-with-chorizo-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-finger-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/veg-patch-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-scallop-and-pea-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-mussel-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-paneer-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/gravlax-baked-salmon-with-sweet-dill-mustard-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/pot-roast-chicken-with-cider-and-tarragon
https://www.sainsburysmagazine.co.uk/recipes/mains/brunch-burger-with-bacon-ketchup
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-broad-bean-and-seaweed-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/tear-and-share-spiced-veg-roll
https://www.sainsburysmagazine.co.uk/recipes/mains/minty-pea-and-bacon-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/carrot-dogs-with-dill-pickles-and-remoulade
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-and-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-griddled-vegetables-grain-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-salmon-fishcakes
https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-feta-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-lamb-kebabs-with-summer-pea-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-stew-cottage-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-ham-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-feta-tart-with-poppy-seed-pastry
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-veggie-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-mole-burritos
https://www.sainsburysmagazine.co.uk/recipes/mains/potato-and-ricotta-gnocchi-with-purple-sprouting-broccoli
https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-orzotto
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-kofta-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-yassa-with-hasselback-sweet-potato
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-salmon-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/butter-bean-pepper-and-chorizo-saute-with-seared-cod-and-asparagus
https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-pea-pesto-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-okra-soup-and-semo
https://www.sainsburysmagazine.co.uk/recipes/mains/fatty-meatloaf-with-bacon-cheese-and-tomato-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/charred-chorizo-peach-pitta-panzanella
https://www.sainsburysmagazine.co.uk/recipes/mains/groundnut-beef-stew-with-ground-rice-balls
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-lamb-and-yogurt-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-pork-and-fennel-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-with-jewelled-rice-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-miso-steak-with-sesame-sweet-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-with-lentil-tabbouleh
https://www.sainsburysmagazine.co.uk/recipes/mains/white-wine-coq-au-vin
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pork-meatloaf
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potatoes-with-almond-tahini
https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-crumbed-cod-with-minty-crushed-potatoes-and-peas
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-pork-rice-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-egg-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-with-a-spring-broth-of-asparagus-jersey-royals-and-mussels
https://www.sainsburysmagazine.co.uk/recipes/mains/stuffed-roast-pork-fillet-with-cannellini-bean-and-tomato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-skewers-with-satay-sauce-and-smashed-cucumber
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pork-skewers-with-cucumber
https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-lamb-burgers-with-pickled-vegetables
https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-chamoy-chicken-with-grains
https://www.sainsburysmagazine.co.uk/recipes/mains/swedish-sandwich-cake
https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-salmon-with-warm-saag-aloo-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-casserole-with-walnut-gremolata
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-prune-and-sherry-casserole
https://www.sainsburysmagazine.co.uk/recipes/mains/herb-crusted-salmon-with-garlic-fries
https://www.sainsburysmagazine.co.uk/recipes/mains/zesty-salmon-parcels-with-dill-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/jalapeno-crusted-salmon-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-spaghetti-with-squash-salami-and-pine-nuts
https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-style-chicken-with-braised-spring-veg-and-horseradish-cream
https://www.sainsburysmagazine.co.uk/recipes/mains/provencal-pine-nut-crusted-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/springtime-frittata-with-roasted-radishes
https://www.sainsburysmagazine.co.uk/recipes/mains/maple-mustard-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/stuffed-picnic-loaf
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-vegetable-pot-pies-with-oxo
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-caper-and-olive-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/masala-mackerel-with-coconut-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sour-pork-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-lamb-with-a-watercress-and-caper-crust
https://www.sainsburysmagazine.co.uk/recipes/mains/curried-halloumi-fritters-raita-and-fried-egg
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-lamb-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-beef-burrito-rolls-with-broccoli-stalk-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/colmans-mint-glazed-roast-leg-of-lamb
https://www.sainsburysmagazine.co.uk/recipes/mains/colmans-herbed-mustard-crust-roast-beef
https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-veggie-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/meatball-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-beef-ragout
https://www.sainsburysmagazine.co.uk/recipes/mains/leek-ham-and-apple-barley-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/seared-steak-taco-board
https://www.sainsburysmagazine.co.uk/recipes/mains/sage-onion-toad-in-the-hole-with-cider-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-steak-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-chicken-and-mushroom-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/batch-cook-honey-balsamic-beef-brisket
https://www.sainsburysmagazine.co.uk/recipes/mains/jackfruit-and-mushroom-stew-with-herb-dumplings
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roasted-lamb-with-spring-onion-mash-and-pistou
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-wellington-with-porcini-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-citrus-roast-duck-with-crispy-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/masala-cauliflower-and-aduki-bean-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-spiced-veggie-sausage-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-baked-beans
https://www.sainsburysmagazine.co.uk/recipes/mains/filo-fish-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-and-haggis-escalopes-with-whisky-and-chicken-butter-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/roots-barley-and-lamb-cawl-with-cheddar-puffs
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-mackerel-and-leek-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-island-lamb
https://www.sainsburysmagazine.co.uk/recipes/mains/moussaka-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-e-fagioli
https://www.sainsburysmagazine.co.uk/recipes/mains/horseradish-coated-cod-fillets-with-beetroot-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-masala-bean-pot-pies
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-cobbler-with-cheese-scones
https://www.sainsburysmagazine.co.uk/recipes/mains/venison-steak-diane-with-truffle-fries
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-roast-pork-and-winter-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-wine-pot-roast-duck
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-red-wine-stew-with-porcini-and-chestnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/triple-cheese-stuffed-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-sage-and-pine-nut-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-rib-of-beef-with-chimichurri
https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-sloe-gin-and-plum-ham
https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-spaghetti-bolognese
https://www.sainsburysmagazine.co.uk/recipes/mains/port-clementine-and-5-spice-ham
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-maple-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-four-cheese-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/vineyard-chicken-with-crispy-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-pork-steaks
https://www.sainsburysmagazine.co.uk/recipes/mains/porchetta-style-turkey
https://www.sainsburysmagazine.co.uk/recipes/mains/5-a-day-turmeric-traybake-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/curried-lamb-shanks-with-mauritian-mango-kutcha
https://www.sainsburysmagazine.co.uk/recipes/mains/jewelled-stuffed-butternut-squash
https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-tacos-with-feta-and-watermelon-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/pepper-and-chorizo-pie-with-a-cornbread-crust
https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-pork-and-crispy-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/wine-braised-sausages-with-olive-oil-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/brown-butter-mushroom-and-sage-pappardelle
https://www.sainsburysmagazine.co.uk/recipes/mains/cheeseboard-nachos
https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-soup-toastie
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-sausage-gnocchi-hotpot
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-mozzarella-turkey-patties-and-griddled-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/plum-and-stilton-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/provencale-daube-style-pot-roast-beef
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-spanish-style-shepherds-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-and-egg-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-pork-with-rosemary-potatoes-and-sprouting-broccoli
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beef-fillet-with-celeriac-puree
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-cabbage-and-leek-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca-baked-cod
https://www.sainsburysmagazine.co.uk/recipes/mains/fajita-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/green-curry-roast-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-eggs-with-garlic-toasts
https://www.sainsburysmagazine.co.uk/recipes/mains/salt-and-pepper-salmon-with-spring-onion-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-tapenade-tart-with-broccoli-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/curried-fish-pie-with-spiced-potato-topping
https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-sage-and-ricotta-open-crust-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-topped-chicken-with-broccoli-smash
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-lamb-patties-and-sweet-potato-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-hake-with-almond-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-chicory-apple-and-walnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/baja-fish-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-caramel-pork-noodle-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-aubergine-pesto-feta-gyros
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-gammon-and-pineapple-with-colcannon
https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-nectarine-and-prosciutto-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-saffron-and-paprika-spiced-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-prawn-and-chorizo-paella
https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-with-mussels-bay-and-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-with-leeks-lime-and-curry-butter
https://www.sainsburysmagazine.co.uk/recipes/mains/sardines-with-cucumber-salsa-verde-salad-and-garlic-mayonnaise
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-rags-with-speedy-sausage-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-cauliflower-steaks-with-pineapple-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-chickpea-and-carrot-veggie-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-inspired-sausage-plait
https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/coriander-feta-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-watercress-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-with-zesty-peas-and-mint
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-prosciutto-cannelloni
https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-sweet-potato-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-chicken-with-green-grains
https://www.sainsburysmagazine.co.uk/recipes/mains/woolton-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-and-rainbow-veg-grain-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-lemon-chicken-casserole
https://www.sainsburysmagazine.co.uk/recipes/mains/campers-caponata
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-salad-with-halloumi
https://www.sainsburysmagazine.co.uk/recipes/mains/store-cupboard-jambalaya
https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-mac-and-cheese-with-caramelised-tomatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-sausages-with-beer-braised-onions
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-risotto-with-basil-oil
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-red-curry-noodle-soup-with-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/charred-spring-onion-and-ricotta-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/open-lasagne-with-roasted-squash-crispy-prosciutto-and-amaretti
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-romesco-sauce-and-butterbean-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-split-pea-dhal-with-roasted-sweet-potato
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-leg-of-lamb-with-garlic-and-rosemary
https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-caponata-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-and-lime-fish-and-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-minestrone
https://www.sainsburysmagazine.co.uk/recipes/mains/minty-salsa-verde-lamb-with-lemony-potatoes-and-leeks
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-chicken-with-roast-veg-and-olives
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-chicken-fennel-and-garlic-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-sardine-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-bourguignon
https://www.sainsburysmagazine.co.uk/recipes/mains/fattoush-salad-with-herbed-labneh-balls
https://www.sainsburysmagazine.co.uk/recipes/mains/kale-and-ricotta-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-smoked-haddock-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-lamb-tabbouleh-and-beetroot
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-turkey-larb
https://www.sainsburysmagazine.co.uk/recipes/mains/curried-mushroom-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-beef-stew-with-gruyere-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/lobster-mac-n-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-miso-mushroom-casarecce
https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-aubergine-rice-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-steak-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-spring-veg-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-hake-lime-and-coconut-pilaf
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-tarragon-crushed-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-avocado-corn-and-feta-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-wrapped-cod-and-mediterranean-vegetables
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-tomato-chicken-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/hake-celeriac-puree-and-balsamic-chickpeas
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-with-caper-lemon-sauce-and-green-bean-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-burgers-with-tahini-coleslaw
https://www.sainsburysmagazine.co.uk/recipes/mains/golden-glow-chicken-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/shepherds-pie-with-root-veg-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-chicken-ramen
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-italian-savoury-bread-pudding
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-stuffed-pork-with-roasted-roots
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-winter-veg-traybake-with-speedy-rhubarb-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-tomato-fusilli-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/scouse
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-soy-beef-and-green-bean-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/green-goddess-glow-bowl-with-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-veggie-rice-with-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-turkey-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/cider-lamb-chops-with-celeriac-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pork-on-a-bed-of-cider-soaked-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-chinese-beef-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/haggis-neeps-and-tatties-tart-with-whisky-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/restorative-chicken-and-leek-noodle-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/tofu-katsu-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-peanut-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-lasagne-with-butternut-squash-and-hazelnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/crayfish-and-champagne-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/curried-smoked-haddock-chowder
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-blue-cheese-ciabatta-with-beetroot-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-stuffing-roulades-with-sprout-and-chestnut-potato-cakes
https://www.sainsburysmagazine.co.uk/recipes/mains/citrus-salmon-with-green-quinoa
https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-madras-with-mango-and-cucumber-raita
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-with-stir-fried-sprouts
https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-lemon-butter-roasted-turkey
https://www.sainsburysmagazine.co.uk/recipes/mains/cashew-and-pistachio-stuffing-nut-roast
https://www.sainsburysmagazine.co.uk/recipes/mains/porcini-bacon-and-sage-roast-rib-of-beef-with-port-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-beef-fillet-with-truffle-mayo
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-chicken-and-leek-sourdough-tartiflette
https://www.sainsburysmagazine.co.uk/recipes/mains/witches-brew-smoky-beef-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pork-with-balsamic-mushrooms-crispy-potatoes-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-chicken-and-courgettes-with-pistachio-yogurt-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/luxe-smoked-salmon-gnocchi-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-florentine-mushroom-sauce-and-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/easy-cheesy-pasta-with-greens
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-loin-with-beetroot-and-lentil-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/duck-and-red-wine-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-beef-stew-and-cheese-biscuits
https://www.sainsburysmagazine.co.uk/recipes/mains/kipper-and-salmon-fish-cakes
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-ham-and-greens-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/marmite-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/ham-and-leek-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-hotdogs-with-crispy-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-jalapeno-quesadillas-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-spinach-tortelloni-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/butter-bean-makhani
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chinese-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/tray-baked-squash-beetroot-and-giant-couscous-with-goats-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-harvest-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-leek-pie-with-crispy-potato
https://www.sainsburysmagazine.co.uk/recipes/mains/fusilli-with-spinach-mushrooms-and-goats-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-taco-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-falafel-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/alpine-chicken-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/beer-marmite-braised-beef-with-horseradish-dumplings
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-salmon-nicoise-with-soft-boiled-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/jerk-pork-belly-with-mango-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-tofu-and-broccoli-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-chicken-and-quinoa-with-corn
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-and-chickpea-kofte
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-steaks-with-squash-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ale-stew-with-cheese-and-mustard-croutes
https://www.sainsburysmagazine.co.uk/recipes/mains/truffled-mushroom-mac-n-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/pea-ham-and-goats-cheese-tortilla
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-veg-gnocchi
https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-pea-and-cauliflower-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-schnitzel-salad-with-cheats-garlic-aioli
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-coconut-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-butter-bean-summer-casserole
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-ratatouille-with-lentils-and-lime-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-fritters-with-pasta-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/fire-grilled-steak-sandwich-with-chimichurri
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-chickpea-and-chorizo-stew-with-fried-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-burgers-with-cucumber-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-roast-mackerel-with-spiced-chickpeas-and-raita
https://www.sainsburysmagazine.co.uk/recipes/mains/buttery-courgette-and-rocket-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ale-pie-with-pickled-walnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-mackerel-with-orange-and-beetroot-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-with-orange-cumin-and-apricot-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/honey-and-lemon-pork-with-grains
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parmo
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-aubergine-with-tabbouleh-and-walnut-turmeric-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-traybake-with-spinach-and-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-chicken-and-peach-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/maple-smoked-salmon-and-potato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/cashew-carrot-pomegranate-and-halloumi-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sloppy-joe-bowls
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-cheese-dogs-with-mustard-onions
https://www.sainsburysmagazine.co.uk/recipes/mains/satay-steak-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/mustard-honey-salmon-with-veggie-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/all-day-breakfast-gnocchi-fry-up
https://www.sainsburysmagazine.co.uk/recipes/mains/springtime-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/pea-and-ricotta-gnocchi-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-pork-with-sweet-potato-wedges-and-sweetcorn-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-cashew-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-burger-and-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/oregano-roast-beef-with-crispy-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-crusted-lamb-rack
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-pea-fishcakes
https://www.sainsburysmagazine.co.uk/recipes/mains/california-salmon-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/tahini-and-olive-spaghetti-with-tomato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-and-goats-cheese-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/cornish-pasties
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-apricot-chickpea-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/cantonese-aubergine
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-fajitas-with-jalapeno-salsa-and-avocado
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-bean-meatballs-in-spiced-tomato-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sticky-pork-char-siu
https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-pasta-with-ham-crisps
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-chicken-curry-with-thai-style-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-jackets-with-beetroot-ratatouille
https://www.sainsburysmagazine.co.uk/recipes/mains/gin-and-tonic-salmon-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-salmon-fishcakes-with-loaded-potato-skins
https://www.sainsburysmagazine.co.uk/recipes/mains/butterflied-lamb-with-kentucky-mop-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-black-bean-and-sweet-potato-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/royal-almond-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-asparagus-tagliatelle
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-paneer-skewers
https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-halloumi-burger-with-sweetcorn-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/brinner-burrito-with-marmite-hollandaise
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chickpea-and-sweet-potato-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-mustard-pork-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-baked-fish-with-orzo
https://www.sainsburysmagazine.co.uk/recipes/mains/individual-minted-lamb-wellingtons
https://www.sainsburysmagazine.co.uk/recipes/mains/maple-soy-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/pomegranate-beef-with-spiced-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-chicken-and-grains
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-roast-lamb-with-leeks-and-caerphilly
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-leg-of-lamb-with-fennel
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-roast-lamb-recipe-gizzi-erskine-reicpe
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-spiced-whole-celeriac-with-raita
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-tortellini-minestrone
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-goats-cheese-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-fajita-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-asparagus-crumble
https://www.sainsburysmagazine.co.uk/recipes/mains/sweetcorn-courgette-and-feta-oven-baked-fritters
https://www.sainsburysmagazine.co.uk/recipes/mains/sizzling-garlic-prawn-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-cauliflower-and-quinoa-goodness-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-salmon-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-jalapeno-buddha-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/choucroute-garnie
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-dill-cullen-skink
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-meatballs-with-courgetti
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-aubergine-squash-feta-grain-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-chicken-wraps-with-quick-pickled-cucumber
https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/friday-night-fish-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-baked-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-lamb-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-omelette-with-tomato-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-kiev-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-roast-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-stir-fry-with-chilli-and-basil-2
https://www.sainsburysmagazine.co.uk/recipes/mains/pineapple-pork-mince-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-chow-mein
https://www.sainsburysmagazine.co.uk/recipes/mains/quinoa-grape-and-feta-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-sage-n-squash-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/soy-butter-steak-with-bearnaise-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/sardine-croquettes
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-beans-on-sourdough
https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-pork-with-water-chestnuts-and-greens
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-stilton-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/sea-bass-with-parsley-and-anchovy-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-beef-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-satay-with-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-patties-with-honey-roast-butternut-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-cornbread-cobbler
https://www.sainsburysmagazine.co.uk/recipes/mains/bolognese-pancakes
https://www.sainsburysmagazine.co.uk/recipes/mains/pappardelle-with-rosemary-and-sausage-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-creamy-leeks-and-butter-beans
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-leek-carbonara
https://www.sainsburysmagazine.co.uk/recipes/mains/best-ever-lamb-casserole
https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-orange-and-almond-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/fondue-mac-and-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-and-halloumi-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/red-pepper-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-vegetable-and-pancetta-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sausage-and-parsnip-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/ham-sweetcorn-and-cheese-bread-pudding
https://www.sainsburysmagazine.co.uk/recipes/mains/rich-lamb-and-pork-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/meatball-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-garlic-thyme-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-sausage-gnocchi-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-lamb-and-sweet-potato-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-chicken-pie-with-chorizo-and-olives
https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-black-bean-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-apple-winter-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-fishcakes
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-pie-with-cheesy-rosti-topping
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-turkey-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/fridge-forage-tex-mex-rice-with-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-mango-chicken-with-fragrant-pilau
https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-lentil-and-halloumi-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-with-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-korma
https://www.sainsburysmagazine.co.uk/recipes/mains/super-green-halloumi-omelette
https://www.sainsburysmagazine.co.uk/recipes/mains/walnut-and-red-pepper-stuffed-cabbage-leaves
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-fennel-and-orange-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-rogan-josh
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-shepherds-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-brisket-with-mexican-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/pineapple-pulled-pork
https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/rolled-turkey-breast-with-rosemary-and-cranberry-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-baked-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beetroot-and-apple-stuffing-loaf
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cauliflower-tart-with-thyme-and-pistachios
https://www.sainsburysmagazine.co.uk/recipes/mains/herbed-mackerel-with-beetroot-puree-and-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/duck-stir-fry-with-plum-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-squash-with-pumpkin-seed-and-cranberry-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-turkey-with-turkey-butter-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/brussels-sprouts-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-crispy-duck-with-cranberry-glaze
https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-gravlax-pasta-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-cauliflower-sage-and-walnut-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/irish-lamb-ragout
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-chook-with-pea-and-parsley-dumplings
https://www.sainsburysmagazine.co.uk/recipes/mains/simple-sausage-and-rosemary-cassoulet
https://www.sainsburysmagazine.co.uk/recipes/mains/philadelphia-style-spiced-steak-naanwiches
https://www.sainsburysmagazine.co.uk/recipes/mains/stilton-mac-and-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/somerset-pork-with-apples-and-cider
https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-sweet-potato-rice-bowls
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-cod-with-sweet-and-smoky-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-tomato-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-pie-potato-skins
https://www.sainsburysmagazine.co.uk/recipes/mains/pearl-barley-risotto-with-blue-cheese-and-squash
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-beef-brisket-with-cranberries
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-citrus-and-spice-turkey
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-shawarma
https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-and-onion-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/full-english-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/coq-au-vin-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-spinach-dhansak
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-boursin-chicken-with-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-beef-brisket
https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-stuffed-potato-skins
https://www.sainsburysmagazine.co.uk/recipes/mains/malaysian-sweet-potato-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-beet-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/hake-with-samphire-and-malai-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/sardine-and-tomato-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-beer-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-melt-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chickpea-and-spinach-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-paprika-goulash
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-and-broccoli-pizzette
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-duck-breast-with-couscous-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-broad-beans-mint-and-ricotta
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-sweet-potato-lentils-and-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/sri-lankan-chicken-and-chickpea-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-cider-and-parsnip-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-shells-with-sardines-fennel-and-saffron
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-masala-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-stroganoff
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-and-ham-hash
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-meatball-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-honey-baked-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-soup-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/minced-beef-tatties
https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-sweet-potato-and-lime-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-walnut-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-beef-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-mustardy-lentils-beetroot-and-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-shepherds-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/wild-mushroom-speltotto-with-pine-nuts
https://www.sainsburysmagazine.co.uk/recipes/mains/healthier-bangers-and-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-quesadillas
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-lemon-and-honey-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-sausage-and-crispy-potato-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/indian-spiced-lamb-chops-with-mango-and-cucumber-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-fish-and-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-couscous-with-roast-squash-and-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/battered-haddock-and-mushy-pea-fishcakes-with-homemade-tartare-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-madras-sandwich
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-herb-and-golden-ale-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/ham-egg-and-chip-tortilla
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-tagliatelle-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-salmon-and-summer-veg-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-courgette-and-basil-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-edamame-and-spring-onion-omelette
https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-stuffed-aubergine
https://www.sainsburysmagazine.co.uk/recipes/mains/hariyali-sea-bream
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-mozzarella-melts
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-mezze-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/fennel-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/store-cupboard-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-indian-lamb-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-pea-and-mushroom-tikka-masala
https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-feta-and-pesto-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-chicken-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-kale-and-butter-bean-one-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-greens-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-mac-and-greens
https://www.sainsburysmagazine.co.uk/recipes/mains/gammon-with-mustard-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-lentil-nut-roast
https://www.sainsburysmagazine.co.uk/recipes/mains/green-veg-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-style-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-spiced-cauliflower
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-tomato-tagliatelle
https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/coffee-and-rum-marinated-steaks
https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-bianca-with-broccoli-and-garlic
https://www.sainsburysmagazine.co.uk/recipes/mains/margherita-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-baked-eggs-with-peperonata
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-roasted-spinach-polpette-and-tomatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-salmon-with-watercress-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-cajun-grilled-corn-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/super-moist-roast-chicken-with-lemon-and-thyme
https://www.sainsburysmagazine.co.uk/recipes/mains/watercress-pesto-and-lemon-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/easy-chicken-shawarma
https://www.sainsburysmagazine.co.uk/recipes/mains/aloo-gosht
https://www.sainsburysmagazine.co.uk/recipes/mains/traybaked-teriyaki-mackerel-with-greens
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parma-ham-wellingtons
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-crayfish-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fish-parcels
https://www.sainsburysmagazine.co.uk/recipes/mains/oven-roasted-chicken-and-avocado-with-asparagus
https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-pasta-with-lemon-avocado-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/stacked-sweet-potato-burgers-with-avocado-lime-cream
https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-stuffed-baked-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-carbonara
https://www.sainsburysmagazine.co.uk/recipes/mains/blt-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mango-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-prawns-with-cinnamon-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/antipasti-tartlets
https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-steak-and-roasted-new-potato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-steak-with-tomato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-smoked-salmon-and-hollandaise-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-onion-goats-cheese-and-rocket-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-finger-sarnies-with-fennel-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/radish-spring-onion-and-watercress-salad-with-baked-trout
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-roast-chicken-with-root-vegetable-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/giant-sharing-bacon-jam-rosti
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-filo-parcel
https://www.sainsburysmagazine.co.uk/recipes/mains/tenderstem-and-goats-cheese-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/persian-herb-rice-with-broad-beans
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chorizo-and-mixed-grain-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-kedgeree
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fish-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/smashed-salmon-hash
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-peanut-pork-with-tenderstem
https://www.sainsburysmagazine.co.uk/recipes/mains/chickpea-cauliflower-and-squash-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/satay-beef-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-and-aubergine-with-minted-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-spanish-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle-meatball-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-pastilla
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-pea-and-watercress-tagliatelle
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-chicken-with-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/riojan-pork-chorizo-and-potato-pot
https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-badger
https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-pork-with-sweet-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-diane
https://www.sainsburysmagazine.co.uk/recipes/mains/artichoke-spaghetti-with-romanesco
https://www.sainsburysmagazine.co.uk/recipes/mains/dill-mustard-trout-with-smashed-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-stuffed-with-lemony-goats-cheese-and-tarragon
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-with-cauliflower-couscous-and-chermoula-tomatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-smoked-salmon-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-steak-with-cheesy-polenta
https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-goats-cheese-and-prosciutto-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-madeira-and-chestnut-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/squash-stilton-and-kale-pies-with-walnut-pastry
https://www.sainsburysmagazine.co.uk/recipes/mains/sicilian-roast-chicken-with-fennel-and-blood-oranges
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-meatballs-with-red-pepper-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-roast-chicken-with-tomato-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-lamb-with-quick-pickled-cucumber
https://www.sainsburysmagazine.co.uk/recipes/mains/kale-carbonara
https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-and-spinach-tortelloni-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/whisky-venison-steaks-with-neeps-and-tatties
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mango-chipotle-rice-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/campfire-roast-fish-with-lemon-and-fennel
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-red-pork
https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-style-crispy-duck-salad-with-blood-oranges
https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-prawn-noodle-jar
https://www.sainsburysmagazine.co.uk/recipes/mains/leaner-chicken-cacciatore
https://www.sainsburysmagazine.co.uk/recipes/mains/pernod-baked-mackerel-with-fennel-and-watercress
https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-chicory-and-blue-cheese-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/venison-and-red-wine-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-salmon-with-sesame-noodles-and-cucumber-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/venison-sausage-and-squash-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-gorgonzola-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-ham-and-pea-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-chicken-nachos
https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-cheese-and-pepper-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/sizzling-squash-with-herby-cucumber-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/noodles-with-chicken-and-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-giant-cous-cous-broth
https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-squash-and-walnut-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-and-black-rice-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-sumac-and-pomegranate-soup-with-roasted-peanuts-coriander-and-lime
https://www.sainsburysmagazine.co.uk/recipes/mains/golden-coconut-and-cauliflower-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/cumin-carrot-and-lamb-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-potato-and-ham-hash-with-sage-fried-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-turkey-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-beef-wellington
https://www.sainsburysmagazine.co.uk/recipes/mains/red-leicester-pickled-onion-and-watercress-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-sage-and-chestnut-wellington
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-tofu-with-miso-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-smoked-salmon-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-gewurztraminer-and-grapes
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-prawn-and-coconut-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/umami-chicken-ramen
https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-cod-and-brussels-sprout-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-orange-chicken-with-broccoli
https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-gnocchi-and-turkey-hash
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-corn-and-bean-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-lamb-pie-with-almonds-and-apricots
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-balsamic-onions-and-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-dhal-with-roasted-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/tartiflette
https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-mac-n-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/colombian-style-chicken-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-style-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potatoes-stuffed-with-chickpeas-and-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/shepherds-sag-aloo
https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-biryani
https://www.sainsburysmagazine.co.uk/recipes/mains/braised-fennel-with-figs
https://www.sainsburysmagazine.co.uk/recipes/mains/asian-chicken-and-ginger-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/moussaka
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-style-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/gammon-and-egg-with-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-lamb-skewers-with-herby-bulgur
https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-schnitzel-with-rose-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-yogurt-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/massaman-curry-buddha-bowls
https://www.sainsburysmagazine.co.uk/recipes/mains/macaroni-cheese-with-crispy-bacon-and-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/chick-n-sours-korean-chicken-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-pesto-tortelloni-broth
https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-spiced-potato-and-zhoug-pitta
https://www.sainsburysmagazine.co.uk/recipes/mains/squid-and-mackerel-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-aubergine-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/pizzadilla
https://www.sainsburysmagazine.co.uk/recipes/mains/feta-aubergine-pomegranate-and-harissa-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/watermelon-jelly-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/cheeseburger-nacho-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle-giant-fishcake
https://www.sainsburysmagazine.co.uk/recipes/mains/zaatar-potato-salad-with-olives-and-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-spelt-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-spinach-egg-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-ginger-chicken-pho
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-sushi-salad-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-korma
https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-and-saffron-chicken-pitta-with-blackened-chilli-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-salmon-with-blueberry-lime-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-tagliatelle-with-roasted-red-pepper-cream-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/golden-vegetable-dhal-with-soft-boiled-eggs
https://www.sainsburysmagazine.co.uk/recipes/mains/rice-spinach-and-eggs-with-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/pad-kra-pao
https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-ricotta-lemon-and-herbs
https://www.sainsburysmagazine.co.uk/recipes/mains/salt-and-pepper-rib-of-beef-with-bloody-mary-sauce-and-roast-potato-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/shoyu-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-tuna-burgers-with-chilli-mango-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/charred-broccoli-borlotti-bean-and-hazelnut-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-cod-with-sesame-cucumbers
https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-chicken-udon-noodles-2
https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-pine-nut-pesto-penne
https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-tea-brined-chicken-with-warm-rice-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-gnocchi-with-savoury-granola
https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roasted-cauliflower
https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-goats-cheese-risotto-mint-gremolata
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-chickpea-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-chicken-and-shiitake-broth
https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-pineapple-and-herb-roasted-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/venison-steaks-with-miso-parsnip-fries
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-coconut-rice-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-marmalade-chicken-drumsticks-with-blue-cheese-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-manchego-potato-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/caribbean-lamb-hotpot-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chorizo-pie-saffron-pastry-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/chermoula-cod-bulgar-salad-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-wonton-soup-one-pound-meals-miguel-barclay
https://www.sainsburysmagazine.co.uk/recipes/mains/garam-masala-chicken-spicy-potatoes-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-roast-chicken-with-cornbread-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/tenderstem-and-ham-quiches
https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-pork-patties-with-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-adobo-with-roast-squash-rice-and-smoky-tomato-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/barbacoa-beef-brisket-tacos
https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-goats-cheese-tart-recipe
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-butternut-with-honey-and-walnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-lamb-pilaf
https://www.sainsburysmagazine.co.uk/recipes/mains/yellow-duck-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/manchego-fritters-with-seville-orange-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/farc-on
https://www.sainsburysmagazine.co.uk/recipes/mains/kedgeree-with-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/tom-daley-s-harissa-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-shallot-and-mushroom-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/houmous-pepper-goats-cheese-and-sweet-chilli-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-and-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/higgledy-piggledy-greek-beef-pasta-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-tacos-with-smoky-coleslaw
https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-schnitzel-with-lemon-and-caper-butter
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-cashew-stir-fried-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-sea-bass-with-ginger-and-spring-onion
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-bites-with-sweet-chilli-dip
https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-mac-n-cheese-cake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-gumbo
https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-salmon-and-kale-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/quick-smoked-haddock-and-cauliflower-chowder
https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-black-bean-and-squash-burrito-bowl
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-salad-of-pan-fried-duck-with-clementines
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-beans-on-toast
https://www.sainsburysmagazine.co.uk/recipes/mains/porky-pancetta-ragu
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beef-sirloin-with-juniper-and-black-pepper-2
https://www.sainsburysmagazine.co.uk/recipes/mains/savoury-parsnip-and-pecan-slice
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-pearl-barley-risotto-with-feta
https://www.sainsburysmagazine.co.uk/recipes/mains/honey-roast-salmon-and-broccoli-crustless-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/korean-barbecue-beef-stir-fry-with-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/french-style-chicken-and-fennel-braise
https://www.sainsburysmagazine.co.uk/recipes/mains/charred-cauliflower-and-coconut-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-crown-with-herb-butter
https://www.sainsburysmagazine.co.uk/recipes/mains/cranberry-and-orange-glazed-ham
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-coulibiac
https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-roasted-cod-with-chorizo-and-persimon
https://www.sainsburysmagazine.co.uk/recipes/mains/jose-hot-dogs
https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-streetcorn-hot-dogs
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-chop-root-veg-and-apple-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/bengali-coconut-fish-pie-with-mustard-seed-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/satay-chicken-with-soba-and-cucumber-noodle-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/singapore-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-beef-pho
https://www.sainsburysmagazine.co.uk/recipes/mains/beer-battered-tofu-tacos-with-a-chipotle-cashew-soured-cream
https://www.sainsburysmagazine.co.uk/recipes/mains/korean-style-stew-with-marinated-tofu-pieces
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-n-sour-bahn-mi-baguettes
https://www.sainsburysmagazine.co.uk/recipes/mains/quick-pickled-rice-noodle-salad-with-soy-glazed-griddled-tofu
https://www.sainsburysmagazine.co.uk/recipes/mains/naan-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-watercress-chilli-pesto-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/firecracker-prawn-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-sausage-and-squash-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-wrapped-chicken-with-creamy-lentils
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-chicken-and-mushroom-hot-pot-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-ham
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-liver-pancetta-tagliatelle-with-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-cheese-toastie
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-easy-black-bean-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-chorizo-and-melting-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-houmous-and-harissa
https://www.sainsburysmagazine.co.uk/recipes/mains/white-bean-soup-with-candied-vanilla-bacon
https://www.sainsburysmagazine.co.uk/recipes/mains/pappardelle-with-prawns-and-vanilla-beurre-blanc
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-spiced-chicken-legs-roasted-with-lemons-tomatoes-and-celery
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cauliflower-pea-and-cashew-pilaf
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-tagine
https://www.sainsburysmagazine.co.uk/recipes/mains/tom-yum-prawn-noodle-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-fish-stew-with-cheese-toasts
https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-noodles-with-spinach-and-ricotta
https://www.sainsburysmagazine.co.uk/recipes/mains/hot-dogs-with-new-york-streetcart-onions
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sesame-pork-noodle-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/ghastly-ghoul-ash
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-pickled-veg-with-seared-tuna-and-herbs
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-duck-with-soft-herb-and-grapefruit-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/herb-omelette-with-feta-and-broad-beans
https://www.sainsburysmagazine.co.uk/recipes/mains/fresh-tomato-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/trout-with-broad-beans-and-courgettes-in-a-wine-and-cream-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-mustard-and-goats-cheese-galette
https://www.sainsburysmagazine.co.uk/recipes/mains/baharat-spiced-baked-aubergines-with-feta-fennel-and-spelt
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-beef-nectarine-and-rocket-salad-with-thai-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-and-mozzarella-pockets
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-spicy-charred-corn-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-glazed-sausages-and-onion-rings-2
https://www.sainsburysmagazine.co.uk/recipes/mains/hot-dogs-with-celeriac-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/tapanade-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-onion-topping
https://www.sainsburysmagazine.co.uk/recipes/mains/satay-noodle-lettuce-wraps-with-prawns-and-mango
https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-chicken-fatteh
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-smoked-mackerel-with-preserved-lemon-quinoa
https://www.sainsburysmagazine.co.uk/recipes/mains/hot-smoked-trout-and-beans-with-a-creamy-dill-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-garden-salad-with-peaches-and-mozzarella
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-burritos
https://www.sainsburysmagazine.co.uk/recipes/mains/cobb-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-soup-bowl-with-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-meze-pitta-pockets
https://www.sainsburysmagazine.co.uk/recipes/mains/loaded-pulled-pork-subs
https://www.sainsburysmagazine.co.uk/recipes/mains/salad-of-roasted-beetroot-goat-s-cheese-and-hazelnuts-on-wild-rocket
https://www.sainsburysmagazine.co.uk/recipes/mains/mac-n-cheese-pancetta-cups
https://www.sainsburysmagazine.co.uk/recipes/mains/zaatar-lamb-chops-with-salted-yogurt-sweet-herbs-and-pomegranate-molasses
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-rogan-josh-with-mint-and-coriander-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/quick-cook-ancho-con-carne-with-cucumber-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-new-potatoes-cherry-tomatoes-and-basil
https://www.sainsburysmagazine.co.uk/recipes/mains/https-www-youtube-com-watch-v-gjbibr6dv1s
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-loin-steaks-in-cider-and-tarragon-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-salmon-and-puy-lentil-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-chicken-with-stir-fried-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-cheddar-and-mushroom-pasta-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/masala-meatball-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-mackerel-with-toasted-fennel-and-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/miso-marinated-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/burger-with-smoky-stilton-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-roast-chicken-with-watercress-and-spiced-peaches
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-pistachio-koftas-with-fennel-radish-and-apricot-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-tuna-nicoise-ratatouille-stack
https://www.sainsburysmagazine.co.uk/recipes/mains/porchetta-style-roast-pork-with-a-courgette-chickpea-and-spinach-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-crusted-fish-with-ginger-pak-choi
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-squash-and-goats-cheese-salad-with-griddled-spring-onions-and-pumpkin-seed-crunch
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-pesto-and-pepper-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/charred-aubergine-and-tomato-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-chicken-and-olive-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/camembert-baked-with-maple-syrup-pecans-and-blueberries
https://www.sainsburysmagazine.co.uk/recipes/mains/brie-and-beef-baguette-with-beetroot-jam
https://www.sainsburysmagazine.co.uk/recipes/mains/twice-cooked-goats-cheese-souffle-s-with-apple-and-walnut-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-cutlets-with-harissa-yogurt-and-midget-trees
https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-creamy-cavolo-nero-and-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/bashed-chicken-with-walnut-and-watercress-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/summer-gnocchi-with-avocado-pesto-and-tomatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-fillet-with-asian-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-stuffed-picnic-rolls
https://www.sainsburysmagazine.co.uk/recipes/mains/sri-lankan-crab-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-maple-glazed-pork-chops
https://www.sainsburysmagazine.co.uk/recipes/mains/somerset-mussels
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-poussins-with-roasties
https://www.sainsburysmagazine.co.uk/recipes/mains/cote-de-boeuf-with-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-lemon-chicken-noodle-broth
https://www.sainsburysmagazine.co.uk/recipes/mains/mattar-paneer-a-medium-curry-of-peas-tomatoes-and-indian-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle
https://www.sainsburysmagazine.co.uk/recipes/mains/chickpea-burgers-in-roasted-portobello-buns
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cod-with-roasted-cauliflower-and-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-lettuce-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-lemon-garlic-and-thyme-chicken-kebabs
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chermoula-lamb-chops-with-herb-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chicken-drumsticks-with-ginger-barbecue-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/tarte-flambee-with-comte-and-watercress-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/anchovy-and-chilli-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/mustardy-mushroom-stroganoff
https://www.sainsburysmagazine.co.uk/recipes/mains/citrus-crusted-salmon-with-buttered-leeks-and-roasted-new-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-and-leek-chicken-topped-with-green-pea-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-ricotta-meatballs-in-tomato-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/gorgonzola-and-pear-salad-with-caramelised-walnuts-and-radicchio
https://www.sainsburysmagazine.co.uk/recipes/mains/florentine-fish-pie-with-smashed-potato-and-chives
https://www.sainsburysmagazine.co.uk/recipes/mains/lightly-curried-haddock-pie-with-crunchy-mustard-seed-and-cardamom-crumbs
https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-scallops-and-prawns-with-black-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/poussins-stuffed-with-spelt
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-savoy-cabbage-with-emmental-and-breadcrumbs
https://www.sainsburysmagazine.co.uk/recipes/mains/quinoa-salad-with-broccoli-preserved-lemon-and-avocado-oil
https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-leek-and-egg-salad-with-mustard-and-caper-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/persian-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/mozzarella-in-carrozza
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parmesan-with-rocket-and-new-potato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-mince-with-aromatic-rice-and-wilted-greens
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-mince
https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-kathi-rolls
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-bacon-and-kale-pies-with-celeriac-and-horseradish-rosti-topping
https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-basil-and-feta-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/cumin-chicken-with-giant-couscous-courgette-peas-and-mint
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-style-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-burgers-with-sweet-potato-wedges
https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-smoked-salmon-and-cream-cheese-topping
https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-avocado
https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-goats-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/duck-green-beans-and-walnuts
https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-bacon-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/piri-piri-rice-and-peas-with-garlic-prawns
https://www.sainsburysmagazine.co.uk/recipes/mains/pot-roasted-curried-chicken-with-garam-masala-mango-chutney-and-lemon
https://www.sainsburysmagazine.co.uk/recipes/mains/orange-sticky-chicken-with-three-green-veg
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-garlic-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-paillards-with-fennel-honey-and-lemon
https://www.sainsburysmagazine.co.uk/recipes/mains/herb-crusted-sea-bass-with-toasted-coconut-and-tomato-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/jerk-fish-with-seaweed-salted-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-salmon-with-cauliflower-couscous-and-garlic-spinach
https://www.sainsburysmagazine.co.uk/recipes/mains/reviving-chicken-broth
https://www.sainsburysmagazine.co.uk/recipes/mains/deep-filled-crab-saffron-and-parmesan-tartlets
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-marmalade-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/light-and-fragrant-fish-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/big-bowl-veggie-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-steak-with-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/haggis-stuffed-chicken-breasts-with-chilli-and-garlic-stir-fried-swede-and-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/sichuan-roasted-red-pepper-soup-with-ve-spice-edamame-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-soup-with-horseradish-lemon-and-parsley
https://www.sainsburysmagazine.co.uk/recipes/mains/kale-pea-and-spring-onion-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-orange-and-quinoa-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-prawn-rice-noodle-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/melting-brie-and-ratatouille-pasta-gratin
https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-oriental-chicken-parcels
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-breast-with-apricot-gammon-and-pistachio-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-venison-haunch-with-beetroot-shallots-and-horseradish
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-guinea-fowl-with-prune-and-bacon-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-with-currants-and-ginger
https://www.sainsburysmagazine.co.uk/recipes/mains/oven-baked-fish-and-chips
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-spiced-cranberry-pecan-stuffing-and-maple-glaze
https://www.sainsburysmagazine.co.uk/recipes/mains/sage-turkey-burgers-with-cranberry-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-pumpkin-and-chickpea-korma-with-crispy-shallots
https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-kale-and-goats-cheese-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-leek-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-thyme-and-garlic-roasted-chicken-with-chestnut-stuffing-filled-red-onions
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-red-pepper-sundried-tomato-and-ricotta-cannelloni
https://www.sainsburysmagazine.co.uk/recipes/mains/top-each-toasted-crumpet-with-2-crispy-bacon-rashers-and-a-poached-egg-serve-with-the-roasted-vine-tomatoes-on-the-side-and-season-with-freshly-ground
https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-crumpet
https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-garlic-raita
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-spiced-duck-legs-with-plums
https://www.sainsburysmagazine.co.uk/recipes/mains/caribbean-vegetable-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-chicken-with-broccoli-and-spelt
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-hoisin-turkey-meatball-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-rostis-with-fried-eggs-and-avocado-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-vegetable-minestrone
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-and-lentil-pies
https://www.sainsburysmagazine.co.uk/recipes/mains/duck-breasts-with-spiced-blackberry-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-roast-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-mozzarella-and-chive-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/best-ever-chicken-caesar-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-bacon-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-spiced-salmon-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/hot-smoked-trout-new-potato-and-pea-shoot-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/30-minute-seafood-paella
https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-aubergine-freekeh
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-pak-choi-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-parma-ham-agrodolce-peppers-and-pangrattato
https://www.sainsburysmagazine.co.uk/recipes/mains/singapore-style-prawn-laksa
https://www.sainsburysmagazine.co.uk/recipes/mains/fast-roast-beef-with-thyme-beets
https://www.sainsburysmagazine.co.uk/recipes/mains/the-ivy-shepherd-s-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-biryani
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-fennel-and-pepper-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/the-ultimate-veggie-burger-with-lemon-chipotle-mayo
https://www.sainsburysmagazine.co.uk/recipes/mains/ham-hock-broad-bean-and-new-potato-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-mackerel-fennel-and-orange-quinoa
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-onion-and-bacon-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/kale-mushroom-and-gorgonzola-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-stew-with-garlic-sourdough
https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-lamb-mint-and-tomato-couscous-wraps
https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-socca-with-cucumber-pepper-and-tahini-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-pepper-and-halloumi-kebabs-with-lemony-houmous
https://www.sainsburysmagazine.co.uk/recipes/mains/persian-style-spatchcock-barbecued-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/fig-gorgonzola-and-thyme-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-style-picnic-loaf
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-vegetable-pot-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/potato-mozzarella-and-spring-onion-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-chicken-drumsticks-with-salsa-and-griddled-apricots
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-lime-and-ginger-sea-bass
https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-with-avocado-roasted-vegetables-and-tahini-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-beef-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-halloumi-kebabs-with-freekeh-orange-and-walnut-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-with-coconut-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-quesadillas
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-pepper-arrabbiata-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-nicoise-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-fish-with-herby-potatoes-and-courgettes
https://www.sainsburysmagazine.co.uk/recipes/mains/paprika-chicken-with-griddled-corn-and-lemon-mayo
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-pancetta-salad-with-raspberries-and-mustard-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/muhammara-lamb-cutlets
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-panzanella-salad-with-deep-fried-mozzarella
https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-tagliatelle-with-kale-caramelised-onions-and-goats-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-avocado-couscous-with-harissa-yogurt
https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-with-aubergine-tomato-feta-and-basil-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/brioche-burger-buns
https://www.sainsburysmagazine.co.uk/recipes/mains/cod-baked-in-paper-with-basil-and-asparagus
https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-lemon-and-mozzarella-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-chicken-coleslaw-with-ginger-almond-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/seared-beef-salad-with-carrot-noodles-and-tahini-dressing
https://www.sainsburysmagazine.co.uk/recipes/mains/crab-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chickpea-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-red-sweet-potato-red-pepper-and-coconut-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-red-onion-flatbreads
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-nicoise-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/turkish
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-pork-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-ed-kebabs
https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pepper-and-onion-frittata-with-manchego
https://www.sainsburysmagazine.co.uk/recipes/mains/raw-thai-citrus-crunch-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/monkfish-chorizo-and-vegetable-paella-with-saffron-aioli
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-chops-with-crushed-new-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/spring-spanakopita
https://www.sainsburysmagazine.co.uk/recipes/mains/orange-roasted-chicken-spring-vegetables
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-tagine-with-preserved-lemon-and-olives
https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-gnocchi-with-watercress-mint-pesto
https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-cod
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-duck-breast-with-plum-star-anise-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-piri-piri-sardines-with-tomato-and-onion-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/the-ultimate-crab-cakes
https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-balls-with-courgetti-and-tomato-and-basil-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/fish-and-spinach-curry-with-cauliflower-rice-and-pistachios
https://www.sainsburysmagazine.co.uk/recipes/mains/basil-tomato-and-aubergine-soba-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/soy-and-ginger-tuna-with-soba-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-fattoush-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/pad-thai-noodles-with-mango-and-crab
https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-prawns-with-mango-noodles-and-herbs
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chicken-stir-fry-with-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/crab-tomato-and-parmesan-lattice-pies
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-shiitake-mushroom-pho-with-carrot-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-fishcakes-with-tomato-chermoula-sauce-and-nutty-couscous
https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-coconut-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/dukkah-sweet-potato-and-spinach-dhal
https://www.sainsburysmagazine.co.uk/recipes/mains/crumbed-katsu-style-chicken-with-cucumber-and-peanut-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-veggie-shepherd-pies
https://www.sainsburysmagazine.co.uk/recipes/mains/cider-chicken-and-bacon-pie-with-crushed-new-potato-topping
https://www.sainsburysmagazine.co.uk/recipes/mains/saag-paneer-with-cauliflower-rice
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-tomato-prawns-with-courgette-spaghetti
https://www.sainsburysmagazine.co.uk/recipes/mains/shredded-duck-and-watercress-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-spanish-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sumac-chicken-with-watercress-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/purple-sprouting-broccoli-blue-cheese-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-broccoli-and-anchovy-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-cheese-pancakes
https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-duck-stir-fry-with-lime
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-macaroni-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-mushroom-pot-pies
https://www.sainsburysmagazine.co.uk/recipes/mains/persian-chicken-with-prunes
https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-and-creme-fraiche-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-spinach-quinoa-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-moroccan-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-meatballs
https://www.sainsburysmagazine.co.uk/recipes/mains/winter-chicken-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/gorgonzola-and-broccoli-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-salmon
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-aubergine-moussaka
https://www.sainsburysmagazine.co.uk/recipes/mains/fast-chicken-butter-bean-and-tomato-stew
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-romesco-goats-cheese-pasta
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-cranberry-stuffing-and-pigs-in-blankets
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-sage-chestnut-and-apple-stuffing-balls-honey-mustard-chipolata-rolls-and-marsala-gravy
https://www.sainsburysmagazine.co.uk/recipes/mains/beef-en-croute
https://www.sainsburysmagazine.co.uk/recipes/mains/porcini-risotto-with-roast-garlic
https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-ribs-of-beef-with-roasted-red-onions-and-watercress-and-stilton-relish
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-breasts-with-date-and-pine-nut-quinoa-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/parmesan-crumbed-pork-with-roasted-tomatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-gnocchi-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/pepper-feta-and-smoked-paprika-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mushroom-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/brazil-nut-and-chestnut-roast
https://www.sainsburysmagazine.co.uk/recipes/mains/three-cheese-roulade
https://www.sainsburysmagazine.co.uk/recipes/mains/three-pepper-steaks
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-cacciatore
https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-with-artichokes
https://www.sainsburysmagazine.co.uk/recipes/mains/crab-tortillas
https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca
https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-burgers
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-schnitzelc
https://www.sainsburysmagazine.co.uk/recipes/mains/macaro-no-cheese
https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-onepot
https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-beef-massaman
https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-pecorino-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/two-cheese-spinach-and-potato-bake
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-aubergine-quinoa-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-chicken-with-warm-courgette-and-fennel-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/fresh-tomato-chickpea-and-spinach-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/the-ritz-game-pie
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-con-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-vegetable-tortilla-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/apple-pear-and-stilton-soup-with-chorizo-and-croutons
https://www.sainsburysmagazine.co.uk/recipes/mains/lambtagine
https://www.sainsburysmagazine.co.uk/recipes/mains/easy-roast-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/red-onion-bangers-with-kale-mash
https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-chorizo-frittata
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-paneer
https://www.sainsburysmagazine.co.uk/recipes/mains/sunshine-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/quick-bolognese
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-onepot
https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-prawn-and-broad-bean-pilaf
https://www.sainsburysmagazine.co.uk/recipes/mains/crumbed-kaffir-lime-chicken-with-lime-and-ginger-mayonnaise
https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-paella
https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-stuffed-aubergines
https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-tomato-bolognese
https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chicken-in-brioche-buns
https://www.sainsburysmagazine.co.uk/recipes/mains/bean-ratatouille
https://www.sainsburysmagazine.co.uk/recipes/mains/dairy-free-fish-pie-with-crispy-kale
https://www.sainsburysmagazine.co.uk/recipes/mains/warm-butter-bean-chicken-and-chorizo-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/herbed-chicken-drumsticks
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-jerk-chicken-skewers-with-brown-rice-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/anjum-anand-s-indian-style-chicken-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-pepper-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/oven-baked-prawn-pea-and-rocket-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-kebabs
https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-goats-cheese-and-hazelnut-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-and-sausage-baguette
https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-quesadillas
https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-courgette-and-pea-quiche
https://www.sainsburysmagazine.co.uk/recipes/mains/fast-roast-chicken-with-asparagus
https://www.sainsburysmagazine.co.uk/recipes/mains/classic-cheese-and-bacon-burger
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-thai-pork-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-sausages-and-goats-cheese-hotdogs
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-watercress-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/baked-chicken-with-olives-and-jalapenos
https://www.sainsburysmagazine.co.uk/recipes/mains/poached-trout-with-watercress-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/duck-breasts-with-a-piquant-lime-and-ginger-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/padthai
https://www.sainsburysmagazine.co.uk/recipes/mains/cannelloni-with-balsamic-glazed-tomatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/purple-sprouting-broccoli-and-parma-ham-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/croque-monsieur
https://www.sainsburysmagazine.co.uk/recipes/mains/pork-fillet-with-smoked-paprika
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-chicken-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chow-mein
https://www.sainsburysmagazine.co.uk/recipes/mains/individual-venison-wellingtons
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-soup
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pheasant-with-grapes-and-sweetheart-cabbage
https://www.sainsburysmagazine.co.uk/recipes/mains/classic-roast-turkey-with-chestnut-stuffing-balls
https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-wensleydale-and-toasted-hazelnut-wellington
https://www.sainsburysmagazine.co.uk/recipes/mains/onion-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/french-goat-s-cheese-ham-pepper-and-red-onion-pizza
https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-lamb-with-feta-peppers-and-dill
https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-and-garlic-tuna-stir-fry
https://www.sainsburysmagazine.co.uk/recipes/mains/tagliatelle-with-prawns-parma-ham-and-parsley
https://www.sainsburysmagazine.co.uk/recipes/mains/john-west-tuna-greek-style-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-rosemary-pearl-barley-risotto
https://www.sainsburysmagazine.co.uk/recipes/mains/john-west-salmon-and-cheddar-grills-2
https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-goats-cheese-tart
https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-turkey-meatballs-and-roasted-peppers
https://www.sainsburysmagazine.co.uk/recipes/mains/langoustine-and-chicken-paella
https://www.sainsburysmagazine.co.uk/recipes/mains/italian-stuffed-roast-pork
https://www.sainsburysmagazine.co.uk/recipes/mains/green-chicken-curry-with-noodles
https://www.sainsburysmagazine.co.uk/recipes/mains/tom-kerridge-slow-roast-lamb-with-boulangere-potatoes
https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-cod-burritos
https://www.sainsburysmagazine.co.uk/recipes/mains/hairy-bikers-massaman-duck-curry
https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-tomato-and-chilli-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-prawn-pockets-with-tomato-salsa
https://www.sainsburysmagazine.co.uk/recipes/mains/oxtail-stew-with-pumpkin-and-cinnamon
https://www.sainsburysmagazine.co.uk/recipes/mains/cumberland-glazed-roast-lamb-shoulder
https://www.sainsburysmagazine.co.uk/recipes/mains/one-pot-moroccan-chicken
https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-herb-and-pistachio-stuffing
https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-style-crispy-duck-with-plum-sauce
https://www.sainsburysmagazine.co.uk/recipes/mains/skinny-roasted-tomato-aubergine-and-chickpea-lasagne
https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-pork-buns
https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-squash-and-prawns-with-coconut-and-chilli
https://www.sainsburysmagazine.co.uk/recipes/mains/easy-teriyaki-chicken-traybake
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-festive-roast-pork
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-slow-cooker-pork-carnitas
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-with-bacon-and-mushrooms
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-spicy-garlic-prawn-and-rose-linguine
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chinese-style-pork-rolls-with-quick-pickled-slaw
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-orange-fennel-and-burrata-salad
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-herby-roast-chicken-with-crispy
https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-rose-harissa-cauli-flatbreads-with-butter-bean-humous
"""

# Split the string into a list of links, removing any extra whitespace
links = [link.strip() for link in links_string.strip().split('\n') if link.strip()]

# Adding quotes and formatting with commas, ensuring each link is on a new line
formatted_links = ',\n'.join([f'"{link}"' for link in links])

# Output the result
print(formatted_links)

"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dhal-with-crispy-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pork-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-salmon-curry",
"https://www

In [28]:
import requests
from bs4 import BeautifulSoup

def scrape_recipe(url):
    try:
        # Simulate a browser request with headers
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()  
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the recipe title
        title_tag = soup.find('h1', class_='d-none')
        title = title_tag.text.strip() if title_tag else "No title found"

        ingredients_list = []
        ingredients_section = soup.find_all('ul')
        for ul in ingredients_section:
            for li in ul.find_all('li', class_='ingredients'):
                if not li.find_parent('footer'):  # Exclude if the li is within a footer
                    ingredients_list.append(li.text.strip())

        instructions_list = []
        instructions_section = soup.find_all('ol')  
        for ol in instructions_section:
            for li in ol.find_all('li', class_='recipeSteps'):
                if not li.find_parent('footer'):  
                    instructions_list.append(li.text.strip())

        return {
            'title': title,
            'ingredients': ingredients_list,
            'instructions': instructions_list
        }
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

urls = ["https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-pale-ale-and-bean-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pineapple-chicken-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-roasted-pear-and-halloumi-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-lamb-meatball-and-giant-couscous-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gochujang-chicken-thighs-with-a-spring-onion-and-sesame-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-jacket-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peach-fennel-and-hazelnut-salad-with-crispy-sage",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-chicken-and-rice-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dhal-with-crispy-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pork-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-salmon-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quick-lamb-biryani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-toad-in-the-hole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/boiled-gammon-with-pease-pudding",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-frying-pan-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-green-prawn-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-meatball-moussaka",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-tagine-and-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-salty-lentil-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seared-horseradish-steak-with-red-wine-and-rosemary-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-cheesy-puy-lentil-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-stroganoff-jacket-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/30-minute-spicy-prawn-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lazy-caramelised-onion-sausage-hotpot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/flatbreads-with-spiced-salmon-and-zesty-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-smoky-corn-and-peppers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-chickpea-and-kale-frittata-with-tomato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/garlic-butter-chicken-with-cheesy-cauli-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-buttermilk-marinated-spatchcock-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/super-quick-cook-steak-skewers-with-chimichurri",
"https://www.sainsburysmagazine.co.uk/recipes/mains/all-american-ribs-with-sticky-bourbon-glaze",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beetroot-greens-and-brie-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-with-charred-sweetcorn-salsa-and-fresh-mint-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/yours-and-mine-seafood-boils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/everything-seasoning-chicken-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-lemonade-chicken-with-rainbow-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/triple-citrus-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/next-level-hot-dogs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hot-honey-barbecue-platter",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sicilian-style-mackerel-with-white-beans-and-caponata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dolcelatte-prosciutto-and-pickled-peach-pizza-with-hot-honey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-courgette-and-butter-bean-braise",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broad-bean-dill-and-hazelnut-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-courgette-involtini",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-summer-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-chicken-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-tuna-kebab",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-loin-tikka-paratha-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-mango-and-cashew-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-gentlemans-relish-galette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-beef-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-carrot-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-chicken-souvlaki",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-cheeseburger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/glazed-pork-with-mango-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/houmous-flatbread-pizzas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-saltimbocca-burger-with-crispy-prosciutto-and-parmesan-mayo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smash-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-chicken-milanese-with-shaved-asparagus-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tortelloni-with-minty-pea-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawns-with-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-sea-bass-larb",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-and-chip-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gazpacho",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spice-crusted-tuna-with-polyphenol-rich-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-rice-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-and-roasted-cauliflower-with-herby-bulgur-wheat-and-tahini",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-poke-recipe-tofu-avocado",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-and-goats-cheese-open-sandwich",
"https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-style-egg-mayo-sandos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-herby-yogurt-and-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-in-a-basket",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hasselback-halloumi-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-pineapple-skewers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pigs-in-blankets-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-steak-tacos-with-cucumber-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-gyoza-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vibrant-watercress-and-pea-soup-with-bacon-crumbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-bream-with-pickled-radish-and-pea-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-saffron-chicken-white-bean-and-chorizo-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-warm-salmon-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-lentil-and-roast-tomato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-and-bean-mash-with-cheesy-fennel",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dill-and-feta-rice-with-crispy-fried-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-puff-plait",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-tenderloin-with-apples-and-walnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-lemon-olive-butter-and-new-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-stuffed-aubergines",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rack-of-lamb-with-anchovy-salsa-and-spring-vegetables",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-tomato-chicken-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-moussaka",
"https://www.sainsburysmagazine.co.uk/recipes/mains/impossible-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sweet-chilli-halloumi-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sesame-beef-and-spring-onion-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kale-and-chickpea-linguine-with-a-nutty-crunch",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-trout-and-pea-cacio-e-pepe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-greens-and-white-bean-rigatoni",
"https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-chicken-with-pepper-caprese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-beef-stifado",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-brisket-and-black-bean-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/garlicky-asparagus-carbonara",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-minced-beef-and-mushroom-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-tofu-with-zingy-lime-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/egyptian-filo-pie-with-chopped-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-sweetcorn-brunch-fritters",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-oven-baked-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jersey-royal-and-crab-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pub-style-steak-sarnie-with-beer-onions",
"https://www.sainsburysmagazine.co.uk/recipes/mains/silk-handkerchiefs-with-smoked-beurre-blanc",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-satay-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-greens-shakshuka",
"https://www.sainsburysmagazine.co.uk/recipes/mains/triple-mustard-roast-beef-and-yorkshires",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-with-spring-veg-and-dill-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-al-limone",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chilli-con-carne-from-knorr-stock-pots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-knorr-spaghetti-bolognese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-lamb-with-mustardy-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butter-basted-sage-pork-chops-with-beetroot-puree-and-garlic-chard",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-chicken-livers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/parma-chicken-schnitz-with-cheesy-kale-and-celeriac-fries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/noodle-omelette-with-sweet-chilli-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-easy-pea-and-seafood",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-and-potatoes-with-preserved-lemon-aioli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hasselback-roast-squash-with-whipped-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-roast-lamb-with-provencal-panzanella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-flatbread-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-kleftiko",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-pillows-in-silky-green-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-lamb-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chilli-with-squash-and-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/welsh-rarebit-galette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-bacon-and-white-bean-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/honey-mustard-chicken-and-apples",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-mushrooms-with-chimichurri-parsnip-and-butter-bean-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-citrus-pork",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tartiflette-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-sweet-potato-succotash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-gnocchi-and-sausage-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-sauerkraut-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-lamb-meatballs-in-a-creamy-porcini-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-short-rubs-with-sweet-potato-mash-and-pickled-walnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/surf-and-turf-skillet-with-garlic-dippers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roast-cacio-e-pepe-cauliflower",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-tofu-goulash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chunky-chorizo-and-butter-bean-soup-1",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-coconut-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-spinach-and-mushroom-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-mushroom-grains-with-date-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-lamb-chops-with-sweet-potato-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-and-sausages-with-pickled-peppers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/colcannon-smoked-fish-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-aubergine-steak-with-creamed-corn-and-a-herby-green-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herby-meatballs-with-broccoli-pesto-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pie-and-liquor",
"https://www.sainsburysmagazine.co.uk/recipes/mains/three-mushroom-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-with-sausage-and-chicory",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-and-apple-soup-with-horseradish-dill-dumplings",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-socca-pancake-with-smoked-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chunky-chorizo-and-butter-bean-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/stove-top-mac-and-cheese-with-pancetta-crumbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beigels-or-bagels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-bhuna",
"https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-risotto-with-celery-and-tarragon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tahini-chicken-greens-and-beans",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-cod-patatas-bravas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/milk-braised-pork-steaks-with-cabbage",
"https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-sausage-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-brunch-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-salmon-and-vegetable-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/emerald-dhal-with-golden-tofu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tofu-and-greens-noodle-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gingered-mushroom-super-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-cajun-cauli-tacos-and-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-cauli-and-sweet-potato-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seared-salmon-with-tarragon-red-wine-sauce-and-braised-chicory",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-wine-vegan-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/double-dip-buttermilk-fried-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-and-paneer-torkari",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-pot-pie-with-puff-pastry-croutons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-chicken-with-citrus-and-lavender",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-mushrooms-madeira-sauce-and-celeriac",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-and-sprout-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-cheesy-shallot-tarte-tatin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/double-chestnut-mushroom-wellington",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-and-confit-duck-with-bubble-and-squeak-cakes-cranberry-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bigos-hunters-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-enchilada-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/root-veggie-nut-roast-with-chestnut-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-and-ginger-jam-glazed-ham-with-nutmeg-and-sweet-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-veggie-tarte-tatin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/winter-chicken-caesar",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-smoked-salmon-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-parsnip-and-cashew-strudels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-ale-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brussels-sprout-chestnut-and-bacon-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-pork-and-potato-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-pasta-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zesty-salmon-and-roasted-broccoli-bulgur",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hariyali-guinea-fowl-with-roasted-onions",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cider-poached-cod-with-spiced-mussels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/campfire-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cider-pulled-pork-and-campfire-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pumpkin-soup-with-nutty-bacon-crumble",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cobweb-pancakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-and-pear-soup-with-blue-cheese-toasts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-cabbage-rolls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-mash-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/root-vegetable-and-coconut-fish-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-mozzarella-sausage-and-sweet-pepper-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-pithivier",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-harissa-lasagne-with-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-braised-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tagliatelle-with-walnut-and-kale-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/maple-mustard-ham-and-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/traditional-welsh-rarebit",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-rice-and-mussels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-tarragon-roast-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-potatoes-and-plums-with-marsala-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/negroni-chicken-skewers-with-fennel-and-orange-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-chicken-and-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-cod-parcels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summery-spaghetti-and-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/easy-honey-garlic-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-and-chickpeas-en-papillote",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asian-style-prawn-and-grapefruit-salad-with-crispy-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ribs-red-beans-and-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crackling-good-pork-and-apple-traybake-with-crunchy-pickle-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-steak-and-blue-cheese-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-style-lamb-with-warm-houmous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-pork-belly-with-pickled-plums",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-chicken-summer-tomato-and-basil-panzanella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pesto-crusted-lamb-with-green-quinoa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/golden-coconut-turkey-and-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-porcini-burger-with-truffle-mayo-and-crispy-onion-rings",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-tenderstem-broccoli-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-chickpea-and-chorizo-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/parmesan-chicken-butties-with-red-cabbage-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/taco-pizza-flatbreads-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/marinated-halloumi-with-honey-and-thyme",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-shoulder",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-feta-pizza-with-hot-honey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-salmon-in-chard-with-caramelised-lemons-garlic-and-almond-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-squash-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-prawn-burger-with-sriracha-butter",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lime-and-coconut-pork-belly-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/feta-stuffed-peppers-with-butter-bean-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bedfordshire-clangers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/the-high-steaks-sharing-board",
"https://www.sainsburysmagazine.co.uk/recipes/mains/afternoon-sea-sharing-board",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peperonata-bucatini-with-sardines",
"https://www.sainsburysmagazine.co.uk/recipes/mains/green-couscous-with-torn-tahini-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/wensleydale-and-apple-tart-with-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-salmon-kedgeree",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-lamb-burger-with-pickled-red-cabbage-chilli-and-garlic-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-prawn-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-bianca-with-ham-and-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-and-crispy-bean-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-tofu-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/runner-bean-linguine-with-ricotta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/arabic-style-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-and-butter-bean-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fennel-olive-and-tomato-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-sea-bass-with-pesto-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/minty-lamb-with-rainbow-veg-and-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-shawarma-picnic-loaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mango-chicken-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ham-and-roasted-pepper-tortilla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-basa-with-smoky-tomato-beans",
"https://www.sainsburysmagazine.co.uk/recipes/mains/green-goddess-salmon-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/honey-mustard-bacon-steaks-with-sweetcorn-crush",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-kipper-and-potato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/feelgood-fish-and-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/frying-pan-hawaiian-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-chicken-caprese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/flavour-bomb-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sherry-braised-chicken-with-chorizo-and-chickpeas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-asparagus-and-bean-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-tarte-tatin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lancashire-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/massaman-beef-and-aubergine-skewers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chargrilled-chicken-with-zingy-coriander-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-paprikash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-beans-on-toast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-roast-lamb",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-beef-chow-mein",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-cutlets-with-pearl-barley",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-nova-scotia-lobster-roll",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bobotie-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-satay-sitr-fry-with-a-sweet-sour-radish-pickle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-chicken-roast-dinner",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-roasted-garlic-and-burnt-spring-onion-butter",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-spatchcock-poussins-with-barbecued-peach-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-portobello-veggie-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-ham-and-leek-filo-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-broccoli-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-en-papillote-with-caponata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-squash-and-salsa-di-noci",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-broccoli-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pea-ham-and-mint-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-bacon-and-corn-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-cauliflower-biryani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-with-figs-and-shallots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tex-mex-5-a-day-veggie-feast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/corned-beef-hash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/popeye-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-carrot-and-lentil-soup-with-bhajia-nuggets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-lamb-stir-fry-with-half-the-carbs-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-and-cheese-meat-pies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lancashire-cheese-leek-and-onion-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-smoked-fish-and-spinach-tagliatelle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/meaty-mushroom-pho",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-meatball-hotpot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rice-stuffed-onions-with-feta-and-dill",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-patties-with-nutty-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-fillet-with-garlicky-mushroom-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-gnocchi-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veg-and-tofu-spicy-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-sausage-and-cabbage-saute",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-and-ginger-beef-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/super-greens-thai-green-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-tomatoes-with-broccoli-smash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/marmalade-sausages-with-parsnip-colcannon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lama-keema",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-swedish-style-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/clementine-cumberland-glazed-ham",
"https://www.sainsburysmagazine.co.uk/recipes/mains/patatas-bravas-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-fish-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-satay-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-beef-bourguignon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cullen-skink",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-stir-fry-with-quorn-and-plums",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broken-lasagne-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/low-waste-butternut-squash-and-sage-rigatoni",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooker-golden-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-mushroom-and-leek-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-coconut-and-lentil-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-teriyaki-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-spaghetti-with-crispy-capers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/curried-turkey-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-leftover-turkey-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leftovers-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/poussins-with-potato-celeriac-and-porcini-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-turkey-minestrone",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tinned-fish-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-sprout-colcannon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-cod-in-prosciutto-di-parma-with-green-pea-puree",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sage-and-apricot-stuffed-pork-tenderloin-with-roasted-pear-parsnip-and-potato",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cider-chicken-casserole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cinnamon-and-saffron-lamb-shank-with-roasted-roots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-cod-with-sake-shiitake-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ipa-casserole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-chestnut-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-thighs-with-mustardy-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-potato-cakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-katsu-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chestnut-and-mushroom-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/truffled-mushroom-soup-with-cheesy-croutes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-meatball-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-gammon-stew-with-buttermilk-dumplings",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-and-garlic-butter-turkey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-and-pesto-traybake-from-sacla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/savoy-cabbage-gratin-with-herby-cheddar-crumbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chickpea-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-steak-with-cheesy-creamed-corn",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-halloumi-and-spinach-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/liver-bacon-and-onions-with-white-bean-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-restaurant-style-butter-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-paneer-and-pea-biryani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-laksa-with-spicy-sambal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/philly-cheeseteak-stuffed-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-chicken-tortellini-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-kale-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-green-peas-and-coconut-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilean-cottage-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-cheese-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-melt-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/murgh-makhani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-barley-one-pot-with-zesty-gremolata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chilli-prawn-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-arrabiata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-pesto-pasta-alla-norma-with-sacla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweetcorn-and-bacon-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jaffna-lamb-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-smoky-bacon-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-basa-with-sweetcorn-pilau",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-harissa-and-marmalade-baked-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-beetroot-pesto-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-dill-cucumber-yogurt-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-chorizo-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-creamed-corn",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-halloumi-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-and-stilton-crumble",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-meatball-red-pepper-and-feta-rigatoni-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-clonakilty-blackpudding-stuffed-peppers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-crispy-filo-feta-and-clonakilty-blackpudding-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-burgers-with-apple-and-fennel",
"https://www.sainsburysmagazine.co.uk/recipes/mains/oven-roasted-jerk-pork",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-lamb-filo-pie-with-chermoula-drizzle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-bacon-and-tomato-toastie-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ham-hock-and-pea-fritters",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-corn-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caprese-grain-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-cheeseboard-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smashed-broad-bean-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-egg-and-brown-rice-bowls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-butter-salmon-and-red-pepper-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-cashew-massaman-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-saag-makhani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-chicken-korma",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-orange-beet-salad-with-crispy-chickpeas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-broccolini-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanakopita-stuffed-peppers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-hot-dogs-with-corn-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/texan-style-barbecue-beef-brisket-and-succotash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-sage-and-blackberry-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ribeye-with-burnt-spring-onion-aioli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/venison-burgers-with-mustard-fries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-tomato-zaatar-houmous-and-harissa-tarts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-lamb-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-nicoise-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-feta-greek-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/blt-panzanella-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tortang-talong",
"https://www.sainsburysmagazine.co.uk/recipes/mains/red-lentil-koftes-with-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-kedgeree",
"https://www.sainsburysmagazine.co.uk/recipes/mains/minted-pea-and-lemon-carbonara",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-roasted-radishes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-steak-fajita-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/poutine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-glazed-tuna-with-ginger-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sea-bass-with-buttered-brown-shrimp-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/trout-with-a-gremolata-crumb",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-spring-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-traybake-lemon-butter-asparagus",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-meatball-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quick-roast-chicken-with-tarragon-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/charred-spring-onion-miso-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-carrot-and-halloumi-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-porcini-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-shoulder-of-lamb-with-chickpeas-onions-garlic-and-oregano",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-and-pistachio-pesto-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/minty-lamb-chops-with-watercress-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-beef-short-ribs-with-haricot-beans-and-spinach",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-pork-shoulder",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sour-squash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-harvest-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crab-and-spring-onion-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-mushroom-cheesecake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-greens-and-mung-beans-with-turmeric-oil",
"https://www.sainsburysmagazine.co.uk/recipes/mains/honey-miso-chicken-squash-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-shrimp-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scrambled-eggs-with-asparagus-salami-and-oregano",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-and-caper-chicken-thighs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/korean-style-beef-noodle-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herby-spring-chicken-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/savoy-cabbage-with-celeriac-puree-and-brown-butter-hazelnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-spiced-meatloaf-roast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-lentil-and-quinoa-pilaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-cottage-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mac-and-3-cheese-stuffed-squash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crab-stuffed-plaice-with-cucumber-cream-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-meatball-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pork-bao-buns-and-homemade-pickles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-butternut-with-chickpeas-and-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-beef-and-noodle-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-trout-with-parsnip-rosti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jackfruit-faitas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/saffron-salmon-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-balti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-balmoral-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-sweet-potato-tomato-and-coconut-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reader-recipe-creamy-black-bean-jackets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-beetroot-dhal-with-homemade-coriander-chutney",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-vegan-mushroom-and-leek-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chard-and-chestnut-gnocchi-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-sweet-potato-and-bean-chilli-from-knorr-stock-pots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-katsu-curry-from-the-vegetarian-butcher",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-pasta-primavera-from-knorr-stock-pots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-potato-and-pea-curry-from-knorr-stock-pots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-cajun-spiced-poke-bowl-with-winter-coleslaw-from-the-vegetarian-butcher",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-ultimate-vegan-tacos-from-the-vegetarian-butcher",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-5-spice-tofu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-caraway-beef-and-beetroot-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pavo-relleno-mexican-stuffed-roast-turkey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-pomegranate-and-clementine-slow-roast-lamb-leg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-seven-spice-pork-belly-with-cranberry-and-orange-ketchup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-orange-glazed-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-parsnip-and-apple-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-roast-pork-loin-with-cranberry-glazed-pigs-in-blankets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/easy-carve-roast-turkey-with-chestnut-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-stuffed-spuds",
"https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-turkey-on-the-barbecue",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-wensleydale-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sugar-plum-ham",
"https://www.sainsburysmagazine.co.uk/recipes/mains/firecracker-chicken-with-cavolo-nero-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chestnut-pesto-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-cauliflower-pasta-with-creamy-tahini-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-chicken-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-roots-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-with-mushroom-lentil-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-beetroot-and-squash-wellington",
"https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roast-celeriac-with-sage-and-walnut-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-tray-roast-with-balsamic-glaze-from-taste-the-difference",
"https://www.sainsburysmagazine.co.uk/recipes/mains/keralan-cauliflower-and-lentil-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-pea-and-watercress-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-barley-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-fennel-and-olive-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-veggie-lasagne-rolls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-crumbed-cod",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-courgette-pesto-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/duck-marsala-and-chestnut-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-soy-and-sesame-tofu-with-broccoli-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-pumpkin-and-sage-souffle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-cashews-and-lentils-with-date-chutney",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-katsu-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leek-cacio-e-pepe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-apple-and-stilton-strudel",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-toad-in-the-hole-with-sherry-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-root-and-barley-broth",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tarka-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-aubergine-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-duck-legs-with-blackberries-and-celeriac-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-madras",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-veggie-curry-with-radish-raita",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-turnip-and-feta-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/one-pot-pasta-with-bacon-and-peas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/squash-and-chickpea-one-pot-with-herby-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/biang-biang-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-stuffed-herb-and-garlic-veggiea",
"https://www.sainsburysmagazine.co.uk/recipes/mains/eggs-florentine-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-and-gnocchi-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-chops-with-smashed-green-bean-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-harissa-seafood-stew-from-belazu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gingered-beef-ribeye-with-noodle-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-garlic-roast-chicken-with-crispy-ham-and-veggies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-plum-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spatchcock-bbq-chicken-with-burnt-onion-aioli-and-caramelised-lemons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-cherry-tomato-and-olive-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-anchovy-butter-and-mustard-roast-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken-pilaf-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-tofu-and-cashew-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-salad-with-pork-scratching-croutons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potatoes-with-chorizo-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-finger-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veg-patch-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-scallop-and-pea-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-mussel-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-paneer-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gravlax-baked-salmon-with-sweet-dill-mustard-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pot-roast-chicken-with-cider-and-tarragon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brunch-burger-with-bacon-ketchup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-broad-bean-and-seaweed-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tear-and-share-spiced-veg-roll",
"https://www.sainsburysmagazine.co.uk/recipes/mains/minty-pea-and-bacon-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/carrot-dogs-with-dill-pickles-and-remoulade",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-sausage-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-and-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-griddled-vegetables-grain-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-salmon-fishcakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-feta-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-lamb-kebabs-with-summer-pea-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-stew-cottage-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-ham-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-feta-tart-with-poppy-seed-pastry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-veggie-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-mole-burritos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/potato-and-ricotta-gnocchi-with-purple-sprouting-broccoli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-orzotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-kofta-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-yassa-with-hasselback-sweet-potato",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-salmon-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butter-bean-pepper-and-chorizo-saute-with-seared-cod-and-asparagus",
"https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-pea-pesto-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-okra-soup-and-semo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fatty-meatloaf-with-bacon-cheese-and-tomato-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/charred-chorizo-peach-pitta-panzanella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/groundnut-beef-stew-with-ground-rice-balls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-lamb-and-yogurt-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-pork-and-fennel-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-with-jewelled-rice-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-miso-steak-with-sesame-sweet-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-with-lentil-tabbouleh",
"https://www.sainsburysmagazine.co.uk/recipes/mains/white-wine-coq-au-vin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pork-meatloaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potatoes-with-almond-tahini",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-crumbed-cod-with-minty-crushed-potatoes-and-peas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-pork-rice-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hawaiian-egg-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-with-a-spring-broth-of-asparagus-jersey-royals-and-mussels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/stuffed-roast-pork-fillet-with-cannellini-bean-and-tomato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-skewers-with-satay-sauce-and-smashed-cucumber",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-pork-skewers-with-cucumber",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-lamb-burgers-with-pickled-vegetables",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-chamoy-chicken-with-grains",
"https://www.sainsburysmagazine.co.uk/recipes/mains/swedish-sandwich-cake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-salmon-with-warm-saag-aloo-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-casserole-with-walnut-gremolata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-prune-and-sherry-casserole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herb-crusted-salmon-with-garlic-fries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zesty-salmon-parcels-with-dill-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jalapeno-crusted-salmon-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-spaghetti-with-squash-salami-and-pine-nuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-style-chicken-with-braised-spring-veg-and-horseradish-cream",
"https://www.sainsburysmagazine.co.uk/recipes/mains/provencal-pine-nut-crusted-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/springtime-frittata-with-roasted-radishes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/maple-mustard-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/stuffed-picnic-loaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-vegetable-pot-pies-with-oxo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-lamb-caper-and-olive-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/masala-mackerel-with-coconut-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sour-pork-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-lamb-with-a-watercress-and-caper-crust",
"https://www.sainsburysmagazine.co.uk/recipes/mains/curried-halloumi-fritters-raita-and-fried-egg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-lamb-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chipotle-beef-burrito-rolls-with-broccoli-stalk-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/colmans-mint-glazed-roast-leg-of-lamb",
"https://www.sainsburysmagazine.co.uk/recipes/mains/colmans-herbed-mustard-crust-roast-beef",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-veggie-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/meatball-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-beef-ragout",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leek-ham-and-apple-barley-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seared-steak-taco-board",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sage-onion-toad-in-the-hole-with-cider-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-steak-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-chicken-and-mushroom-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/batch-cook-honey-balsamic-beef-brisket",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jackfruit-and-mushroom-stew-with-herb-dumplings",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roasted-lamb-with-spring-onion-mash-and-pistou",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-wellington-with-porcini-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-citrus-roast-duck-with-crispy-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/masala-cauliflower-and-aduki-bean-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-spiced-veggie-sausage-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-baked-beans",
"https://www.sainsburysmagazine.co.uk/recipes/mains/filo-fish-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-and-haggis-escalopes-with-whisky-and-chicken-butter-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roots-barley-and-lamb-cawl-with-cheddar-puffs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-mackerel-and-leek-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-island-lamb",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moussaka-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-e-fagioli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/horseradish-coated-cod-fillets-with-beetroot-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-masala-bean-pot-pies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-cobbler-with-cheese-scones",
"https://www.sainsburysmagazine.co.uk/recipes/mains/venison-steak-diane-with-truffle-fries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-roast-pork-and-winter-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-wine-pot-roast-duck",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-red-wine-stew-with-porcini-and-chestnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/triple-cheese-stuffed-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-sage-and-pine-nut-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-rib-of-beef-with-chimichurri",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-sloe-gin-and-plum-ham",
"https://www.sainsburysmagazine.co.uk/recipes/mains/christmas-spaghetti-bolognese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/port-clementine-and-5-spice-ham",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-maple-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-four-cheese-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vineyard-chicken-with-crispy-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-pork-steaks",
"https://www.sainsburysmagazine.co.uk/recipes/mains/porchetta-style-turkey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/5-a-day-turmeric-traybake-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/curried-lamb-shanks-with-mauritian-mango-kutcha",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jewelled-stuffed-butternut-squash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-tacos-with-feta-and-watermelon-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pepper-and-chorizo-pie-with-a-cornbread-crust",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-pork-and-crispy-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/wine-braised-sausages-with-olive-oil-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brown-butter-mushroom-and-sage-pappardelle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheeseboard-nachos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-soup-toastie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-sausage-gnocchi-hotpot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-mozzarella-turkey-patties-and-griddled-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/plum-and-stilton-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/provencale-daube-style-pot-roast-beef",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-spanish-style-shepherds-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-and-egg-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-pork-with-rosemary-potatoes-and-sprouting-broccoli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beef-fillet-with-celeriac-puree",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-cabbage-and-leek-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca-baked-cod",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fajita-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/green-curry-roast-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-eggs-with-garlic-toasts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salt-and-pepper-salmon-with-spring-onion-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-tapenade-tart-with-broccoli-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/curried-fish-pie-with-spiced-potato-topping",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-sage-and-ricotta-open-crust-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-topped-chicken-with-broccoli-smash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-lamb-patties-and-sweet-potato-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-style-hake-with-almond-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-chicory-apple-and-walnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baja-fish-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-caramel-pork-noodle-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-aubergine-pesto-feta-gyros",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-gammon-and-pineapple-with-colcannon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-nectarine-and-prosciutto-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-saffron-and-paprika-spiced-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-prawn-and-chorizo-paella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-with-mussels-bay-and-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-with-leeks-lime-and-curry-butter",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sardines-with-cucumber-salsa-verde-salad-and-garlic-mayonnaise",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-rags-with-speedy-sausage-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-cauliflower-steaks-with-pineapple-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-chickpea-and-carrot-veggie-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-inspired-sausage-plait",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coriander-feta-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-watercress-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-with-zesty-peas-and-mint",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-prosciutto-cannelloni",
"https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-sweet-potato-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-chicken-with-green-grains",
"https://www.sainsburysmagazine.co.uk/recipes/mains/woolton-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-and-rainbow-veg-grain-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-lemon-chicken-casserole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/campers-caponata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-salad-with-halloumi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/store-cupboard-jambalaya",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-mac-and-cheese-with-caramelised-tomatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-sausages-with-beer-braised-onions",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-risotto-with-basil-oil",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-red-curry-noodle-soup-with-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/charred-spring-onion-and-ricotta-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/open-lasagne-with-roasted-squash-crispy-prosciutto-and-amaretti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-romesco-sauce-and-butterbean-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-split-pea-dhal-with-roasted-sweet-potato",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-leg-of-lamb-with-garlic-and-rosemary",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-caponata-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-and-lime-fish-and-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-minestrone",
"https://www.sainsburysmagazine.co.uk/recipes/mains/minty-salsa-verde-lamb-with-lemony-potatoes-and-leeks",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-chicken-with-roast-veg-and-olives",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-chicken-fennel-and-garlic-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-sardine-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-bourguignon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fattoush-salad-with-herbed-labneh-balls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kale-and-ricotta-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-smoked-haddock-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-lamb-tabbouleh-and-beetroot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-turkey-larb",
"https://www.sainsburysmagazine.co.uk/recipes/mains/curried-mushroom-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-beef-stew-with-gruyere-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lobster-mac-n-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-miso-mushroom-casarecce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-aubergine-rice-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-steak-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-spring-veg-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-hake-lime-and-coconut-pilaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-tarragon-crushed-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-avocado-corn-and-feta-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-wrapped-cod-and-mediterranean-vegetables",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-tomato-chicken-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hake-celeriac-puree-and-balsamic-chickpeas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-with-caper-lemon-sauce-and-green-bean-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-burgers-with-tahini-coleslaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/golden-glow-chicken-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/shepherds-pie-with-root-veg-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-chicken-ramen",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-italian-savoury-bread-pudding",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-stuffed-pork-with-roasted-roots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-winter-veg-traybake-with-speedy-rhubarb-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-tomato-fusilli-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scouse",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-soy-beef-and-green-bean-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/green-goddess-glow-bowl-with-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-veggie-rice-with-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-turkey-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cider-lamb-chops-with-celeriac-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pork-on-a-bed-of-cider-soaked-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-chinese-beef-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/haggis-neeps-and-tatties-tart-with-whisky-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/restorative-chicken-and-leek-noodle-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tofu-katsu-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-peanut-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-lasagne-with-butternut-squash-and-hazelnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crayfish-and-champagne-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/curried-smoked-haddock-chowder",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-and-blue-cheese-ciabatta-with-beetroot-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-stuffing-roulades-with-sprout-and-chestnut-potato-cakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/citrus-salmon-with-green-quinoa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-madras-with-mango-and-cucumber-raita",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-with-stir-fried-sprouts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-lemon-butter-roasted-turkey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cashew-and-pistachio-stuffing-nut-roast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/porcini-bacon-and-sage-roast-rib-of-beef-with-port-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peppered-beef-fillet-with-truffle-mayo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-chicken-and-leek-sourdough-tartiflette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/witches-brew-smoky-beef-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pork-with-balsamic-mushrooms-crispy-potatoes-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-chicken-and-courgettes-with-pistachio-yogurt-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/luxe-smoked-salmon-gnocchi-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-florentine-mushroom-sauce-and-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/easy-cheesy-pasta-with-greens",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-loin-with-beetroot-and-lentil-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coronation-chicken-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/duck-and-red-wine-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-beef-stew-and-cheese-biscuits",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kipper-and-salmon-fish-cakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-ham-and-greens-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/marmite-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ham-and-leek-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-hotdogs-with-crispy-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-jalapeno-quesadillas-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-spinach-tortelloni-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butter-bean-makhani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chinese-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tray-baked-squash-beetroot-and-giant-couscous-with-goats-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ploughmans-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-harvest-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-leek-pie-with-crispy-potato",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fusilli-with-spinach-mushrooms-and-goats-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-taco-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-falafel-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/alpine-chicken-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beer-marmite-braised-beef-with-horseradish-dumplings",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-salmon-nicoise-with-soft-boiled-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jerk-pork-belly-with-mango-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-tofu-and-broccoli-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-chicken-and-quinoa-with-corn",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-and-chickpea-kofte",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-steaks-with-squash-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ale-stew-with-cheese-and-mustard-croutes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/truffled-mushroom-mac-n-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pea-ham-and-goats-cheese-tortilla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-veg-gnocchi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-pea-and-cauliflower-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-schnitzel-salad-with-cheats-garlic-aioli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-coconut-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-butter-bean-summer-casserole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-ratatouille-with-lentils-and-lime-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-fritters-with-pasta-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fire-grilled-steak-sandwich-with-chimichurri",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-chickpea-and-chorizo-stew-with-fried-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-burgers-with-cucumber-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-roast-mackerel-with-spiced-chickpeas-and-raita",
"https://www.sainsburysmagazine.co.uk/recipes/mains/buttery-courgette-and-rocket-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-ale-pie-with-pickled-walnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-mackerel-with-orange-and-beetroot-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-chicken-with-orange-cumin-and-apricot-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/honey-and-lemon-pork-with-grains",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parmo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-aubergine-with-tabbouleh-and-walnut-turmeric-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-traybake-with-spinach-and-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-chicken-and-peach-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/maple-smoked-salmon-and-potato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cashew-carrot-pomegranate-and-halloumi-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sloppy-joe-bowls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-cheese-dogs-with-mustard-onions",
"https://www.sainsburysmagazine.co.uk/recipes/mains/satay-steak-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mustard-honey-salmon-with-veggie-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/all-day-breakfast-gnocchi-fry-up",
"https://www.sainsburysmagazine.co.uk/recipes/mains/springtime-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pea-and-ricotta-gnocchi-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-and-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-pork-with-sweet-potato-wedges-and-sweetcorn-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/storecupboard-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-cashew-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-burger-and-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/oregano-roast-beef-with-crispy-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-crusted-lamb-rack",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-pea-fishcakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/california-salmon-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tahini-and-olive-spaghetti-with-tomato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-and-goats-cheese-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cornish-pasties",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-apricot-chickpea-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cantonese-aubergine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-fajitas-with-jalapeno-salsa-and-avocado",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-bean-meatballs-in-spiced-tomato-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-and-sticky-pork-char-siu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-pasta-with-ham-crisps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-chicken-curry-with-thai-style-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-jackets-with-beetroot-ratatouille",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gin-and-tonic-salmon-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-salmon-fishcakes-with-loaded-potato-skins",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butterflied-lamb-with-kentucky-mop-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-black-bean-and-sweet-potato-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/royal-almond-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-asparagus-tagliatelle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-paneer-skewers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-halloumi-burger-with-sweetcorn-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brinner-burrito-with-marmite-hollandaise",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chickpea-and-sweet-potato-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-mustard-pork-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuscan-baked-fish-with-orzo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/individual-minted-lamb-wellingtons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/maple-soy-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pomegranate-beef-with-spiced-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-chicken-and-grains",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-roast-lamb-with-leeks-and-caerphilly",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-leg-of-lamb-with-fennel",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-roast-lamb-recipe-gizzi-erskine-reicpe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-spiced-whole-celeriac-with-raita",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-tortellini-minestrone",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-goats-cheese-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-fajita-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-asparagus-crumble",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweetcorn-courgette-and-feta-oven-baked-fritters",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sizzling-garlic-prawn-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-cauliflower-and-quinoa-goodness-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-salmon-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-jalapeno-buddha-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/choucroute-garnie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-dill-cullen-skink",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pork-meatballs-with-courgetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-aubergine-squash-feta-grain-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-chicken-wraps-with-quick-pickled-cucumber",
"https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/friday-night-fish-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-baked-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-lamb-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-omelette-with-tomato-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-kiev-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-roast-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-stir-fry-with-chilli-and-basil-2",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pineapple-pork-mince-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-chow-mein",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quinoa-grape-and-feta-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-sage-n-squash-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/soy-butter-steak-with-bearnaise-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sardine-croquettes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-beans-on-sourdough",
"https://www.sainsburysmagazine.co.uk/recipes/mains/5-spice-pork-with-water-chestnuts-and-greens",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-stilton-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sea-bass-with-parsley-and-anchovy-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-beef-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-satay-with-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-patties-with-honey-roast-butternut-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-cornbread-cobbler",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bolognese-pancakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pappardelle-with-rosemary-and-sausage-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-with-creamy-leeks-and-butter-beans",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-leek-carbonara",
"https://www.sainsburysmagazine.co.uk/recipes/mains/best-ever-lamb-casserole",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-orange-and-almond-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fondue-mac-and-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-and-halloumi-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/red-pepper-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-vegetable-and-pancetta-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sausage-and-parsnip-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ham-sweetcorn-and-cheese-bread-pudding",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rich-lamb-and-pork-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/meatball-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-garlic-thyme-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-sausage-gnocchi-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-lamb-and-sweet-potato-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-chicken-pie-with-chorizo-and-olives",
"https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-and-black-bean-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-apple-winter-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-fishcakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-pie-with-cheesy-rosti-topping",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-turkey-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fridge-forage-tex-mex-rice-with-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-mango-chicken-with-fragrant-pilau",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-lentil-and-halloumi-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-with-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-korma",
"https://www.sainsburysmagazine.co.uk/recipes/mains/super-green-halloumi-omelette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/walnut-and-red-pepper-stuffed-cabbage-leaves",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-with-fennel-and-orange-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-rogan-josh",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-shepherds-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-brisket-with-mexican-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pineapple-pulled-pork",
"https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rolled-turkey-breast-with-rosemary-and-cranberry-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-baked-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beetroot-and-apple-stuffing-loaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cauliflower-tart-with-thyme-and-pistachios",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herbed-mackerel-with-beetroot-puree-and-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/duck-stir-fry-with-plum-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-squash-with-pumpkin-seed-and-cranberry-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-turkey-with-turkey-butter-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brussels-sprouts-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-crispy-duck-with-cranberry-glaze",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-gravlax-pasta-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-cauliflower-sage-and-walnut-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/irish-lamb-ragout",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-cooked-chook-with-pea-and-parsley-dumplings",
"https://www.sainsburysmagazine.co.uk/recipes/mains/simple-sausage-and-rosemary-cassoulet",
"https://www.sainsburysmagazine.co.uk/recipes/mains/philadelphia-style-spiced-steak-naanwiches",
"https://www.sainsburysmagazine.co.uk/recipes/mains/stilton-mac-and-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/somerset-pork-with-apples-and-cider",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-sweet-potato-rice-bowls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-cod-with-sweet-and-smoky-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-tomato-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-pie-potato-skins",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pearl-barley-risotto-with-blue-cheese-and-squash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-beef-brisket-with-cranberries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-citrus-and-spice-turkey",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-shawarma",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-and-onion-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/full-english-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coq-au-vin-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-spinach-dhansak",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-boursin-chicken-with-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mulled-beef-brisket",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bloody-mary-stuffed-potato-skins",
"https://www.sainsburysmagazine.co.uk/recipes/mains/malaysian-sweet-potato-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-and-beet-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hake-with-samphire-and-malai-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sardine-and-tomato-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-and-beer-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-melt-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chickpea-and-spinach-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-paprika-goulash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-and-broccoli-pizzette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-duck-breast-with-couscous-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-broad-beans-mint-and-ricotta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-sweet-potato-lentils-and-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sri-lankan-chicken-and-chickpea-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-cider-and-parsnip-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-shells-with-sardines-fennel-and-saffron",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-masala-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-stroganoff",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-and-ham-hash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-meatball-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-honey-baked-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-onion-soup-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/minced-beef-tatties",
"https://www.sainsburysmagazine.co.uk/recipes/mains/nutty-sweet-potato-and-lime-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-walnut-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-beef-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-mustardy-lentils-beetroot-and-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-shepherds-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/wild-mushroom-speltotto-with-pine-nuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/healthier-bangers-and-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-quesadillas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-lemon-and-honey-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-sausage-and-crispy-potato-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/indian-spiced-lamb-chops-with-mango-and-cucumber-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-fish-and-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/autumn-couscous-with-roast-squash-and-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/battered-haddock-and-mushy-pea-fishcakes-with-homemade-tartare-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-madras-sandwich",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-herb-and-golden-ale-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ham-egg-and-chip-tortilla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-tagliatelle-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-salmon-and-summer-veg-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zingy-courgette-and-basil-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-edamame-and-spring-onion-omelette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-stuffed-aubergine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hariyali-sea-bream",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-mozzarella-melts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-mezze-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fennel-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/store-cupboard-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-indian-lamb-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-pea-and-mushroom-tikka-masala",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-feta-and-pesto-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-chicken-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-kale-and-butter-bean-one-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-greens-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-mac-and-greens",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gammon-with-mustard-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-lentil-nut-roast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/green-veg-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-style-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tikka-spiced-cauliflower",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-tomato-tagliatelle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegan-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coffee-and-rum-marinated-steaks",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-bianca-with-broccoli-and-garlic",
"https://www.sainsburysmagazine.co.uk/recipes/mains/margherita-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-baked-eggs-with-peperonata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-roasted-spinach-polpette-and-tomatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salt-baked-salmon-with-watercress-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-cajun-grilled-corn-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/super-moist-roast-chicken-with-lemon-and-thyme",
"https://www.sainsburysmagazine.co.uk/recipes/mains/watercress-pesto-and-lemon-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/easy-chicken-shawarma",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aloo-gosht",
"https://www.sainsburysmagazine.co.uk/recipes/mains/traybaked-teriyaki-mackerel-with-greens",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parma-ham-wellingtons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-crayfish-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fish-parcels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/oven-roasted-chicken-and-avocado-with-asparagus",
"https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-pasta-with-lemon-avocado-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/stacked-sweet-potato-burgers-with-avocado-lime-cream",
"https://www.sainsburysmagazine.co.uk/recipes/mains/avocado-stuffed-baked-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-carbonara",
"https://www.sainsburysmagazine.co.uk/recipes/mains/blt-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mango-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tamarind-prawns-with-cinnamon-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/antipasti-tartlets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/peri-peri-steak-and-roasted-new-potato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-steak-with-tomato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-smoked-salmon-and-hollandaise-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/balsamic-onion-goats-cheese-and-rocket-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-finger-sarnies-with-fennel-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/radish-spring-onion-and-watercress-salad-with-baked-trout",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-roast-chicken-with-root-vegetable-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/giant-sharing-bacon-jam-rosti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-filo-parcel",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tenderstem-and-goats-cheese-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/persian-herb-rice-with-broad-beans",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chorizo-and-mixed-grain-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mackerel-kedgeree",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-fish-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smashed-salmon-hash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-peanut-pork-with-tenderstem",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chickpea-cauliflower-and-squash-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/satay-beef-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-and-aubergine-with-minted-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/all-in-one-spanish-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle-meatball-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-pastilla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-pea-and-watercress-tagliatelle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-chicken-with-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/riojan-pork-chorizo-and-potato-pot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bacon-badger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-pork-with-sweet-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-diane",
"https://www.sainsburysmagazine.co.uk/recipes/mains/artichoke-spaghetti-with-romanesco",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dill-mustard-trout-with-smashed-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-stuffed-with-lemony-goats-cheese-and-tarragon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-chicken-with-cauliflower-couscous-and-chermoula-tomatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-smoked-salmon-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rosemary-steak-with-cheesy-polenta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-goats-cheese-and-prosciutto-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-madeira-and-chestnut-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/squash-stilton-and-kale-pies-with-walnut-pastry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sicilian-roast-chicken-with-fennel-and-blood-oranges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-meatballs-with-red-pepper-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-roast-chicken-with-tomato-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-lamb-with-quick-pickled-cucumber",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kale-carbonara",
"https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-and-spinach-tortelloni-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/whisky-venison-steaks-with-neeps-and-tatties",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mango-chipotle-rice-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/campfire-roast-fish-with-lemon-and-fennel",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-red-pork",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-style-crispy-duck-salad-with-blood-oranges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-prawn-noodle-jar",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leaner-chicken-cacciatore",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pernod-baked-mackerel-with-fennel-and-watercress",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-chicory-and-blue-cheese-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/venison-and-red-wine-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-salmon-with-sesame-noodles-and-cucumber-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/venison-sausage-and-squash-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-gorgonzola-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gnocchi-ham-and-pea-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-chicken-nachos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cauliflower-cheese-and-pepper-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-caesar-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sizzling-squash-with-herby-cucumber-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/noodles-with-chicken-and-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-giant-cous-cous-broth",
"https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-squash-and-walnut-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-squash-and-black-rice-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-sumac-and-pomegranate-soup-with-roasted-peanuts-coriander-and-lime",
"https://www.sainsburysmagazine.co.uk/recipes/mains/golden-coconut-and-cauliflower-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cumin-carrot-and-lamb-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-potato-and-ham-hash-with-sage-fried-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-turkey-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ultimate-beef-wellington",
"https://www.sainsburysmagazine.co.uk/recipes/mains/red-leicester-pickled-onion-and-watercress-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-sage-and-chestnut-wellington",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-tofu-with-miso-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-smoked-salmon-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-with-gewurztraminer-and-grapes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-prawn-and-coconut-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/umami-chicken-ramen",
"https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-cod-and-brussels-sprout-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-orange-chicken-with-broccoli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-gnocchi-and-turkey-hash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-corn-and-bean-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-spiced-lamb-pie-with-almonds-and-apricots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-chops-with-balsamic-onions-and-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/coconut-dhal-with-roasted-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tartiflette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-mac-n-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/colombian-style-chicken-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/scandi-style-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potatoes-stuffed-with-chickpeas-and-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/shepherds-sag-aloo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-biryani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/braised-fennel-with-figs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asian-chicken-and-ginger-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moussaka",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-style-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gammon-and-egg-with-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-lamb-skewers-with-herby-bulgur",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-schnitzel-with-rose-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-yogurt-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/massaman-curry-buddha-bowls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/macaroni-cheese-with-crispy-bacon-and-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chick-n-sours-korean-chicken-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-pesto-tortelloni-broth",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-spiced-potato-and-zhoug-pitta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/squid-and-mackerel-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-aubergine-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pizzadilla",
"https://www.sainsburysmagazine.co.uk/recipes/mains/feta-aubergine-pomegranate-and-harissa-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/watermelon-jelly-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheeseburger-nacho-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle-giant-fishcake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zaatar-potato-salad-with-olives-and-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-veg-spelt-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-spinach-egg-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-ginger-chicken-pho",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-sushi-salad-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sausage-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-korma",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-and-saffron-chicken-pitta-with-blackened-chilli-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-salmon-with-blueberry-lime-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-tagliatelle-with-roasted-red-pepper-cream-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/golden-vegetable-dhal-with-soft-boiled-eggs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rice-spinach-and-eggs-with-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pad-kra-pao",
"https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-ricotta-lemon-and-herbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salt-and-pepper-rib-of-beef-with-bloody-mary-sauce-and-roast-potato-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/shoyu-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-tuna-burgers-with-chilli-mango-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/charred-broccoli-borlotti-bean-and-hazelnut-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-cod-with-sesame-cucumbers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ginger-chicken-udon-noodles-2",
"https://www.sainsburysmagazine.co.uk/recipes/mains/broccoli-pine-nut-pesto-penne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/asparagus-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-tea-brined-chicken-with-warm-rice-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/blue-cheese-gnocchi-with-savoury-granola",
"https://www.sainsburysmagazine.co.uk/recipes/mains/whole-roasted-cauliflower",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-goats-cheese-risotto-mint-gremolata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-chickpea-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-chicken-and-shiitake-broth",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-pineapple-and-herb-roasted-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/venison-steaks-with-miso-parsnip-fries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-salmon-coconut-rice-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-marmalade-chicken-drumsticks-with-blue-cheese-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-and-manchego-potato-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/caribbean-lamb-hotpot-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chorizo-pie-saffron-pastry-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chermoula-cod-bulgar-salad-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-wonton-soup-one-pound-meals-miguel-barclay",
"https://www.sainsburysmagazine.co.uk/recipes/mains/garam-masala-chicken-spicy-potatoes-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-roast-chicken-with-cornbread-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tenderstem-and-ham-quiches",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-pork-patties-with-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-adobo-with-roast-squash-rice-and-smoky-tomato-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbacoa-beef-brisket-tacos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-goats-cheese-tart-recipe",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-butternut-with-honey-and-walnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-lamb-pilaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/yellow-duck-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/manchego-fritters-with-seville-orange-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/farc-on",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kedgeree-with-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tom-daley-s-harissa-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-shallot-and-mushroom-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/houmous-pepper-goats-cheese-and-sweet-chilli-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-and-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/higgledy-piggledy-greek-beef-pasta-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-tacos-with-smoky-coleslaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-schnitzel-with-lemon-and-caper-butter",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-cashew-stir-fried-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-sea-bass-with-ginger-and-spring-onion",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-chicken-bites-with-sweet-chilli-dip",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spaghetti-mac-n-cheese-cake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-chorizo-gumbo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/teriyaki-salmon-and-kale-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quick-smoked-haddock-and-cauliflower-chowder",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-black-bean-and-squash-burrito-bowl",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-salad-of-pan-fried-duck-with-clementines",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-beans-on-toast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/porky-pancetta-ragu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-beef-sirloin-with-juniper-and-black-pepper-2",
"https://www.sainsburysmagazine.co.uk/recipes/mains/savoury-parsnip-and-pecan-slice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-pearl-barley-risotto-with-feta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/honey-roast-salmon-and-broccoli-crustless-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/korean-barbecue-beef-stir-fry-with-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-style-chicken-and-fennel-braise",
"https://www.sainsburysmagazine.co.uk/recipes/mains/charred-cauliflower-and-coconut-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-crown-with-herb-butter",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cranberry-and-orange-glazed-ham",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-coulibiac",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spanish-roasted-cod-with-chorizo-and-persimon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jose-hot-dogs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-streetcorn-hot-dogs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-chop-root-veg-and-apple-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bengali-coconut-fish-pie-with-mustard-seed-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/satay-chicken-with-soba-and-cucumber-noodle-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/singapore-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-beef-pho",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beer-battered-tofu-tacos-with-a-chipotle-cashew-soured-cream",
"https://www.sainsburysmagazine.co.uk/recipes/mains/korean-style-stew-with-marinated-tofu-pieces",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-n-sour-bahn-mi-baguettes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quick-pickled-rice-noodle-salad-with-soy-glazed-griddled-tofu",
"https://www.sainsburysmagazine.co.uk/recipes/mains/naan-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-and-watercress-chilli-pesto-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/firecracker-prawn-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-sausage-and-squash-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-wrapped-chicken-with-creamy-lentils",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-chicken-and-mushroom-hot-pot-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-ham",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-liver-pancetta-tagliatelle-with-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kimchi-cheese-toastie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-easy-black-bean-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-chorizo-and-melting-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-sweet-potato-with-houmous-and-harissa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/white-bean-soup-with-candied-vanilla-bacon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pappardelle-with-prawns-and-vanilla-beurre-blanc",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-spiced-chicken-legs-roasted-with-lemons-tomatoes-and-celery",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cauliflower-pea-and-cashew-pilaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-tagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tom-yum-prawn-noodle-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-fish-stew-with-cheese-toasts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/butternut-noodles-with-spinach-and-ricotta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hot-dogs-with-new-york-streetcart-onions",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-sesame-pork-noodle-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ghastly-ghoul-ash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-pickled-veg-with-seared-tuna-and-herbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-duck-with-soft-herb-and-grapefruit-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herb-omelette-with-feta-and-broad-beans",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fresh-tomato-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/trout-with-broad-beans-and-courgettes-in-a-wine-and-cream-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-mustard-and-goats-cheese-galette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baharat-spiced-baked-aubergines-with-feta-fennel-and-spelt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-beef-nectarine-and-rocket-salad-with-thai-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/parma-ham-and-mozzarella-pockets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausages-with-spicy-charred-corn-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-glazed-sausages-and-onion-rings-2",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hot-dogs-with-celeriac-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tapanade-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-onion-topping",
"https://www.sainsburysmagazine.co.uk/recipes/mains/satay-noodle-lettuce-wraps-with-prawns-and-mango",
"https://www.sainsburysmagazine.co.uk/recipes/mains/middle-eastern-chicken-fatteh",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-smoked-mackerel-with-preserved-lemon-quinoa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hot-smoked-trout-and-beans-with-a-creamy-dill-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-garden-salad-with-peaches-and-mozzarella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-bean-burritos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cobb-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-soup-bowl-with-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-meze-pitta-pockets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/loaded-pulled-pork-subs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salad-of-roasted-beetroot-goat-s-cheese-and-hazelnuts-on-wild-rocket",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mac-n-cheese-pancetta-cups",
"https://www.sainsburysmagazine.co.uk/recipes/mains/zaatar-lamb-chops-with-salted-yogurt-sweet-herbs-and-pomegranate-molasses",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-rogan-josh-with-mint-and-coriander-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quick-cook-ancho-con-carne-with-cucumber-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-new-potatoes-cherry-tomatoes-and-basil",
"https://www.sainsburysmagazine.co.uk/recipes/mains/https-www-youtube-com-watch-v-gjbibr6dv1s",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-loin-steaks-in-cider-and-tarragon-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-salmon-and-puy-lentil-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hoisin-chicken-with-stir-fried-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-cheddar-and-mushroom-pasta-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/masala-meatball-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-mackerel-with-toasted-fennel-and-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/miso-marinated-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/burger-with-smoky-stilton-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-roast-chicken-with-watercress-and-spiced-peaches",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-and-pistachio-koftas-with-fennel-radish-and-apricot-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-tuna-nicoise-ratatouille-stack",
"https://www.sainsburysmagazine.co.uk/recipes/mains/porchetta-style-roast-pork-with-a-courgette-chickpea-and-spinach-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sesame-crusted-fish-with-ginger-pak-choi",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-squash-and-goats-cheese-salad-with-griddled-spring-onions-and-pumpkin-seed-crunch",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-pesto-and-pepper-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/charred-aubergine-and-tomato-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-style-chicken-and-olive-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/camembert-baked-with-maple-syrup-pecans-and-blueberries",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brie-and-beef-baguette-with-beetroot-jam",
"https://www.sainsburysmagazine.co.uk/recipes/mains/twice-cooked-goats-cheese-souffle-s-with-apple-and-walnut-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-cutlets-with-harissa-yogurt-and-midget-trees",
"https://www.sainsburysmagazine.co.uk/recipes/mains/steak-with-creamy-cavolo-nero-and-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bashed-chicken-with-walnut-and-watercress-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/summer-gnocchi-with-avocado-pesto-and-tomatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-fillet-with-asian-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vegetable-stuffed-picnic-rolls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sri-lankan-crab-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-maple-glazed-pork-chops",
"https://www.sainsburysmagazine.co.uk/recipes/mains/somerset-mussels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-poussins-with-roasties",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cote-de-boeuf-with-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-lemon-chicken-noodle-broth",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mattar-paneer-a-medium-curry-of-peas-tomatoes-and-indian-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-middle",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chickpea-burgers-in-roasted-portobello-buns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-cod-with-roasted-cauliflower-and-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-lettuce-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-lemon-garlic-and-thyme-chicken-kebabs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chermoula-lamb-chops-with-herb-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chicken-drumsticks-with-ginger-barbecue-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tarte-flambee-with-comte-and-watercress-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/anchovy-and-chilli-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mustardy-mushroom-stroganoff",
"https://www.sainsburysmagazine.co.uk/recipes/mains/citrus-crusted-salmon-with-buttered-leeks-and-roasted-new-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-and-leek-chicken-topped-with-green-pea-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-ricotta-meatballs-in-tomato-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gorgonzola-and-pear-salad-with-caramelised-walnuts-and-radicchio",
"https://www.sainsburysmagazine.co.uk/recipes/mains/florentine-fish-pie-with-smashed-potato-and-chives",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lightly-curried-haddock-pie-with-crunchy-mustard-seed-and-cardamom-crumbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fragrant-scallops-and-prawns-with-black-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/poussins-stuffed-with-spelt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-savoy-cabbage-with-emmental-and-breadcrumbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quinoa-salad-with-broccoli-preserved-lemon-and-avocado-oil",
"https://www.sainsburysmagazine.co.uk/recipes/mains/grilled-leek-and-egg-salad-with-mustard-and-caper-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/persian-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mozzarella-in-carrozza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-parmesan-with-rocket-and-new-potato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-mince-with-aromatic-rice-and-wilted-greens",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-turkey-mince",
"https://www.sainsburysmagazine.co.uk/recipes/mains/paneer-kathi-rolls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-bacon-and-kale-pies-with-celeriac-and-horseradish-rosti-topping",
"https://www.sainsburysmagazine.co.uk/recipes/mains/harissa-basil-and-feta-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cumin-chicken-with-giant-couscous-courgette-peas-and-mint",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-style-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-burgers-with-sweet-potato-wedges",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-smoked-salmon-and-cream-cheese-topping",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-avocado",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chive-crispbreads-with-goats-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/duck-green-beans-and-walnuts",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-bacon-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/piri-piri-rice-and-peas-with-garlic-prawns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pot-roasted-curried-chicken-with-garam-masala-mango-chutney-and-lemon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/orange-sticky-chicken-with-three-green-veg",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-garlic-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-paillards-with-fennel-honey-and-lemon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herb-crusted-sea-bass-with-toasted-coconut-and-tomato-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/jerk-fish-with-seaweed-salted-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-salmon-with-cauliflower-couscous-and-garlic-spinach",
"https://www.sainsburysmagazine.co.uk/recipes/mains/reviving-chicken-broth",
"https://www.sainsburysmagazine.co.uk/recipes/mains/deep-filled-crab-saffron-and-parmesan-tartlets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-marmalade-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/light-and-fragrant-fish-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/big-bowl-veggie-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-steak-with-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/haggis-stuffed-chicken-breasts-with-chilli-and-garlic-stir-fried-swede-and-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sichuan-roasted-red-pepper-soup-with-ve-spice-edamame-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/celeriac-soup-with-horseradish-lemon-and-parsley",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kale-pea-and-spring-onion-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-orange-and-quinoa-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-prawn-rice-noodle-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/melting-brie-and-ratatouille-pasta-gratin",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aromatic-oriental-chicken-parcels",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-breast-with-apricot-gammon-and-pistachio-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-venison-haunch-with-beetroot-shallots-and-horseradish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-guinea-fowl-with-prune-and-bacon-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-salmon-with-currants-and-ginger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/oven-baked-fish-and-chips",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-spiced-cranberry-pecan-stuffing-and-maple-glaze",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sage-turkey-burgers-with-cranberry-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-pumpkin-and-chickpea-korma-with-crispy-shallots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-kale-and-goats-cheese-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-and-leek-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-thyme-and-garlic-roasted-chicken-with-chestnut-stuffing-filled-red-onions",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-red-pepper-sundried-tomato-and-ricotta-cannelloni",
"https://www.sainsburysmagazine.co.uk/recipes/mains/top-each-toasted-crumpet-with-2-crispy-bacon-rashers-and-a-poached-egg-serve-with-the-roasted-vine-tomatoes-on-the-side-and-season-with-freshly-ground",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pizza-crumpet",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roasted-garlic-raita",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-spiced-duck-legs-with-plums",
"https://www.sainsburysmagazine.co.uk/recipes/mains/caribbean-vegetable-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-chicken-with-broccoli-and-spelt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-hoisin-turkey-meatball-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-rostis-with-fried-eggs-and-avocado-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-vegetable-minestrone",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken-and-lentil-pies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/duck-breasts-with-spiced-blackberry-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/vietnamese-roast-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-mozzarella-and-chive-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/best-ever-chicken-caesar-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/greek-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-bacon-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cajun-spiced-salmon-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hot-smoked-trout-new-potato-and-pea-shoot-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/30-minute-seafood-paella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tuna-aubergine-freekeh",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-pak-choi-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pasta-with-parma-ham-agrodolce-peppers-and-pangrattato",
"https://www.sainsburysmagazine.co.uk/recipes/mains/singapore-style-prawn-laksa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fast-roast-beef-with-thyme-beets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/the-ivy-shepherd-s-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-biryani",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-fennel-and-pepper-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/the-ultimate-veggie-burger-with-lemon-chipotle-mayo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ham-hock-broad-bean-and-new-potato-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-mackerel-fennel-and-orange-quinoa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-onion-and-bacon-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/kale-mushroom-and-gorgonzola-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seafood-stew-with-garlic-sourdough",
"https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-lamb-mint-and-tomato-couscous-wraps",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turmeric-socca-with-cucumber-pepper-and-tahini-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/aubergine-pepper-and-halloumi-kebabs-with-lemony-houmous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/persian-style-spatchcock-barbecued-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fig-gorgonzola-and-thyme-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-style-picnic-loaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-vegetable-pot-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/potato-mozzarella-and-spring-onion-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-chicken-drumsticks-with-salsa-and-griddled-apricots",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-lime-and-ginger-sea-bass",
"https://www.sainsburysmagazine.co.uk/recipes/mains/falafel-with-avocado-roasted-vegetables-and-tahini-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-beef-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-halloumi-kebabs-with-freekeh-orange-and-walnut-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-salmon-with-coconut-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-quesadillas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-pepper-arrabbiata-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chorizo-nicoise-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-fish-with-herby-potatoes-and-courgettes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/paprika-chicken-with-griddled-corn-and-lemon-mayo",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-pancetta-salad-with-raspberries-and-mustard-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/muhammara-lamb-cutlets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-panzanella-salad-with-deep-fried-mozzarella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/wholewheat-tagliatelle-with-kale-caramelised-onions-and-goats-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-avocado-couscous-with-harissa-yogurt",
"https://www.sainsburysmagazine.co.uk/recipes/mains/rigatoni-with-aubergine-tomato-feta-and-basil-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brioche-burger-buns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cod-baked-in-paper-with-basil-and-asparagus",
"https://www.sainsburysmagazine.co.uk/recipes/mains/courgette-lemon-and-mozzarella-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/japanese-chicken-coleslaw-with-ginger-almond-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/seared-beef-salad-with-carrot-noodles-and-tahini-dressing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crab-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chickpea-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-red-sweet-potato-red-pepper-and-coconut-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-red-onion-flatbreads",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-nicoise-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-pork-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-ed-kebabs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoky-pepper-and-onion-frittata-with-manchego",
"https://www.sainsburysmagazine.co.uk/recipes/mains/raw-thai-citrus-crunch-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/monkfish-chorizo-and-vegetable-paella-with-saffron-aioli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-lamb-chops-with-crushed-new-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spring-spanakopita",
"https://www.sainsburysmagazine.co.uk/recipes/mains/orange-roasted-chicken-spring-vegetables",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-tagine-with-preserved-lemon-and-olives",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pan-fried-gnocchi-with-watercress-mint-pesto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mediterranean-cod",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-duck-breast-with-plum-star-anise-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-piri-piri-sardines-with-tomato-and-onion-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/the-ultimate-crab-cakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-balls-with-courgetti-and-tomato-and-basil-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fish-and-spinach-curry-with-cauliflower-rice-and-pistachios",
"https://www.sainsburysmagazine.co.uk/recipes/mains/basil-tomato-and-aubergine-soba-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/soy-and-ginger-tuna-with-soba-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/halloumi-fattoush-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pad-thai-noodles-with-mango-and-crab",
"https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-prawns-with-mango-noodles-and-herbs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-chicken-stir-fry-with-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemon-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crab-tomato-and-parmesan-lattice-pies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-shiitake-mushroom-pho-with-carrot-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-fishcakes-with-tomato-chermoula-sauce-and-nutty-couscous",
"https://www.sainsburysmagazine.co.uk/recipes/mains/prawn-and-coconut-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dukkah-sweet-potato-and-spinach-dhal",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crumbed-katsu-style-chicken-with-cucumber-and-peanut-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-veggie-shepherd-pies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cider-chicken-and-bacon-pie-with-crushed-new-potato-topping",
"https://www.sainsburysmagazine.co.uk/recipes/mains/saag-paneer-with-cauliflower-rice",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-tomato-prawns-with-courgette-spaghetti",
"https://www.sainsburysmagazine.co.uk/recipes/mains/shredded-duck-and-watercress-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/one-pan-spanish-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sticky-sumac-chicken-with-watercress-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/purple-sprouting-broccoli-blue-cheese-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lemony-broccoli-and-anchovy-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-and-cheese-pancakes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/thai-style-duck-stir-fry-with-lime",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-haddock-macaroni-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-leek-and-mushroom-pot-pies",
"https://www.sainsburysmagazine.co.uk/recipes/mains/persian-chicken-with-prunes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/smoked-salmon-and-creme-fraiche-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mushroom-and-spinach-quinoa-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-moroccan-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkey-meatballs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/winter-chicken-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/gorgonzola-and-broccoli-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/mexican-salmon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-potato-and-aubergine-moussaka",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fast-chicken-butter-bean-and-tomato-stew",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-romesco-goats-cheese-pasta",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-cranberry-stuffing-and-pigs-in-blankets",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-sage-chestnut-and-apple-stuffing-balls-honey-mustard-chipolata-rolls-and-marsala-gravy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beef-en-croute",
"https://www.sainsburysmagazine.co.uk/recipes/mains/porcini-risotto-with-roast-garlic",
"https://www.sainsburysmagazine.co.uk/recipes/mains/moroccan-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-ribs-of-beef-with-roasted-red-onions-and-watercress-and-stilton-relish",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-breasts-with-date-and-pine-nut-quinoa-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/parmesan-crumbed-pork-with-roasted-tomatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-gnocchi-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pepper-feta-and-smoked-paprika-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-mushroom-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/brazil-nut-and-chestnut-roast",
"https://www.sainsburysmagazine.co.uk/recipes/mains/three-cheese-roulade",
"https://www.sainsburysmagazine.co.uk/recipes/mains/three-pepper-steaks",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-cacciatore",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lamb-with-artichokes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crab-tortillas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/puttanesca",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sweet-chilli-burgers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-schnitzelc",
"https://www.sainsburysmagazine.co.uk/recipes/mains/macaro-no-cheese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sausage-onepot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheats-beef-massaman",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pumpkin-pecorino-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/two-cheese-spinach-and-potato-bake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-aubergine-quinoa-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/griddled-chicken-with-warm-courgette-and-fennel-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fresh-tomato-chickpea-and-spinach-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/the-ritz-game-pie",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-con-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/speedy-vegetable-tortilla-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/apple-pear-and-stilton-soup-with-chorizo-and-croutons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lambtagine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/easy-roast-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/red-onion-bangers-with-kale-mash",
"https://www.sainsburysmagazine.co.uk/recipes/mains/leek-and-chorizo-frittata",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-paneer",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sunshine-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/quick-bolognese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-onepot",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salmon-prawn-and-broad-bean-pilaf",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crumbed-kaffir-lime-chicken-with-lime-and-ginger-mayonnaise",
"https://www.sainsburysmagazine.co.uk/recipes/mains/veggie-paella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/turkish-stuffed-aubergines",
"https://www.sainsburysmagazine.co.uk/recipes/mains/lentil-and-tomato-bolognese",
"https://www.sainsburysmagazine.co.uk/recipes/mains/barbecued-chicken-in-brioche-buns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bean-ratatouille",
"https://www.sainsburysmagazine.co.uk/recipes/mains/dairy-free-fish-pie-with-crispy-kale",
"https://www.sainsburysmagazine.co.uk/recipes/mains/warm-butter-bean-chicken-and-chorizo-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/herbed-chicken-drumsticks",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-jerk-chicken-skewers-with-brown-rice-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/anjum-anand-s-indian-style-chicken-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/goats-cheese-and-pepper-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/oven-baked-prawn-pea-and-rocket-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-tikka-kebabs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/beetroot-goats-cheese-and-hazelnut-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheese-and-sausage-baguette",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cheesy-quesadillas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/creamy-courgette-and-pea-quiche",
"https://www.sainsburysmagazine.co.uk/recipes/mains/fast-roast-chicken-with-asparagus",
"https://www.sainsburysmagazine.co.uk/recipes/mains/classic-cheese-and-bacon-burger",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-thai-pork-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/bbq-sausages-and-goats-cheese-hotdogs",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-and-watercress-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/baked-chicken-with-olives-and-jalapenos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/poached-trout-with-watercress-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/duck-breasts-with-a-piquant-lime-and-ginger-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/salsa-verde-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/padthai",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cannelloni-with-balsamic-glazed-tomatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/purple-sprouting-broccoli-and-parma-ham-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/croque-monsieur",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pork-fillet-with-smoked-paprika",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-chicken-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chicken-chow-mein",
"https://www.sainsburysmagazine.co.uk/recipes/mains/individual-venison-wellingtons",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-chicken-soup",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-pheasant-with-grapes-and-sweetheart-cabbage",
"https://www.sainsburysmagazine.co.uk/recipes/mains/classic-roast-turkey-with-chestnut-stuffing-balls",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spinach-wensleydale-and-toasted-hazelnut-wellington",
"https://www.sainsburysmagazine.co.uk/recipes/mains/onion-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/french-goat-s-cheese-ham-pepper-and-red-onion-pizza",
"https://www.sainsburysmagazine.co.uk/recipes/mains/slow-roast-lamb-with-feta-peppers-and-dill",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chilli-and-garlic-tuna-stir-fry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tagliatelle-with-prawns-parma-ham-and-parsley",
"https://www.sainsburysmagazine.co.uk/recipes/mains/john-west-tuna-greek-style-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-rosemary-pearl-barley-risotto",
"https://www.sainsburysmagazine.co.uk/recipes/mains/john-west-salmon-and-cheddar-grills-2",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tomato-and-goats-cheese-tart",
"https://www.sainsburysmagazine.co.uk/recipes/mains/ratatouille-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/linguine-with-turkey-meatballs-and-roasted-peppers",
"https://www.sainsburysmagazine.co.uk/recipes/mains/langoustine-and-chicken-paella",
"https://www.sainsburysmagazine.co.uk/recipes/mains/italian-stuffed-roast-pork",
"https://www.sainsburysmagazine.co.uk/recipes/mains/green-chicken-curry-with-noodles",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tom-kerridge-slow-roast-lamb-with-boulangere-potatoes",
"https://www.sainsburysmagazine.co.uk/recipes/mains/crispy-cod-burritos",
"https://www.sainsburysmagazine.co.uk/recipes/mains/hairy-bikers-massaman-duck-curry",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spicy-tomato-and-chilli-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/tandoori-prawn-pockets-with-tomato-salsa",
"https://www.sainsburysmagazine.co.uk/recipes/mains/oxtail-stew-with-pumpkin-and-cinnamon",
"https://www.sainsburysmagazine.co.uk/recipes/mains/cumberland-glazed-roast-lamb-shoulder",
"https://www.sainsburysmagazine.co.uk/recipes/mains/one-pot-moroccan-chicken",
"https://www.sainsburysmagazine.co.uk/recipes/mains/roast-turkey-with-herb-and-pistachio-stuffing",
"https://www.sainsburysmagazine.co.uk/recipes/mains/chinese-style-crispy-duck-with-plum-sauce",
"https://www.sainsburysmagazine.co.uk/recipes/mains/skinny-roasted-tomato-aubergine-and-chickpea-lasagne",
"https://www.sainsburysmagazine.co.uk/recipes/mains/pulled-pork-buns",
"https://www.sainsburysmagazine.co.uk/recipes/mains/spiced-squash-and-prawns-with-coconut-and-chilli",
"https://www.sainsburysmagazine.co.uk/recipes/mains/easy-teriyaki-chicken-traybake",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-festive-roast-pork",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-slow-cooker-pork-carnitas",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chicken-with-bacon-and-mushrooms",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-spicy-garlic-prawn-and-rose-linguine",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-chinese-style-pork-rolls-with-quick-pickled-slaw",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-orange-fennel-and-burrata-salad",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-herby-roast-chicken-with-crispy",
"https://www.sainsburysmagazine.co.uk/recipes/mains/sponsored-rose-harissa-cauli-flatbreads-with-butter-bean-humous"]
        


recipes = []
for url in urls:
    recipe_data = scrape_recipe(url)
    if recipe_data:
        recipes.append(recipe_data)

for recipe in recipes:
    print(f"Title: {recipe['title']}")
    print("Ingredients:")
    for ingredient in recipe['ingredients']:
        print(f"- {ingredient}")
    print("Instructions:")
    for instruction in recipe['instructions']:
        print(f"1. {instruction}")
    print()

KeyboardInterrupt: 